编辑人:苏则茜
Project:Prob-Na-Learning
+ WBI行为数据分析：时间轴对齐，运动参数提取
+ 数据裁剪，数据拼接

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib as mpl
import os
import csv
import cv2
import heapq
from scipy.ndimage import label
import matplotlib.colors as mcolors
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
from scipy.ndimage import grey_opening,grey_closing
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
from datetime import datetime
from pathlib import Path
from matplotlib.lines import Line2D
import networkx as nx
from scipy.interpolate import interp1d
from scipy import ndimage
from scipy.interpolate import splprep, splev
from scipy.signal import savgol_filter
from scipy.ndimage import gaussian_filter1d
# 设置字体为 SimHei（黑体）
plt.rcParams['font.sans-serif'] = ['SimHei']  # 设置字体为 SimHei
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

# Preprocessing function definition

## Coordination Alignment 

In [ ]:
def Realign_coordinate(p_f, df_motion): 
    '''
    不同琼脂垫空间对齐
    2025.2.27：目前stage calibrate设置原点在右下角，这个代码将原点根据四个标定点重新设为左上角。
    另外根据x轴轨迹做镜面翻转
    '''
    
    # 初始化
    df_ul = pd.DataFrame({})
    df_ur = pd.DataFrame({})
    df_ll = pd.DataFrame({})
    df_lr = pd.DataFrame({})
    files = [f for f in os.listdir(p_f) if '.txt' in f ]
    
    # 找到四个顶点的坐标
    for f in files:
        if 'upper_left' in f:
            # 运动参数位置文件
            column_names = [ 'X', 'Y']
            df_ul = pd.read_csv(os.path.join(p_f, f),sep=r',', header=None, names=column_names)
            if len(df_ul) == 1:
                df_ul.index = ['upper_left']
            else:
                print('upper_left标记数量不对')
        if 'upper_right' in f:
            # 运动参数位置文件
            column_names = [ 'X', 'Y']
            df_ur = pd.read_csv(os.path.join(p_f, f),sep=r',', header=None, names=column_names)
            if len(df_ur) == 1:
                df_ur.index = ['upper_right']
            else:
                print('upper_right标记数量不对')
        if ('lower_left' in f) :
            # 运动参数位置文件
            column_names = [ 'X', 'Y']
            df_ll = pd.read_csv(os.path.join(p_f, f),sep=r',', header=None, names=column_names)
            if len(df_ll) == 1:
                df_ll.index = ['lower_left']
            else:
                print('lower_left标记数量不对')
        if 'lower_right' in f:
            # 运动参数位置文件
            column_names = [ 'X', 'Y']
            df_lr = pd.read_csv(os.path.join(p_f, f),sep=r',', header=None, names=column_names)
            if len(df_lr) == 1:
                df_lr.index = ['lower_right']
            else:
                print('lower_right标记数量不对')

    df_label = pd.concat([df_ul, df_ll, df_ur, df_lr])
    df_label['X'] = df_label['X'].astype(float)/1000
    df_label['Y'] = df_label['Y'].astype(float)/1000
    # 调整x坐标(平移y轴)
    x_min = (df_label.loc['upper_left','X'] + df_label.loc['lower_left','X'])/2
    x_max = (df_label.loc['upper_right','X'] + df_label.loc['lower_right','X'])/2
    if x_min > x_max:
        # 如果原点在高钠一侧，则需要减去left的坐标后将X坐标值变号
        # 目前的情况
        df_motion_r = df_motion.copy()
        df_motion_r['X'] = df_motion_r['X']-x_min
        df_motion_r['X'] = -df_motion_r['X']
    else:
        df_motion_r = df_motion.copy()
        df_motion_r['X'] = df_motion_r['X']-x_min
        
    # 调整y轴坐标，平移x轴  (另外y轴会和原坐标翻转)
    y_min = (df_label.loc['lower_left','Y'] + df_label.loc['lower_right','Y'])/2
    y_max = (df_label.loc['upper_left','Y'] + df_label.loc['upper_right','Y'])/2
    
    if y_min > y_max:
        df_motion_r1 = df_motion_r.copy()
        df_motion_r1['Y'] = df_motion_r1['Y']-y_max
    else:
        # 目前的情况
        df_motion_r1 = df_motion_r.copy()
        df_motion_r1['Y'] = df_motion_r1['Y']-y_max
        df_motion_r1['Y'] = -df_motion_r1['Y']
    return df_label, df_motion_r1

In [ ]:
def Realign_coordinate_false(p_f, df_motion): 
    '''
     手动设置四个角的坐标
    '''
    upper_left = [64000, 32000]
    upper_right = [24000, 32000]
    lower_left = [64000, 0]
    df_label['X'] = df_label['X'].astype(float)/1000
    df_label['Y'] = df_label['Y'].astype(float)/1000
    # 调整x坐标(平移y轴)
    x_min = upper_left[0]
    x_max = upper_right[0]
    if x_min > x_max:
        # 如果原点在高钠一侧，则需要减去left的坐标后将X坐标值变号
        df_motion_r = df_motion.copy()
        df_motion_r['X'] = df_motion_r['X']-x_min
        df_motion_r['X'] = -df_motion_r['X']
    else:
        df_motion_r = df_motion.copy()
        df_motion_r['X'] = df_motion_r['X']-x_min
        
    # 调整y轴坐标，平移x轴  (另外y轴会和原坐标翻转)
    y_min = lower_left[1]
    y_max = upper_left[1]
    
    if y_min > y_max:
        df_motion_r1 = df_motion_r.copy()
        df_motion_r1['Y'] = df_motion_r1['Y']-y_max
    else:
        df_motion_r1 = df_motion_r.copy()
        df_motion_r1['Y'] = df_motion_r1['Y']-y_max
        df_motion_r1['Y'] = -df_motion_r1['Y']
    return df_label, df_motion_r1

In [ ]:
def Realign_coordinate_by_edge(p_f, df_motion):
    '''
    旧数据标记edge0和edge45，需要判断0和4.5哪一个在左侧，然后将x轴原点设置在0g边缘，且正方向与梯度相同
    只修改x轴，不改y轴数据
    '''
    # 初始化
    df_ul = pd.DataFrame({})
    df_ur = pd.DataFrame({})
    df_ll = pd.DataFrame({})
    df_lr = pd.DataFrame({})
    files = [f for f in os.listdir(p_f) if ('.txt' in f) & ('edge' in f) ]
    # 找到四个顶点的坐标
    for f in files:
        if 'edge0' in f:
            edge0 = np.loadtxt(os.path.join(p_f, f),delimiter=',')
            edge0_x = edge0[:,0]
        if 'edge45' in f:
            edge45 = np.loadtxt(os.path.join(p_f, f),delimiter=',')
            edge45_x = edge45[:,0]
    rand_seq_0 = np.random.randint(0, 2, size=len(edge0_x))
    mean_x0 = np.mean(edge0_x[rand_seq_0==1])/1000
    rand_seq_45 = np.random.randint(0, 2, size=len(edge45_x))
    mean_x45 = np.mean(edge45_x[rand_seq_45==1])/1000
    print(f'edge0的x平均坐标为{mean_x0},edge45的x平均坐标为{mean_x45}')
    if mean_x0 > mean_x45:
        print("0的坐标更大，说明需要在减去0边界的x坐标后完成坐标轴转换")
        df_motion_r = df_motion.copy()
        df_motion_r['X'] = df_motion_r['X']-mean_x0
        df_motion_r['X'] = -df_motion_r['X']
    else:
        print("不需要坐标轴转换，只需要将原点平移到edge0")
        df_motion_r = df_motion.copy()
        df_motion_r['X'] = df_motion_r['X']-mean_x0
    
    # 作图验证
    plt.scatter(df_motion_r['X'], df_motion_r['Y'], s=0.7, c=df_motion_r['Time'], cmap='Reds')
    plt.title(os.path.basename(p_f))
    plt.show()
    
    return df_motion_r
    

In [ ]:
# p_f = r'Z:\data space+\C. elegans chemotaxis\2025_WBI\good_WBI\no_binary\20250116_4.5g-ov_05'
# df_stage = pd.read_csv(os.path.join(p_f, "stage_data.txt"),sep=r'\s+', header=None,names=column_names)
# columns_to_clean = ['Time', 'p_X', 'p_Y', 'X', 'Y']
# for col in columns_to_clean:
#     df_stage[col] = df_stage[col].astype(str).str.replace(',', '', regex=False).astype(float)
# df_motion = df_stage[['Time','X','Y']]/1000  # 转换单位为s和mm

# Realign_coordinate_by_edge(p_f, df_motion)

## Visualization Functions

In [ ]:
def cut_and_draw(df, n, alg_spd = 1, CTX = 0, speed = 0,turn=1, reorientation=1):
    # 将轨迹分段可视化
    df_idx_list = np.arange(0, len(df), n)   # 切数据
    for i, idx in enumerate(df_idx_list[:-1]):
        df_d = df.loc[idx: df_idx_list[i+1], :]
        plt.figure()
        plt.scatter(df_d.X,df_d.Y, s=0.8,c = df_d.Time, cmap='jet')
        ax = plt.gca()
        plt.title(f'idx{idx}')
        plt.colorbar(label='Time(s)')
        ax.set_aspect(1)
        plt.show()
        
        if alg_spd:
            #画角速度
            fig, ax = plt.subplots(1,2)
            ax1 = ax[0].scatter(df_d.X,df_d.Y, s=0.8,c = df_d.agl_speed, cmap='jet')
            plt.title(f'idx{idx} agl_speed')
            plt.colorbar(ax1, label='Time(s)')
            ax[0].set_aspect(1)
            ax[1].hist(df_d.agl_speed)
            plt.show()
        if CTX:
            # 画CTX
            fig, ax = plt.subplots(1,2)
            ax1 = ax[0].scatter(df_d.X,df_d.Y, s=0.8,c = df_d.CTX_left, cmap='jet')
            plt.title(f'idx{idx} CTX')
            plt.colorbar(ax1, label='Time(s)')
            ax[0].set_aspect(1)
            ax[1].hist(df_d.CTX_left)
            ax[1].set_title('CTX_left distribution')
            plt.show()
        if speed:
            # 画速度
            fig, ax = plt.subplots(1,2)
            ax1 = ax[0].scatter(df_d.X,df_d.Y, s=0.8,c = df_d.speed, cmap='jet')
            plt.title(f'idx{idx} agl_speed')
            plt.colorbar(ax1,label='Time(s)')
            ax[0].set_aspect(1)
            ax[1].hist(df_d.speed)
            ax[1].set_title('speed distribution')
            plt.show()
        if reorientation:
            fig, ax = plt.subplots(1,2)
            ax[1].scatter(df_d.X, df_d.Y, c = 'grey',s=0.8, alpha=0.05)
            ax[0].scatter(df_d.X, df_d.Y, c = 'grey',s=0.8, alpha=0.05)
            df_pt = df_d[df_d.Reorientation==1]
            df_tn = df_d[df_d.Event==1]
            ax[0].scatter(df_tn.X, df_tn.Y, s = 0.8, c='r')
            ax[1].scatter(df_pt.X, df_pt.Y, s = 0.8, c='r')
            ax[0].set_title('Sharp turn')
            ax[1].set_title('Reorientation')
            plt.show()

In [ ]:
def Scatter_Pos_Grad_Dir(df, Pos_CTX_vec, scale_size = 2, folder = '', x_lim=[]):
    fig,ax = plt.subplots(1,1)
    
    mean_x = df.X.mean()
    mean_y = df.Y.mean()
    origin = [mean_x, mean_y]
    vector = np.array(Pos_CTX_vec)*scale_size
    
    plt.quiver(*origin, *vector, angles='xy', scale_units='xy',
               scale=1, color='k', label='Pos_CTX_vec')
#     # 在指定位置标注向量值
#     vector_ctr = np.array([mean_x+0.5*vector[0], mean_y+0.5*vector[0]])
#     print(vector_ctr)
#     position = vector_ctr + np.array([0, 1])
#     print('Position', position)
#     plt.text(position[0], position[1], f"{Pos_CTX_vec}", fontsize=8, color='k')
    plt.legend()
    plt.scatter(df.X,df.Y, s=0.8,c = df.Time, cmap='jet')
    ax = plt.gca()
    plt.colorbar(label='Time(s)')
    
    if len(x_lim):
        plt.xlim(*x_lim)
    ax.set_aspect(1)
    if folder:
        fig.savefig(folder+'Pos-Grad-Dir.png')
    else:
        plt.show()

In [ ]:
def Scatter_Pirouette(df, folder = '', x_lim = []):
    fig,ax = plt.subplots(1,1)
    plt.scatter(df.X, df.Y, c = 'grey',s=0.8, alpha=0.05)
    df_pt = df[df.Reorientation==1]
    plt.scatter(df_pt.X, df_pt.Y, s = 0.8, c='r')
    plt.title('Reorientation')
    if len(x_lim):
        plt.xlim(*x_lim)
    ax.set_aspect(1)
    if folder:
        fig.savefig(folder+'Visual_Reorientation.png')
    else:
        plt.show()

In [ ]:
def map_color_dict(key_items,color_map='tab10'):
    '''
    # 提前生成统一的color_dict()
    # color(根据dates元素的数量分配，输出以key_items元素(可以是condition或dates)为key的颜色映射字典)
    '''
    
    color_vec_norm = np.linspace(0.01,0.99,len(key_items)+1)
    colormap = mpl.colormaps[color_map]
    colors  = colormap(color_vec_norm)
    color_dict = {}
    for i, d in enumerate(key_items):
        color_dict[d] = colors[i]
    return color_dict

In [ ]:
def set_time_intervals(df, time_len, time_inv, time_col='Vol_Time'):
    """
    设置时间间隔并为 DataFrame 添加时间段标签。
    时间列单位为s，不需要frame_rate

    Parameters:
    - df: 输入 DataFrame，需包含 'Timestamp' 列。
    - time_len: 总时间长度（小时）。
    - time_inv: 时间间隔（分钟）。

    Returns:
    - 更新后的 DataFrame，添加了 'Period_label' 列。
    - 时间段标签列表。
    """
    periods = np.arange(0, time_len + 1, time_inv)
    labels = [(periods[i] + periods[i + 1]) / 2 for i in range(len(periods) - 1)]
    df['Period_label'] = pd.cut(
        df[time_col], bins=periods, labels=labels, right=False, include_lowest=True
    )
    return df, labels

## Motion Parameter Extract Functions

### 角度计算  
  Pirouette labeling(outdated)

In [ ]:
# 定义角度计算公式
def ang_cal(vec_1,vec_2):
    dot_pro = np.dot(vec_1, vec_2)
    mod_1 = np.sqrt(np.dot(vec_1,vec_1))
    mod_2 = np.sqrt(np.dot(vec_2,vec_2)) 
    if mod_1 == 0 or mod_2 ==0:
        angle = 0
    else:
        cos = dot_pro/(mod_1*mod_2)
        if np.isnan(cos) == True:
            angle = np.nan
        else:  
            angle = np.arccos(round(cos,1))  #弧度制
    return angle

def clws_delta_phi(vec_1, vec_2, vec_0 = [1,0]):
    # vec_1 is the first vector, and it rotates to the vec_2
    vec_0 = np.array(vec_0)
    agl_1 = ang_cal(vec_1,vec_2)
    agl_2 = ang_cal(vec_0, vec_1)
    agl_3 = ang_cal(vec_0, vec_2)
    if agl_1 == 0:
        agl_1 = 0
        # print('angle = 0°')
    elif agl_1 == np.pi:
        agl_1 = np.pi
        # print('angle = 180°')
    elif vec_1[1] >= 0 and vec_2[1] >= 0:    #同时在第一第二象限
        if (agl_3 - agl_2) > 0:
            agl_1 = (-1)*np.abs(agl_1)
        else:
            agl_1 = np.abs(agl_1)
    elif vec_1[1] <= 0 and vec_2[1] <= 0:    #同时在第三第四象限
        if (agl_3 - agl_2) > 0:
            agl_1 = np.abs(agl_1)
        else:
            agl_1 = (-1)*np.abs(agl_1)
    elif vec_1[1] >= 0 and vec_2[1] <= 0:
        if (agl_3 + agl_2) > np.pi:
            agl_1 = (-1)*np.abs(agl_1)
        else:                                  #如果等于180°认为是顺时针旋转
            agl_1 = np.abs(agl_1)
    elif vec_1[1] <= 0 and vec_2[1] >= 0:
        if (agl_3 + agl_2) < np.pi:
            agl_1 = (-1)*np.abs(agl_1)
        else:                                  #如果等于180°认为是顺时针旋转
            agl_1 = np.abs(agl_1)   
    return agl_1*180/np.pi

In [ ]:
# def Label_Pirouettes_by_id(g_df,t_crit=6.16, frame_rate = 30,min_agl_spd=None):
#     '''根据角速度和turn的间隔标记Pirouette'''
#     # 输入：同一个id的df, t_crit(sec)，如果需要重新分turn可以输入min_agl_spd
#     # 注意输入的df中不能是不同组混合的，id必须独立
#     t_f_crit = t_crit*frame_rate
#     if min_agl_spd:
#         g_df.loc[:,'Event']=0
#         g_df.loc[g_df.agl_speed >= min_agl_spd, 'Event']=1
        
#     g_df.loc[:,'Reorientation']=g_df['Event']
#     g_df = g_df.reset_index(drop=True)
#     # nan值是计算速度和角速度时舍弃的点，默认是0不计算c
#     event_vec = g_df.Event.fillna(0).values

#     # 向后移动一位，上下相减
#     diff_vec = event_vec[1:]-event_vec[:-1]
#     if (diff_vec==0).all():
#         pass
#     elif ~(diff_vec==-1).any() or ~(diff_vec==1).any():
#         print('只有结尾有turn或只有开头有turn')
#     else:
#         # 统计所有出现1和-1的索引
#         idx_1 = np.where(diff_vec == 1)[0]
#         idx_n1 = np.where(diff_vec == -1)[0]
#         if np.min(idx_1)<np.min(idx_n1):
#             # 如果最小的索引是1而不是-1，说明开头是run，然后才开始出现turn
#             if len(idx_1)>len(idx_n1):
#                 # 如果1的数量大于-1,说明最后一个turn没有结束
# #                 print(f'验证最后是否以turn结束:{(np.max(idx_n1)<np.max(idx_1))}')
#                 # 去掉第一个1的索引，用1减去所有-1的索引，判断长度
#                 run_lens_btw_turns = idx_1[1:]-idx_n1
#                 for i,r in enumerate(run_lens_btw_turns):
#                     if r <= t_f_crit:
#                         low_b = idx_n1[i]+1
#                         upper_b = idx_1[i+1]+1      # 注意idx_1去头，所以索引内加以
#                         g_df.loc[low_b:upper_b, 'Reorientation']=1

#             elif len(idx_1)==len(idx_n1):
#                 # 如果两者数量相同，说明最后以run结尾，这条轨迹前后是完整的多个turn
# #                 print(f'验证最后是否以run结束:{(np.max(idx_n1)>np.max(idx_1))}')
#                 # 去掉1的所索引的首部和-1的尾部，用1减去-1的索引判断长度
#                 run_lens_btw_turns = idx_1[1:]-idx_n1[:-1]
#                 for i,r in enumerate(run_lens_btw_turns):
#                     if r <= t_f_crit:
#                         low_b = idx_n1[i]+1
#                         upper_b = idx_1[i+1]+1
#                         g_df.loc[low_b:upper_b, 'Reorientation']=1

#             # 无论是上述哪种情况，turn的数量都记为出现1的数量
#             num_turn = len(idx_1)
# #             print(f'Turn number: {num_turn}')
#         elif np.min(idx_1)>np.min(idx_n1):
#             # 如果最小索引是-1而不是1，说明开头就是turn
#             if (len(idx_n1)== len(idx_1)+1):
#                 # 如果-1比1多一个，说明以run结尾
# #                 print(f'验证最后是否以run结束:{(np.max(idx_n1)>np.max(idx_1))}')
#                 # 去掉最后一个-1的索引，用1减去-1
#                 run_lens_btw_turns = idx_1-idx_n1[:-1]
#                 for i,r in enumerate(run_lens_btw_turns):
#                     if r <= t_f_crit:
#                         low_b = idx_n1[i]+1
#                         upper_b = idx_1[i]+1
#                         g_df.loc[low_b:upper_b, 'Reorientation']=1
#             elif (len(idx_n1) == len(idx_1)):
#                 # 如果-1和1一样多，说明以turn结尾
# #                 print(f'验证最后是否以turn结束:{(np.max(idx_n1)<np.max(idx_1))}')
#                 # 直接用1的索引减去-1
#                 run_lens_btw_turns = idx_1-idx_n1
#                 for i,r in enumerate(run_lens_btw_turns):
#                     if r <= t_f_crit:
#                         low_b = idx_n1[i]+1
#                         upper_b = idx_1[i]+1
#                         g_df.loc[low_b:upper_b, 'Reorientation']=1
#     return g_df

In [ ]:
# def Insert_SharpTurn(df_als, min_agl = 50, max_agl = 110):
#     print('=======================开始事件分类===================================')
#     print('注：事件分类仅完成分类（turn = 1, run = 0），tunrning rate的计算较为灵活，在汇总分析作图时使用')
#     # 新建一列为Event
#     if 'Event' in df_als:
#         df_als = df_als.drop('Event', axis = 1)
#     df_als['Event'] = np.nan
#     # 选择角速度区间
#     # sharp turn 标记为1
#     df_als.loc[(np.abs(df_als.agl_velocity)>= min_agl)&(np.abs(df_als.agl_velocity) <= max_agl), 'Event'] = 1  
#     # run 标记为0
#     df_als.loc[(np.abs(df_als.agl_velocity) < min_agl), 'Event'] = 0
#     return df_als

### 计算连续运动参数变量:速度，CTX

In [ ]:
# 速度，角速度，bearing angle, Dist_to_center, ctx计算函数
# 输入：筛选为虫子的csv文件，画圆的csv文件，ID索引，速度平滑，角速度速度平滑，角速度平滑，追踪帧率，跳帧数量，像素长度
def Sliding_CTX_calation(df_worms, grad_vec,sm_inv, spd_sm_inv, hlf_sm_inv,window_size = 20,
                         frame_rate = 20, track_jump_frame = 1,
                        X_shift = 0):
    # 只适用于线性梯度，因为CTX只计算到左和右边缘的
    # 速度平滑窗s sm_inv
    # 计算角速度的速度平滑窗s spd_sm_inv
    # 角速度平滑床s hlf_sm_inv
    
    # ======================================数据平滑=================
    # 1. 移动窗口平滑
    # 对X和Y坐标进行移动平均
    df_slide = df_worms.copy()
    df_slide['X_org'] = df_slide['X']                     # 将平滑前的轨迹也加入df
    df_slide['Y_org'] = df_slide['Y']
    df_slide['X'] = df_slide['X'].rolling(window=window_size, center=True).mean()
    df_slide['Y'] = df_slide['Y'].rolling(window=window_size, center=True).mean()
    
    if X_shift:
        # 如果X有偏离，向左偏离减去，向右偏移加上，单位mm
        df_slide['X'] += X_shift
        
        
    # ======================================速度计算=================
    
    
    # 根据帧率计算半平滑窗，但是具体计算速度和角速度的时间根据时间戳来确定
    print(sm_inv, frame_rate, track_jump_frame)
    half_bins_spd = int((sm_inv*frame_rate)//track_jump_frame)
    bins_spd = 2*half_bins_spd

    trajectory_0 = df_slide[['X','Y','Time']].values      # 提取x,y坐标和时间戳
    trajectory_1 = (trajectory_0.copy()).astype('float')                            # 转float数据类型
    x = trajectory_1[:,0]
    y = trajectory_1[:,1]
    delta_x = (x[bins_spd:]-x[:len(x)-bins_spd])                       # 得到减去头尾数据点的delta_x和delta_y
    delta_y = (y[bins_spd:]-y[:len(y)-bins_spd])
    time_step = trajectory_1[:,2] 
    print('time_step', time_step)
    time_step_vec = time_step[bins_spd:]-time_step[:len(time_step)-bins_spd]        # 得到减去头尾数据点的对应delta-x和delta-y的时间
    print('平均时间间隔'+str(np.average(time_step_vec))+'应该等于总平滑窗'+str(sm_inv*2))
    mean_time_inv = np.average(time_step_vec)   # 平均时间间隔
    velocity_bef = np.dstack((delta_x/time_step_vec,delta_y/time_step_vec))[0]      # 使用dstack函数合并x,y方向计算的速度为一个矩阵
    speed_bef = np.linalg.norm(velocity_bef, axis = 1).reshape(-1,1)
    nan_bin_vec = np.full((half_bins_spd,2),np.nan)
    nan_bin_spd = np.full((half_bins_spd,1),np.nan)
    velocity = np.vstack((nan_bin_vec, velocity_bef, nan_bin_vec))                  # 速度向量
    speed = np.vstack((nan_bin_spd, speed_bef, nan_bin_spd))                        # 速率向量
    
    df_idx = df_slide[['X','X_org','Y','Y_org','Time']].copy()      
    df_idx['speed'] = pd.Series(speed[:,0],index = df_idx.index)                    # 将speed和velocity加入dataframe
    df_idx['x_velocity'] = pd.Series(velocity[:,0],index = df_idx.index)
    df_idx['y_velocity'] = pd.Series(velocity[:,1],index = df_idx.index)
    
    # ======================================================角速度计算===========

    half_bins_spd_agl = int((spd_sm_inv*frame_rate)//track_jump_frame)
    bins_spd_agl = 2*half_bins_spd_agl                                 # 角速度速度平滑窗
    hlf_agl_bins = int((hlf_sm_inv*frame_rate)//track_jump_frame)      # 角速度半平滑窗
    print('角速度半平滑窗：', hlf_agl_bins)
    trajectory_0 = df_slide[['X','Y','Time']].values      # 提取x,y坐标和时间戳
    trajectory_1 = (trajectory_0.copy()).astype('float')                            # 转float数据类型
    x = trajectory_1[:,0]
    y = trajectory_1[:,1]
    delta_x = (x[bins_spd_agl:]-x[:len(x)-bins_spd_agl])                                     # 得到减去头尾数据点的delta_x和delta_y
    delta_y = (y[bins_spd_agl:]-y[:len(y)-bins_spd_agl])
    time_step = trajectory_1[:,2]   
    time_step_vec = time_step[bins_spd_agl:]-time_step[:len(time_step)-bins_spd_agl]        # 得到减去头尾数据点的对应delta-x和delta-y的时间
    velocity_bef = np.dstack((delta_x/time_step_vec,delta_y/time_step_vec))[0]             # 使用dstack函数合并x,y方向计算的速度为一个矩阵
    speed_bef = np.linalg.norm(velocity_bef, axis = 1).reshape(-1,1)
    nan_bin_vec = np.full((half_bins_spd_agl,2),np.nan)
    nan_bin_spd = np.full((half_bins_spd_agl,1),np.nan)
    velocity = np.vstack((nan_bin_vec, velocity_bef, nan_bin_vec))
    speed = np.vstack((nan_bin_spd, speed_bef, nan_bin_spd))[:,0]
    time_step_vector = np.vstack((nan_bin_spd, time_step_vec.reshape(-1,1), nan_bin_spd))[:,0]
    
    df_speed_cal = pd.DataFrame(speed,columns = ['speed'])              # 生成一个包含speed的df并提取索引
    
    idx_vec = df_speed_cal.index                                        # 生成索引列表时先掐头去尾再取非零值
    df_washed_speed = df_speed_cal[half_bins_spd_agl:len(idx_vec)-half_bins_spd_agl]
    idx_nz_vec = df_washed_speed[df_washed_speed['speed']!=0].index     # 索引是相对于总长度，但是这个列表中直接去掉了头尾速度为nan的和速率为0的点的索引
    df_speed_cal['agl_velocity'] = pd.Series([np.nan]*len(idx_vec))

    for i in range(hlf_agl_bins,len(idx_nz_vec)-hlf_agl_bins):
        n = hlf_agl_bins
        agl_i = clws_delta_phi(velocity[idx_nz_vec[i-n]],velocity[idx_nz_vec[i+n]]) 
        time_end= time_step[idx_nz_vec[i+n]]
        time_start = time_step[idx_nz_vec[i-n]]
        delta_t_i = time_end-time_start
#         print("timestamp", time_start, time_start)
#         print('角度计算时间间隔：',delta_t_i)
        agl_vel_i = agl_i/delta_t_i

        if np.isnan(agl_vel_i) == False:
            df_speed_cal['agl_velocity'].loc[idx_nz_vec[i]] = agl_vel_i
        else:
#             print(velocity[idx_nz_vec[i-n]],velocity[idx_nz_vec[i+n]])
            df_speed_cal['agl_velocity'].loc[idx_nz_vec[i]] = 0 
    # 将angular_velocity和angular_speed加入dataframe(df_idx)
    df_idx['agl_velocity'] = pd.Series(df_speed_cal['agl_velocity'].values,index = df_idx.index)
    # df_idx.loc[:,'x_velocity_agl'] = velocity[:,0]
    # df_idx.loc[:,'y_velocity_agl'] = velocity[:,1]
    df_idx.loc[df_idx.speed == 0, 'agl_velocity'] = 0                 # 将speed为0的点角速度设为0
    df_idx['agl_speed'] = np.abs(df_idx['agl_velocity'])
    
    # ===========================================计算CTX=====================================
    vec_left = grad_vec
    vel_vec = df_idx[['x_velocity','y_velocity']].values
    bearing_left = []
    
    ctxs = []
    for i in range(len(vel_vec)):
        agl_i_left = clws_delta_phi(vec_left,vel_vec[i,:])
        agl_i_left = agl_i_left/180*np.pi     # bearing angle使用的是弧度制
        bearing_left.append(agl_i_left)                                      # 使用弧度制
        
        vel_vec_i = vel_vec[i,:]
        deno = np.sqrt(vel_vec_i[0]**2+vel_vec_i[1]**2)
        if deno==0:
            ctx_i = 0.0
        else:
            ctx_i = -vel_vec_i[0]/deno
        ctxs.append(ctx_i)
        
    df_idx['CTX_left'] = pd.Series(ctxs, index = df_idx.index)
    df_idx['bearing_left'] = pd.Series(bearing_left, index = df_idx.index)
#     df_idx['CTX_left']=df_idx['bearing_left'].apply(np.cos,axis = 0)
    


    return df_idx

### 前后后退

In [ ]:
def label_smth_forward_reverse(df, close_size = 50, open_size = 50):
    '''
    df: 包含angle_m列
    返回：df：包含平滑后的forward列
    '''
    # 前进后退
    # 根据阈值分割前进后退
    threshold = 120
    df.loc[:,'forward'] = 0
    df.loc[df["angle_m"]>=threshold,'forward'] = 1

    # 平滑reversal
    # 先进行闭操作（先膨胀后腐蚀）：填充小的空洞
    closed = ndimage.binary_closing(df['forward'].values, structure=np.ones(close_size))
    # 再进行开操作（先腐蚀后膨胀）：去除小的噪声点
    opened = ndimage.binary_opening(closed, structure=np.ones(open_size))
    # 将结果转换为整数并添加到 DataFrame
    df['forward'] = opened.astype(int)
    return df

### 身体前进方向速度

In [ ]:
# 计算根据运动方向旋转后的朝头部方向运动向量
def rotation_mat_2(theta_degrees, mov_vec):
    mov_vec = np.array(mov_vec)
    if (theta_degrees == np.nan) or (theta_degrees == None):
        return None
    if (mov_vec is None) or (mov_vec.any()==np.nan):
        return None
    """
    统一的 2D 旋转矩阵
    θ > 0: 逆时针旋转
    θ < 0: 顺时针旋转
    """
    theta_rad = -np.radians(theta_degrees)
    cos_theta = np.cos(theta_rad)
    sin_theta = np.sin(theta_rad)
    rot_mat =  np.array([
        [cos_theta, -sin_theta],
        [sin_theta,  cos_theta]
    ])
    return np.dot(rot_mat, mov_vec)

rotation_mat_vec = np.vectorize(rotation_mat_2)
# 求向量投影
def project_vector_A_on_B(vector_A, vector_B):
    """
    返回C向量,方向与B相同,长度为A的投影
    """
    # 转换为numpy数组
    A = np.array(vector_A)
    B = np.array(vector_B)
    # 计算向量B的单位向量
    B_norm = np.linalg.norm(B)
    if B_norm == 0:
        print('B向量不能为零')
        return None
        # raise ValueError("向量B不能为零向量")
    B_unit = B / B_norm
    
    # 计算向量A在向量B上的投影长度
    projection_length = np.dot(A, B_unit)
    # 计算向量C = 投影长度 × 单位向量B
    vector_C = projection_length * B_unit
    return vector_C

In [ ]:
def get_head_moving(df):
    '''
    df: 包含'x_velocity'和'y_velocity'列
    返回 df includes:
        'moving_vec': moving direction vector;
        'heading_vec': heading vector ;
       "head_moving": the component of velocity projected on the direction of heading
    '''
    # 运动方向向量
    df.loc[:,'moving_vec'] = df.apply(lambda x: (x['x_velocity'], x['y_velocity']),axis = 1)

    # 头部朝向向量
    df.loc[:,'heading_vec'] = df.apply(lambda x: rotation_mat_2(x['angle_md'], x['moving_vec']), axis=1)
    # 求投影头部朝向后的运动向量
    df.loc[:,'head_moving'] = df.apply(lambda x: project_vector_A_on_B(x['moving_vec'], x['heading_vec']), axis=1)
    return df

### 标记Omega turn和头部曲率

#### 骨架合并与拆分

In [ ]:
def path_length(coords: np.ndarray) -> float:
    """polyline 长度（逐段欧氏距离累加）。coords: (N,2)"""
    if coords is None or len(coords) < 2:
        return 0.0
    d = np.diff(coords, axis=0)
    return float(np.sum(np.sqrt((d**2).sum(axis=1))))

def quantize_point(pt, q=1):
    """把像素点量化到网格上（容差聚类）。q=1 等价于原始整数像素匹配。"""
    return (int(round(pt[0]/q)*q), int(round(pt[1]/q)*q))

def cut_paths_to_edges(all_paths, q=1):
    """
    把多条骨架在“端点”和“交叉点”处切段，返回 MultiGraph：
    - 节点：关键点坐标（量化后）
    - 边：两关键点之间的子折线，数据里带 coords/length
    """
    # 1) 统计每个像素（量化后）被多少条路径/多少次命中，用于发现交叉
    occ = {}  # key -> list of (path_idx, local_idx)
    for pi, path in enumerate(all_paths):
        if path is None or len(path) == 0:
            continue
        for si, pt in enumerate(path):
            key = quantize_point(pt, q)
            occ.setdefault(key, []).append((pi, si))

    # 2) 关键点：每条 polyline 的首尾点 + 被多个 path 命中的点（交叉/重合点）
    keypoints = set()
    for pi, path in enumerate(all_paths):
        if path is None or len(path) == 0:
            continue
        keypoints.add(quantize_point(path[0], q))
        keypoints.add(quantize_point(path[-1], q))
    for key, hits in occ.items():
        # 命中数量>1，基本可视为交叉/共点（或一个 path 在该点有重复）
        if len(hits) > 1:
            keypoints.add(key)

    # 3) 在关键点处把每条 polyline 切段，生成边
    G = nx.MultiGraph()
    # 节点位置（用于方向判断）
    node_pos = {}  # key(node) -> 原始坐标（选第一次出现的真实像素）
    for kp in keypoints:
        # 选一个代表性坐标（量化格内实际像素），尽量从 occ 里取真实像素
        if kp in occ:
            pi, si = occ[kp][0]
            node_pos[kp] = tuple(map(int, all_paths[pi][si]))
        else:
            node_pos[kp] = (int(kp[0]), int(kp[1]))
        G.add_node(kp)

    for pi, path in enumerate(all_paths):
        if path is None or len(path) < 2:
            continue
        # 找到该 path 上属于关键点的索引
        cut_idx = []
        for si, pt in enumerate(path):
            if quantize_point(pt, q) in keypoints:
                cut_idx.append(si)
        # 确保首尾在里头
        if 0 not in cut_idx:
            cut_idx.insert(0, 0)
        if (len(path)-1) not in cut_idx:
            cut_idx.append(len(path)-1)
        # 去重并排序
        cut_idx = sorted(set(cut_idx))
        # 相邻关键点之间形成一条边
        for a, b in zip(cut_idx[:-1], cut_idx[1:]):
            if b <= a:
                continue
            sub = path[a:b+1]
            if len(sub) < 2:
                continue
            u = quantize_point(sub[0], q)
            v = quantize_point(sub[-1], q)
            w = path_length(sub)
            # 可能 u==v（零长度/回折），跳过
            if u == v or w == 0:
                continue
            # 入图，边上存 coords 和 length
            G.add_edge(u, v, length=w, coords=sub)

    # 把 node 真实坐标也记录上（便于后续方向判断）
    nx.set_node_attributes(G, {n: {'pos': node_pos[n]} for n in G.nodes})
    return G

def oriented_coords(edge_data, start_node_pos):
    """
    根据起点节点位置，返回边的坐标正向/反向（避免连接时倒序）。
    edge_data['coords'] 是边的折线；start_node_pos 是当前节点实际坐标
    """
    coords = edge_data['coords']
    if len(coords) == 0:
        return coords
    # 与起点更接近的一端作为开头
    d0 = np.linalg.norm(coords[0] - start_node_pos)
    d1 = np.linalg.norm(coords[-1] - start_node_pos)
    return coords if d0 <= d1 else coords[::-1]

def extract_non_branching_chains(G: nx.MultiGraph):
    """
    提取所有“非分叉链”：中间节点度数=2，端点为“度!=2 的节点”或到头。
    同时处理纯环（整个连通分量所有节点度数=2）的情况。
    返回：list of dict，每个 dict 包含 {'nodes': [...], 'coords': np.ndarray, 'length': float}
    """
    chains = []
    visited = set()  # 记录已走过的具体边 (u,v,key)（无向，按排序存）

    def mark(u,v,k):  # 无向规范化
        return (u, v, k) if u <= v else (v, u, k)

    deg = dict(G.degree())
    terminals = [n for n,d in deg.items() if d != 2]

    # —— 从端点出发的所有链 —— #
    for start in terminals:
        for _, nbr, key, data in G.edges(start, keys=True, data=True):
            m = mark(start, nbr, key)
            if m in visited:
                continue
            # 开始沿着非分叉链走
            path_nodes = [start]
            path_coords = []
            cur, prev = start, None

            while True:
                # 选择从 cur 出发且未访问的边
                next_edge = None
                for _, nb, k, d in G.edges(cur, keys=True, data=True):
                    mk = mark(cur, nb, k)
                    if mk in visited:
                        continue
                    # 避免立刻折返（如果有其它可选）
                    if prev is not None and nb == prev:
                        continue
                    next_edge = (cur, nb, k, d)
                    break
                if next_edge is None:
                    # 如果没有其它边，就看看能不能把“回退那条”也并上（首步情况）
                    for _, nb, k, d in G.edges(cur, keys=True, data=True):
                        mk = mark(cur, nb, k)
                        if mk not in visited:
                            next_edge = (cur, nb, k, d)
                            break
                if next_edge is None:
                    break

                u, v, k, d = next_edge
                # 方向化坐标并拼接（避免重复第一个点）
                start_pos = np.array(G.nodes[u]['pos'])
                seg = oriented_coords(d, start_pos)
                if len(path_coords) == 0:
                    path_coords = seg.copy()
                else:
                    path_coords = np.vstack([path_coords, seg[1:]])
                visited.add(mark(u, v, k))

                prev, cur = u, v
                path_nodes.append(cur)
                # 到达端点（度!=2）就停止
                if deg[cur] != 2:
                    break

            if len(path_coords) >= 2:
                chains.append({
                    'nodes': path_nodes,
                    'coords': path_coords,
                    'length': path_length(path_coords)
                })

    # —— 处理纯环：该连通分量所有节点度数都为2 —— #
    # 还有未访问的边说明它们属于环
    for u, v, k in list(G.edges(keys=True)):
        m = mark(u, v, k)
        if m in visited:
            continue
        # 从 u 出发沿环走到无法继续
        path_nodes = [u]
        path_coords = []
        cur, prev = u, None
        while True:
            next_edge = None
            for _, nb, kk, d in G.edges(cur, keys=True, data=True):
                mk = mark(cur, nb, kk)
                if mk in visited:
                    continue
                # 在环里第一次可以任意选边，之后避免直接折返
                if prev is not None and nb == prev:
                    continue
                next_edge = (cur, nb, kk, d)
                break
            if next_edge is None:
                # 没有未访问边了，环走完
                break
            a, b, kk, d = next_edge
            start_pos = np.array(G.nodes[a]['pos'])
            seg = oriented_coords(d, start_pos)
            if len(path_coords) == 0:
                path_coords = seg.copy()
            else:
                path_coords = np.vstack([path_coords, seg[1:]])
            visited.add(mark(a, b, kk))
            prev, cur = a, b
            path_nodes.append(cur)

        if len(path_coords) >= 2:
            chains.append({
                'nodes': path_nodes,
                'coords': path_coords,
                'length': path_length(path_coords)
            })

    return chains

def merge_and_find_longest_non_branching(all_paths, quant=1):
    """
    主入口：
    1) 在交叉/端点处切段并建图
    2) 提取所有非分叉链（含纯环）
    3) 返回最长链的坐标和长度
    """
    G = cut_paths_to_edges(all_paths, q=quant)
    chains = extract_non_branching_chains(G)
    if not chains:
        return None, 0.0, chains, G
    longest = max(chains, key=lambda c: c['length'])
    chain_dict = chains[0]
    all_coords = chain_dict['coords']
    # 返回所有轨迹
    # all_chains = list({key: value['coords'] for key, value in chain_dict.items()}.values())
    return longest['coords'], longest['length'], all_coords, G
def merge_and_find_non_branching(all_paths, quant=1):
    """
    主入口：
    1) 在交叉/端点处切段并建图
    2) 提取所有非分叉链（含纯环）
    3) 返回所有链的坐标
    """
    G = cut_paths_to_edges(all_paths, q=quant)
    chains = extract_non_branching_chains(G)
    if not chains:
        return None, G
    # chains是list，找一个元素，使得其在某个指标下最大，指标为元素长度
    # chain的每个元素是一条链的各个信息
    chains = sorted(chains, key=lambda c: c['length'], reverse=True)
    all_seq_path = [i['coords'] for i in chains]
    # 返回所有轨迹
    # all_chains = list({key: value['coords'] for key, value in chain_dict.items()}.values())
    return all_seq_path, G

In [ ]:
# 拆分相关函数
def path_length(path):
    """计算一条 path 的几何长度"""
    diffs = np.diff(path, axis=0)
    return np.sum(np.sqrt((diffs ** 2).sum(axis=1)))

def build_graph(paths, tol=1e-6):
    """把所有 paths 合并成一张图"""
    G = nx.Graph()
    node_id = {}

    def get_node_id(point):
        for nid, coord in node_id.items():
            if np.allclose(coord, point, atol=tol):
                return nid
        nid = len(node_id)
        node_id[nid] = point
        return nid

    for path in paths:
        src, dst = path[0], path[-1]
        src_id, dst_id = get_node_id(src), get_node_id(dst)
        length = path_length(path)
        G.add_edge(src_id, dst_id, weight=length, coords=path)

    return G, node_id

def longest_path(paths):
    """返回合并后图上的最长路径坐标"""
    G, node_id = build_graph(paths)
    degrees = dict(G.degree())
    endpoints = [n for n, d in degrees.items() if d == 1]

    longest_len = -1
    longest_coords = None

    for i in range(len(endpoints)):
        for j in range(i+1, len(endpoints)):
            u, v = endpoints[i], endpoints[j]
            for path_nodes in nx.all_simple_paths(G, u, v):
                coords = []
                total_len = 0
                for k in range(len(path_nodes)-1):
                    a, b = path_nodes[k], path_nodes[k+1]
                    edge = G[a][b]
                    coords_edge = edge["coords"]

                    # 判断方向
                    if np.allclose(coords_edge[0], node_id[a]):
                        coords.extend(coords_edge.tolist())
                    else:
                        coords.extend(coords_edge[::-1].tolist())

                    total_len += edge["weight"]

                if total_len > longest_len:
                    longest_len = total_len
                    longest_coords = np.array(coords)

    return longest_coords
def longest_path_old(paths):
    """返回合并后图上的最长路径坐标"""
    G, node_id = build_graph(paths)
    degrees = dict(G.degree())
    endpoints = [n for n, d in degrees.items() if d == 1]

    longest_len = -1
    longest_coords = None

    for i in range(len(endpoints)):
        for j in range(i+1, len(endpoints)):
            u, v = endpoints[i], endpoints[j]
            for path_nodes in nx.all_simple_paths(G, u, v):
                coords = []
                total_len = 0
                for k in range(len(path_nodes)-1):
                    edge = G[path_nodes[k]][path_nodes[k+1]]
                    coords.extend(edge["coords"].tolist())
                    total_len += edge["weight"]
                if total_len > longest_len:
                    longest_len = total_len
                    longest_coords = np.array(coords)

    return longest_coords


#### 身体曲率计算

In [ ]:
def resample_skeleton(skeleton, step=1.0):
    """
    把骨架按固定像素间距重采样
    skeleton: (N,2) 数组
    step: 每隔多少像素取一个点
    """
    skeleton = np.array(skeleton)
    if len(skeleton) < 3:
        return skeleton
    
    # 去掉重复点
    diffs = np.diff(skeleton, axis=0)
    mask = np.any(diffs != 0, axis=1)
    skeleton = skeleton[np.r_[True, mask]]
    if len(skeleton) < 3:
        return skeleton
    
    x, y = skeleton[:,0], skeleton[:,1]
    seglen = np.sqrt(np.diff(x)**2 + np.diff(y)**2)
    dist = np.concatenate(([0], np.cumsum(seglen)))   # 单位：像素
    total_len = dist[-1]
    
    # 归一化到 [0,1] 供 splprep 使用
    u = dist / total_len
    try:
        tck, _ = splprep([x, y], s=0, u=u)
    except Exception as e:
        print("splprep error:", e)
        return skeleton
    
    # 在 [0,total_len] 上等间距采样
    new_dist = np.arange(0, total_len, step)
    new_u = new_dist / total_len
    new_points = np.array(splev(new_u, tck)).T
    return new_points

def resample_skeleton_2(skeleton, step=1.0):
    # 等像素间距采样
    total_len = len(skeleton)
    if total_len <= step*2:
        return []
    idx_series = np.arange(0,total_len, step)
    selected_points = skeleton[idx_series,:]
    return selected_points

def curvature_from_angles(skeleton, step=1.0):
    """
    通过角度差计算骨架曲率
    返回：
        curvatures: 每个点的角度差 (带正负号)
        mean_curvature: 曲率强度指标
    """
    pts = resample_skeleton_2(skeleton, step=step)
    if not len(pts):
        return np.nan

    diffs = np.diff(pts, axis=0)
    angles = np.arctan2(diffs[:,1], diffs[:,0])
    # 解开角度防止跳变
    angles_unwrapped = np.unwrap(angles)
    # 相邻角度差
    dtheta = np.diff(angles_unwrapped)
    # # 曲率指标（你可以换成 np.mean(np.abs(dtheta))）
    # mean_curvature = np.sqrt(np.mean(dtheta**2))
    return np.degrees(dtheta)

In [ ]:
# 等间距重采样和滑动平均

def moving_average(data, window_size=5):
    """对二维数据做滑动平均"""
    kernel = np.ones(window_size) / window_size
    x = np.convolve(data[:,0], kernel, mode='same')
    y = np.convolve(data[:,1], kernel, mode='same')
    return np.vstack((x,y)).T

def resample_and_smooth(points, window_size=5):
    # --- 等弧长重采样 ---
    x, y = points[:,0], points[:,1]
    dist = np.sqrt(np.diff(x)**2 + np.diff(y)**2)
    s = np.concatenate(([0], np.cumsum(dist)))
    total_length = s[-1]
    s_uniform = np.linspace(0, total_length, len(points))
    fx = interp1d(s, x, kind='linear')
    fy = interp1d(s, y, kind='linear')
    resampled = np.vstack((fx(s_uniform), fy(s_uniform))).T
    
    # --- 平滑 ---
    smoothed = moving_average(resampled, window_size)
    return resampled, smoothed
def remove_outliers_by_jump(points, max_step=10):
    """
    去掉瞬间跨度过大的点
    points: (N,2) array
    max_step: 相邻点最大允许移动距离 (像素)
    """
    x, y = points[:,0], points[:,1]
    dist = np.sqrt(np.diff(x)**2 + np.diff(y)**2)
    # 第一个点保留
    mask = np.ones(len(points), dtype=bool)
    
    # 如果跨度大于阈值，就把后一帧去掉
    mask[1:][dist > max_step] = False
    
    return points[mask]
def resample_and_dthatas(long_paths, step):
    # 采样和平均
    resampled = {}
    # smoothed = {}
    for key, skelon in long_paths.items():
        if type(skelon) == type(None):
            resampled[key] =  []
        else:
            # 去掉瞬移离群点
            skelon = remove_outliers_by_jump(skelon, max_step=10)
            resampled[key],_ = resample_and_smooth(skelon, window_size=50)
    dtheta = {}
    for key, skelon in resampled.items():
        if type(None)==type(skelon):
            dtheta[key] = None
        elif (len(skelon)) < 200:
            dtheta[key] = None
        else:
            dtheta_i = curvature_from_angles(skelon, step=step)
            sum_dtheta_i = round(np.sum(dtheta_i),2)
            dtheta[key]={'dtheta':dtheta_i,'sum_dtheta':sum_dtheta_i}
    return resampled, dtheta


#### 路径比例

In [ ]:
# 求路径比例和前两条路径长度的df
def path_ratio(paths):
    ratios = {}
    len_dict = {}
    for key, value in paths.items():
        if len(value) >= 2:
            # 如果大于2，求前两个的比例
            len_ls = [len(path) for path in value]
            len_ls.sort(reverse=True)
            ratio = len_ls[0]/len_ls[1]
            lens = len_ls[:2]
        else:
            ratio = np.nan
            lens = []
        ratios[key] = ratio
        len_dict[key] = lens
    df_ratio = pd.Series(ratios, name='path_ratio').to_frame()
    df_len = pd.Series(len_dict, name='path_len').to_frame()
    df = df_ratio.join(df_len, how='left')

    return df

#### 标记turn主函数

In [ ]:
def get_cor_turn_reversal(df,  after_turn_rev_lim = 50):
    '''
    df:包含已经平滑后的turn_pc列, 以及'forward'列
    return: 包含两列的子df: 'turn_cor', 'forward_subs_turn'
    '''
    # 首先根据turn将reversal切断，只保留不为turn的reversal段
    df.loc[:, 'forward_subs_turn'] = df['forward'].copy()
    df.loc[df['turn_pc']==1,'forward_subs_turn'] = 0
    df.loc[:,'turn_cor']  = df['turn_pc'].copy()

    # 计算所有reversal段的长度 ，如果大于threshold,看前面是否有turn，有的话将这个turn删除
    n = len(df)
    # 找到连续的 reversal 段
    df['turn_group'] = (df["turn_cor"].diff().ne(0)).cumsum() * df["turn_cor"]
    # 遍历每个 reversal 区间
    for gid, grp in df.groupby('turn_group'):
        if gid == 0:
            continue
        # turn 段的结束位置（label）与位置索引（整数位置）
        end_label = grp.index[-1]
        end_pos = df.index.get_loc(end_label)

        # 寻找紧接着出现 reversal 的位置（允许 small gap）
        start_search_pos = end_pos + 1
        if start_search_pos >= n:
            # 如果数据到头了，舍弃
            continue
        found_pos = None

        for offset in range(after_turn_rev_lim + 1):
            p = start_search_pos + offset
            if p >= n:
                break
            if df['forward_subs_turn'].iat[p] == 1:
                found_pos = p
                break
        if found_pos is None:
            # 没有紧接的 reversal
            continue

        # 计数 reversal 连续长度
        q = found_pos
        while q + 1 < n and df['forward_subs_turn'].iat[q + 1] == 1:
            # 遍历reversal的长度，当超过reversal时停
            q += 1
        rev_len = q - found_pos + 1

        if rev_len >= after_turn_rev_lim:
            # 仅把这个 turn 段（grp.index）置 0：相当于不算这个turn
            df.loc[grp.index, "turn_cor"] = 0
            # 另外将这一段turn对应的前进状态改为reversal，便于后续分类转向行为
            df.loc[grp.index, "forward_subs_turn"] = 1 
        else:
            df.iloc[found_pos:found_pos+rev_len, df.columns.get_loc('forward_subs_turn')] = 0
            # 如果这个长度小于阈值，则仍然保留turn，并且把接着的这个reverse变为前进
    return df[['turn_cor', 'forward_subs_turn']]

##### 可视化验证

In [ ]:
# 求1的开始和结束点
def get_turn_interval(df, col_name):
    # labels打印矩形
    labels = df[col_name].values
    n = len(labels)
    # 找出连续的 label==1 区间
    turn_intervals = []
    in_turn = False
    start = 0

    for i in range(n):
        if labels[i] == 1 and not in_turn:
            # turn开始
            start = i
            in_turn = True
        elif labels[i] != 1 and in_turn:
            # turn结束
            end = i - 1
            turn_intervals.append((start, end))
            in_turn = False

    # 如果最后一个点也是turn状态
    if in_turn:
        turn_intervals.append((start, n - 1))
    return turn_intervals
def visualize_turn_correction(df, close_size = 50, open_size = 50, x_lim = []):
    '''
    这个函数是可视化turn的矫正(删除reversal内部的)
    '''
    # 可视化验证
    reversal_ints = get_turn_interval(df, 'forward')
    subs_reversal_ints = get_turn_interval(df, 'forward_subs_turn')

    # Turns: 开闭后的turn
    # turn_ints = get_turn_interval(df, 'turn')
    turn_pc_ints = get_turn_interval(df, 'turn_pc')
    turn_cor_ints = get_turn_interval(df, 'turn_cor')

    if 'label' in df.columns:
        # artificial labeling
        label_ints = get_turn_interval(df, 'label')
        # 绘图
        fig,ax = plt.subplots(5,1,figsize=(20,10), sharex=True)
        for start, end in subs_reversal_ints:
            ax[-1].axvspan(start, end, color='blue', alpha=1)  # alpha控制透明度
        for start, end in label_ints:
            ax[-1].axvspan(start, end, color='blue', alpha=1)  # alpha控制透明度
        ax[-1].set_title(f'artificial labels')
    else:
        fig,ax = plt.subplots(4,1,figsize=(20,10), sharex=True)

    # 矫正前自动标记
    for start, end in reversal_ints:
        ax[0].axvspan(start, end, color='blue', alpha=1)  # alpha控制透明度
    for start, end in turn_pc_ints:
        ax[0].axvspan(start, end, color='orange', alpha=1)  # alpha控制透明度
    ax[0].set_title(f'automated labeling of turn close:{close_size},open:{open_size}')

    # 矫正后标记
    for start, end in subs_reversal_ints:
        ax[1].axvspan(start, end, color='blue', alpha=1)  # alpha控制透明度
    for start, end in turn_cor_ints:
        ax[1].axvspan(start, end, color='orange', alpha=1)  # alpha控制透明度
    ax[1].set_title(f'corrected turn labeling')

    # 纯平滑后的reversal
    for start, end in subs_reversal_ints:
        ax[2].axvspan(start, end, color='blue', alpha=1)  # alpha控制透明度
    ax[2].set_title(f'reversal_substract_turn')

    # 平滑前的reversal
    for start, end in reversal_ints:
        ax[3].axvspan(start, end, color='blue', alpha=1)  # alpha控制透明度
    ax[3].set_title(f'reversal')

    desired_ticks = np.arange(0, len(df), 2000) # 生成一个数组 [10, 30, 50, 70, 90]
    ax[-1].set_xticks(desired_ticks)
    if len(x_lim):
        ax[-1].set_xlim(*x_lim)
    plt.show()

def visualize_turn_labeling(df,path_ratio_uplim,dtheta_lim, close_size = 50, open_size = 50, x_lim=[]):
    '''
    可视化turn的标注条件和标注
    '''
    # 可视化验证
    reversal_ints = get_turn_interval(df, 'forward')
    turn_ints = get_turn_interval(df, 'turn')
    # 开闭后的turn
    turn_pc_ints = get_turn_interval(df, 'turn_pc')
    # circular
    cir_ints = get_turn_interval(df, 'circular')
    # branching
    bra_ints = get_turn_interval(df, 'branching')
    # path_ratio
    df['path_ratio_dis'] = np.where(df['path_ratio'] <= path_ratio_uplim, 1, 0)
    ratio_ints = get_turn_interval(df,'path_ratio_dis')
    # sum_dtheta
    df['sum_dtheta_dis'] = np.where(df['sum_dtheta'].abs() >= dtheta_lim , 1, 0)
    dtheta_ints = get_turn_interval(df, "sum_dtheta_dis")

    if 'label' in df.columns:
        # artificial labeling
        label_ints = get_turn_interval(df, 'label')
        # 绘图
        fig,ax = plt.subplots(7,1,figsize=(20,10), sharex=True)
        for start, end in label_ints:
            ax[-1].axvspan(start, end, color='red', alpha=1)  # alpha控制透明度
        ax[-1].set_title('artificial labeling of turn')
    else:
        fig,ax = plt.subplots(6,1,figsize=(20,10), sharex=True)

    # 绘制人工打标记

    for start, end in turn_ints:
        ax[-2].axvspan(start, end, color='orange', alpha=1)  # alpha控制透明度
    ax[-2].set_title('automated labeling of turn')
    # for start, end in reversal_ints:
    #     ax[1].axvspan(start, end, color='blue', alpha=1)  # alpha控制透明度

    for start, end in cir_ints:
        ax[0].axvspan(start, end, color='orange', alpha=1)  # alpha控制透明度
    ax[0].set_title('circular')
    for start, end in bra_ints:
        ax[1].axvspan(start, end, color='orange', alpha=1)  # alpha控制透明度
    ax[1].set_title('branching')
    for start, end in ratio_ints:
        ax[2].axvspan(start, end, color='orange', alpha=1)  # alpha控制透明度
    ax[2].set_title('path_ratio')
    for start, end in dtheta_ints:
        ax[3].axvspan(start, end, color='orange', alpha=1)  # alpha控制透明度
    ax[3].set_title('sum_dtheta')
    desired_ticks = np.arange(0, len(df), 2000) # 生成一个数组 [10, 30, 50, 70, 90]
    ax[-1].set_xticks(desired_ticks)
    for start, end in reversal_ints:
        ax[4].axvspan(start, end, color='blue', alpha=1)  # alpha控制透明度
    for start, end in turn_pc_ints:
        ax[4].axvspan(start, end, color='orange', alpha=1)  # alpha控制透明度
    ax[4].set_title(f'turn_open_closed. close:{close_size},open:{open_size}')
    if len(x_lim):
        ax[-1].set_xlim(*x_lim)
    plt.show()

#### 头部曲率和角速度

确定骨架和头朝向函数

In [ ]:
# 定义函数根据咽喉位置决定翻转
def process_skeleton(points, pos_phr, n_segments=30):
    """
    points: list of (x, y)
    pos_phr: (x, y) 咽喉点
    n_segments: 要分的段数
    """
    # 1. 转成 DataFrame
    points = np.array(points)
    points = resample_skeleton(points,step=1)
    n_points = len(points)

    # 2. 找最近的点
    pos_phr = np.array(pos_phr)
    dist = np.linalg.norm(points - pos_phr, axis=1)
    idx_phr = np.argmin(dist)
    # 4. 比较长度
    # len_left = idx_phr
    # len_right = len(dist)-idx_phr
    # 3. 计算左侧累计长度
    len_left = np.sum(
        np.linalg.norm(points[1:idx_phr+1] - points[:idx_phr], axis=1)
    )
    # 4. 计算右侧累计长度
    len_right = np.sum(
        np.linalg.norm(points[idx_phr+1:] - points[idx_phr:-1], axis=1)
    )

    # 5. 是否翻转
    if len_left > len_right:
        points = points[::-1]
    return points
def select_backbone(paths, closest_idx,pos_phr, longest_paths):
    '''
    提取最长骨架，确定头朝向
    '''
    # 提取原最接近咽喉的骨架
    closest_path = {}
    for key, all_paths in paths.items():   # paths 是 dict 的情况
        idx = closest_idx[key]
        closest_path[key] = all_paths[idx]
    # 选择骨架
    sel_paths = {}
    for key, longest in longest_paths.items():
        closest = closest_path[key]
        if type(longest)==type(None):
            # 重合并无骨架，选择closest
            sel_paths[key] = closest_path[key]
            continue
        # 计算头尾距离
        head = longest[0]
        tail = longest[-1]
        closest_head = closest[0]
        dist_head = (head[0]-closest_head[0])**2 + (head[1]-closest_head[1])**2
        dist_tail = (tail[0]-closest_head[0])**2 + (tail[1]-closest_head[1])**2
        if len(longest) > len(closest) & (min(dist_tail, dist_head) <= 10):
            # 两条轨迹的头是一样的，否则即使longest更长也不要
            sel_paths[key] = longest
        else:
            sel_paths[key] = closest
    # 根据咽喉坐标,距离咽喉近的设为头
    rev_sel_path = {}
    for key, sel in sel_paths.items():
        pos_phr_i = pos_phr[key]
        re_pos_phr_i = [pos_phr_i[1], pos_phr_i[0]]
        rev_sel = process_skeleton(sel, re_pos_phr_i)
        rev_sel_path[key] = rev_sel
    return rev_sel_path

计算曲率函数

In [ ]:
def calc_angle(vector1, vector2):
    """
        计算带符号的角度（-180° 到 180°）
        正角度表示从 vector1 到 vector2 是逆时针旋转
        负角度表示顺时针旋转
        """
    dot_product = np.dot(vector1, vector2)
    cross_product = np.cross(vector1, vector2)  # 在 2D 中，这给出标量值
    magnitude1 = np.linalg.norm(vector1)
    magnitude2 = np.linalg.norm(vector2)
    denominator = magnitude1 * magnitude2
    if denominator < 1e-10:  # 设置一个很小的阈值
        # 处理零向量的情况
        cos_theta = 1.0 if np.allclose(vector1, vector2) else np.nan
        sin_theta = 0.0
    else:
        cos_theta = dot_product / (magnitude1 * magnitude2)
        sin_theta = cross_product / (magnitude1 * magnitude2)
    # 使用 arctan2 获取带符号的角度
    angle_rad = np.arctan2(sin_theta, cos_theta)
    return np.degrees(angle_rad)

def label_segments(points, step=30):
    df_bb = pd.DataFrame(points, columns=['X', 'Y'])
    seg_len = np.linalg.norm(points[1:] - points[:-1], axis=1)
    # 累计距离
    cumdist = np.concatenate([[0], np.cumsum(seg_len)])
    # 分段编号
    seg_id = (cumdist // step).astype(int)
    df_bb = df_bb.copy()
    df_bb['seg_id'] = seg_id
    colors_bgr = [
    (255, 0, 0),     # 蓝色
    (0, 255, 0),     # 绿色
    (0, 0, 255),     # 红色
    (0, 255, 255),   # 黄色 (BGR: cyan+red)
    (255, 0, 255),   # 洋红 (magenta)
    (255, 255, 0)    # 青色 (cyan)
    ]
    # 建立 seg_id 到颜色的映射
    dict_colors = {key: colors_bgr[key % len(colors_bgr)] for key in range(len(colors_bgr))}
    df_bb['color'] = df_bb['seg_id'].apply(lambda x: dict_colors[x%6])
    return df_bb
def label_segments_curvature(points,pos_phr, step=30):
    if not len(points):
        return None, None, None
    df_bb = pd.DataFrame(points, columns=['X', 'Y'])
    seg_len = np.linalg.norm(points[1:] - points[:-1], axis=1)
    # 累计距离
    cumdist = np.concatenate([[0], np.cumsum(seg_len)])
    # 分段编号
    seg_id = (cumdist // step).astype(int)
    df_bb = df_bb.copy()
    df_bb['seg_id'] = seg_id

    # 分段后根据咽喉所在段求头部曲率
    pos_phr = np.array([pos_phr[0],pos_phr[1]])
    dist = np.linalg.norm(points - pos_phr, axis=1)
    idx_phr = np.argmin(dist)
    df_phr = df_bb.iloc[idx_phr]
    seg_phr_idx = df_bb.iloc[idx_phr]['seg_id']
    # 提取靠后的点
    # seg_pos_idx = (seg_phr_idx-2)
    df_seg_pos = df_bb[df_bb['seg_id']==(seg_phr_idx+6)]
    # 头部顶点
    df_start = df_bb.iloc[0]
    if not len(df_seg_pos):
        curvature = None
        pos_phr = None
        phr_node = None
    else:
        df_pos_end = df_seg_pos.iloc[-1]
        # 指向向量
        pos_phr = [df_phr['X']-df_pos_end['X'], df_phr['Y']-df_pos_end['Y']]
        phr_node = [df_start['X']-df_phr['X'], df_start['Y']-df_phr['Y']]
        curvature = calc_angle(pos_phr, phr_node)

    # 根据前两段求头部指向向量用于计算角速度
    df_seg2 = df_bb[df_bb['seg_id']==2]
    if not len(df_seg2):
        head_vector = [np.nan, np.nan]
    else:
        seg2_end = df_seg2.iloc[-1]
        head_vector = [df_start['X']-seg2_end['X'], df_start['Y']-seg2_end['Y']]

    return df_bb, curvature, head_vector, pos_phr, phr_node

计算头部角速度函数

In [ ]:
def cal_head_agl_velocity(df, window_size=300, frame_rate=38):
    '''
    对向量的角度变化进行滤波计算
    '''
    # 平滑计算角速度
    head_vectors =  df['head_vector']
    head_vec_filled = head_vectors.apply(lambda v: v if isinstance(v, (list, np.ndarray)) else [np.nan, np.nan])
    head_arr = np.vstack(head_vec_filled)
    dy = head_arr[:,0]
    dx = head_arr[:,1]
    # 假设 dx, dy shape (T,)
    angles = np.arctan2(dy,dx)          # radians
    angles_interp = pd.Series(angles).interpolate(limit_direction="both").to_numpy()
    angles_unwrapped = np.unwrap(angles_interp)

    # 采样间隔 dt (s) —— 例：300帧 = 15s -> dt = 15/300 = 0.05
    dt = 1 / frame_rate  # = 0.05

    angles_unwrapped = np.array(angles_unwrapped, dtype=float)
    mask = np.isfinite(angles_unwrapped)
    angles_clean = angles_unwrapped[mask]

    # 2. 设置合适的窗口
    window = min(window_size, len(angles_clean) // 2 * 2 + 1)  # 不超过数据长度的最大奇数
    if window < 3: 
        window = 3  # 最小窗口

    # 3. 滤波
    angular_velocity = savgol_filter(
        angles_clean,
        window_length=window,
        polyorder=1,
        deriv=1,
        delta=dt,
        mode="interp"
    )
    # 若需要度/秒：
    angular_velocity_deg = np.degrees(angular_velocity)
    df['ang_velocity'] = angular_velocity_deg
    return df


#### Turn和曲率主函数

In [ ]:
# # 输入中线地址，输出包含中线相关参数的df
# def get_omega_turn_curvature(mid_line_path, path_ratio_uplim = 1.5, path_uplim=200, dtheta_lim=220,
#                               cl_turn_size = 50, op_turn_size = 50, visualize_turn=True,
#                               seg_pixel_step = 13, agl_window = 300, frame_rate=38, ):
#     '''
#     mid_line_path:中线文件路径
#     '''
#     npz_data = np.load(mid_line_path, allow_pickle=True)
#     data = npz_data['arr_0'].item()
#     # data_keys = list(data.keys())
#     paths= {key: value['all_paths'] for key, value in data.items()}
#     closest_idx = {key: value['closest path'] for key, value in data.items()}
#     pos_phr = {key: value['pos_phr'] for key, value in data.items()}
#     vector_h = {key: value['vector_h'] for key, value in data.items()} 
#     circular = {key: value['circular'] for key, value in data.items()}
#     branching = {key: value['branching'] for key, value in data.items()}

#     # 骨架拆分
#     seq_paths = {}
#     for key, value in paths.items():
#         # print(merge_and_find_non_branching(value, quant=3))
#         seq_path_i, _ = merge_and_find_non_branching(
#             value,
#             quant=3   # 若不同段的交点不完全重合，可设 quant=2 或 3 像素，把近点聚成同一节点
#         )
#         seq_paths[key] = seq_path_i
#     # 骨架合并，得到最长骨架
#     longest_paths = {}
#     for key, value in seq_paths.items():
#         longest_coords = longest_path(value)
#         longest_paths[key] = longest_coords

#     # 对最常骨架重采样计算曲率
#     resampled, dtheta = resample_and_dthatas(longest_paths,step=20)
#     sum_dtheta = {
#         key: (value['sum_dtheta'] if isinstance(value, dict) and 'sum_dtheta' in value else None)
#         for key, value in dtheta.items()
#         }
#     # 曲率计算结果生成df
#     df_sum_dtheta = pd.Series(sum_dtheta, name='sum_dtheta')
#     print(f'df_sum_dtheta的长度{len(df_sum_dtheta)}和最大index{df_sum_dtheta.index.max()}')
#     # 计算路径比例
#     df = path_ratio(paths)
#     df['sum_dtheta'] = df_sum_dtheta
#     # 合并环形和分叉信息
#     df_cir_branching = pd.DataFrame.from_dict(
#         {"circular": circular, "branching": branching},
#         orient="index").T
#     df[["circular", "branching"]] = df_cir_branching[["circular", "branching"]]

#     # 分类turn
#     df['path_lim'] = df['path_len'].apply(
#         lambda x: 1 if (isinstance(x, (list, tuple, np.ndarray)) and len(x) > 1 and x[1] >= path_uplim) else 0
#     )
#     df['turn'] = 0
#     df.loc[
#         (df['circular'] == 1) |
#         (df['branching'] == 1) |
#         ((df['path_ratio'] <= path_ratio_uplim)&(df['path_lim'] == 1)) |
#         (df['sum_dtheta'].abs() >= dtheta_lim)
#         ,
#         'turn'] = 1
#     # 平滑turn
#     closed = ndimage.binary_closing(df['turn'].values, structure=np.ones(cl_turn_size))
#     # 再进行开操作（先腐蚀后膨胀）：去除小的噪声点
#     opened = ndimage.binary_opening(closed, structure=np.ones(op_turn_size))
#     # 将结果转换为整数并添加到 DataFrame
#     df['turn_pc'] = opened.astype(int)
#     df_turn_forw = df[['turn_pc', 'forward']]
#     df[['turn_cor', 'forward_subs_turn']] = get_cor_turn_reversal(df_turn_forw)

#     if visualize_turn:
#         visualize_turn_labeling(df,path_ratio_uplim,dtheta_lim, cl_turn_size, op_turn_size)
#         visualize_turn_correction(df, close_size=cl_turn_size, open_size=op_turn_size)
    
#     # 计算曲率
#     rev_sel_path = select_backbone(paths, closest_idx,pos_phr, longest_paths)
#     curvature = {}
#     head_vector = {}
#     post_phr = {}
#     phr_noses = {}
#     for key, sel in rev_sel_path.items():
#         pos_phr_i = pos_phr[key]
#         re_pos_phr_i = [pos_phr_i[1], pos_phr_i[0]]
#         df_bb_i, curvature_i, head_vector_i, post_phr_i, phr_noses_i = label_segments_curvature(sel,re_pos_phr_i, step=seg_pixel_step)
#         curvature[key] = curvature_i
#         head_vector[key] = head_vector_i
#         post_phr[key] = post_phr_i
#         phr_noses[key] = phr_noses_i
#     df_cur = pd.Series(curvature, name='curvature')
#     df_head_vec = pd.Series(head_vector, name='head_vector')
#     df_sel = pd.Series(rev_sel_path, name='sel_paths')
#     df['curvature'] = df_cur
#     df['head_vector'] = df_head_vec
#     df['sel_paths'] = df_sel
#     df = cal_head_agl_velocity(df, window_size=agl_window, frame_rate=frame_rate)
#     print(df.columns)
#     return df

In [ ]:
# 输入中线地址，和运动参数df
# 输出包含中线相关参数（以及时间列‘Time’）的df
def get_omega_turn_curvature(df_merged, mid_line_path, path_ratio_uplim = 1.5, path_uplim=200, dtheta_lim=220,
                              cl_turn_size = 50, op_turn_size = 50, cl_rev_size = 50, op_rev_size = 50,
                                visualize_turn=True, seg_pixel_step = 13, agl_window = 300, frame_rate=38):
    '''
    处理中线标记前进后退和turn,reversal等加入运动参数df
    df_merged: 包含运动数据和其它中线数据(row和choose_frame)
    mid_line_path:中线文件路径
    '''
    npz_data = np.load(mid_line_path, allow_pickle=True)
    data = npz_data['arr_0'].item()
    # data_keys = list(data.keys())
    paths= {key: value['all_paths'] for key, value in data.items()}
    closest_idx = {key: value['closest path'] for key, value in data.items()}
    pos_phr = {key: value['pos_phr'] for key, value in data.items()}
    # angle_m = {key: value['angle_m'] for key, value in data.items()}
    vector_h = {key: value['vector_h'] for key, value in data.items()} 
    circular = {key: value['circular'] for key, value in data.items()}
    branching = {key: value['branching'] for key, value in data.items()}

    # 前进后退
    df_mot_slice = df_merged[['Time','X', 'Y',"x_velocity","y_velocity", 'angle_m','angle_md']].copy()
    df_mot_slice = label_smth_forward_reverse(df_mot_slice, close_size = cl_rev_size, open_size = op_rev_size)
    # 运动方向向量
    df_mot_slice.loc[:,'moving_vec'] = df_mot_slice.apply(lambda x: (x['x_velocity'], x['y_velocity']),axis = 1)
    # 头部朝向向量
    df_mot_slice.loc[:,'heading_vec'] = df_mot_slice.apply(lambda x: rotation_mat_2(x['angle_md'], x['moving_vec']), axis=1)
    # 求投影头部朝向后的运动向量
    df_mot_slice.loc[:,'head_moving'] = df_mot_slice.apply(lambda x: project_vector_A_on_B(x['moving_vec'], x['heading_vec']), axis=1)

    # 骨架拆分
    seq_paths = {}
    for key, value in paths.items():
        # print(merge_and_find_non_branching(value, quant=3))
        seq_path_i, _ = merge_and_find_non_branching(
            value,
            quant=3   # 若不同段的交点不完全重合，可设 quant=2 或 3 像素，把近点聚成同一节点
        )
        seq_paths[key] = seq_path_i
    # 骨架合并，得到最长骨架
    longest_paths = {}
    for key, value in seq_paths.items():
        longest_coords = longest_path(value)
        longest_paths[key] = longest_coords

    # 对最常骨架重采样计算曲率
    resampled, dtheta = resample_and_dthatas(longest_paths,step=20)
    sum_dtheta = {
        key: (value['sum_dtheta'] if isinstance(value, dict) and 'sum_dtheta' in value else None)
        for key, value in dtheta.items()
        }
    # 曲率计算结果生成df
    df_sum_dtheta = pd.Series(sum_dtheta, name='sum_dtheta')
    print(f'df_sum_dtheta的长度{len(df_sum_dtheta)}和最大index{df_sum_dtheta.index.max()}')
    # 计算路径比例
    df = path_ratio(paths)
    df.loc[:,'sum_dtheta'] = df_sum_dtheta
    # 合并环形和分叉信息
    df_cir_branching = pd.DataFrame.from_dict(
        {"circular": circular, "branching": branching},
        orient="index").T
    df[["circular", "branching"]] = df_cir_branching[["circular", "branching"]].copy()

    # 分类turn
    df['path_lim'] = df['path_len'].apply(
        lambda x: 1 if (isinstance(x, (list, tuple, np.ndarray)) and len(x) > 1 and x[1] >= path_uplim) else 0
    )
    df['turn'] = 0
    df.loc[
        (df['circular'] == 1) |
        (df['branching'] == 1) |
        ((df['path_ratio'] <= path_ratio_uplim)&(df['path_lim'] == 1)) |
        (df['sum_dtheta'].abs() >= dtheta_lim)
        ,
        'turn'] = 1
    # 平滑turn
    closed = ndimage.binary_closing(df['turn'].values, structure=np.ones(cl_turn_size))
    # 再进行开操作（先腐蚀后膨胀）：去除小的噪声点
    opened = ndimage.binary_opening(closed, structure=np.ones(op_turn_size))
    # 将结果转换为整数并添加到 DataFrame
    df.loc[:,'turn_pc'] = opened.astype(int)


    # 合并前进后退信息
    df[['Time','forward', 'moving_vec', 'heading_vec', 'head_moving']] = df_mot_slice[['Time', 'forward', 'moving_vec', 'heading_vec', 'head_moving']].copy()
    df_turn_forw = df[['turn_pc', 'forward']].copy()
    df[['turn_cor', 'forward_subs_turn']] = get_cor_turn_reversal(df_turn_forw)

    if visualize_turn:
        visualize_turn_labeling(df,path_ratio_uplim,dtheta_lim, cl_turn_size, op_turn_size)
        visualize_turn_correction(df, close_size=cl_turn_size, open_size=op_turn_size)

    # 计算曲率
    rev_sel_path = select_backbone(paths, closest_idx,pos_phr, longest_paths)
    curvature = {}
    head_vector = {}
    post_phr = {}
    phr_noses = {}
    for key, sel in rev_sel_path.items():
        pos_phr_i = pos_phr[key]
        re_pos_phr_i = [pos_phr_i[1], pos_phr_i[0]]
        df_bb_i, curvature_i, head_vector_i, post_phr_i, phr_noses_i = label_segments_curvature(sel,re_pos_phr_i, step=seg_pixel_step)
        curvature[key] = curvature_i
        head_vector[key] = head_vector_i
        post_phr[key] = post_phr_i
        phr_noses[key] = phr_noses_i
    df_cur = pd.Series(curvature, name='curvature')
    df_head_vec = pd.Series(head_vector, name='head_vector')
    df_sel = pd.Series(rev_sel_path, name='sel_paths')
    df['curvature'] = df_cur
    df['head_vector'] = df_head_vec
    df['sel_paths'] = df_sel
    df = cal_head_agl_velocity(df, window_size=agl_window, frame_rate=frame_rate)
    print('Columns:',df.columns)
    return df
    # 最后得到的中线df的key和运动相同，并且时间是一样的

## 单文件处理函数

### 函数定义

In [ ]:
def is_contained_or_not(p_f, label_corner = True):
    '''
    判断文件夹中是否包含所需要的文件
    '''
    if label_corner:
        required_files = ['stage_data.txt', 'c1.txt',
                        'upper_left.txt', 'lower_left.txt', 'upper_right.txt',
                        'lower_right.txt']   # 数据处理需要的文件
    else:   
        required_files = ['stage_data.txt', 'c1.txt',
                        'edge0.txt', 'edge45.txt']   # 数据处理需要的文件
    required_files_set = set(required_files)
    files = [f for f in os.listdir(p_f) if '.txt' in f ]
    files_set = set(files)
    is_contained = required_files_set.issubset(files_set)
    # 找出missing_files
    missing_files = [file for file in required_files if file not in files]
    return is_contained, missing_files

In [ ]:
def get_choose_frame(data, data_keys, data_iou):
    # 步骤零点五：根据骨架处理结果筛选符合条件的帧
    circle = np.array(list({key: value['circular'] for key, value in data.items()}.values())) 
    branching = np.array(list({key: value['branching'] for key, value in data.items()}.values())) 
    choose = np.array(list({key: value['choose_frame'] for key, value in data.items()}.values())) 
    circular_frames = np.array(data_keys)[circle == 1]
    branching_frames = np.array(data_keys)[branching == 1]
    merged = list(heapq.merge(circular_frames, branching_frames))
    choose_frame = np.array(data_keys)[choose == 1]
    # 识别面积抖动区间
    # iou = np.load(os.path.join(folder_path,'iou_results.npz'))['iou']
    iou = data_iou[data_iou.files[0]] 
    ano_f = np.vstack([[1, 1], iou]) < 0.7
    pre_f= np.where(ano_f[:,0] == True)[0]
    post_f= np.where(ano_f[:,1] == True)[0]
    ano_merged = list(heapq.merge(pre_f, post_f))
    # 排除IOU异常帧
    choose_frame = np.setdiff1d(choose_frame, ano_merged)
    return choose_frame

In [ ]:
def get_laser_on_and_off(s_data):
    # 提取每个激光照射起始和终止时刻
    s_start = s_data[s_data[1] == 2].index
    s_stop = s_data[s_data[1] == 3].index
    if s_stop.empty:
        s_s = [len(s_data[1])]
    else:
        s_s = []
        for m in s_start:
            # 如果有多个照射片段，分别找到结束的时间装入s_s
            data_1 = (s_stop >= m)
            s_s.append(s_stop[data_1 == 1][0])

    i=0  # 不考虑其它段，只考虑第一段
    #         t_l = int(round((s_data[0][s_start[i] + 1 : s_s[i] - 1]).diff().sum(skipna=True) / (s_s[i] - s_start[i] - 2)))
    t_l = (s_data[0][s_start[i] + 1 : s_s[i] - 1]).diff().sum(skipna=True) / (s_s[i] - s_start[i] - 2)
    print('时间间隔t_l',t_l)
    t = s_data[0].values
    # 取从激光照射开始前2帧到激光照射结束的数据
    t_frame = np.concatenate([[t[s_start[i]] - t_l *2, t[s_start[i]] - t_l], t[s_start[i]:s_s[i]]])
    t_start = s_data[0][s_start[i]] - t_l *2
    t_stop = s_data[0][s_s[i] - 1] + t_l
    return t_frame, t_start, t_stop

In [ ]:
# 对齐时间
def get_nearest_value(df1, df2, col_1 = 'Time', col_2 = 'Vol_Time', ind_col = 'Nearest_Index', col1_copy = 'Nearest_Time'):
    '''
    根据df2的col2找最接近的df1中的col1的值生成新列赋值与df2
    '''
    nearest_times = []
    nearest_indices = []
    for vol_time in df2[col_2]:
        # 计算时间差的绝对值
        differences = np.abs(df1[col_1] - vol_time)
        # 找到最小差值的索引
        nearest_idx = np.argmin(differences)
        # 保存最近时间值和索引
        nearest_times.append(df1.iloc[nearest_idx, df1.columns.get_loc(col_1)])
        nearest_indices.append(nearest_idx)

    # 将结果存入 DataFrame
    df2[col1_copy] = nearest_times
    df2[ind_col] = nearest_indices

    return df2

In [ ]:
# 加入condition
def add_condition_info(folder_name, df_matched):
    features = folder_name.split('_')
    if len(features) <3:
        print('文件名异常，未检测到Condition信息')
    else:
        # 如果Condition信息存在，在df_vol_time中加入condition信息
        df_matched['Date'] = features[0]
        df_matched['Rep_id'] = features[-1]
        condition_ls = features[-2].split('-')
        print(f'日期:{features[0]}, 编号：{features[-1]}\n Conditions:{condition_ls}')
        for i, c in enumerate(condition_ls):
            if len(condition_ls) > 1:
                con_col = 'Condition'+str(i)
                df_matched[con_col] = c
            else:
                df_matched['Condition'] = features[-2]
        if len(condition_ls) > 1:
            df_matched['Date_Con_Rep'] = features[-2]
    return df_matched

### 单文件

##### 1. 参数定义与数据检查

In [ ]:
p_f = r'Y:\SZX\\2025_wbi_analysis\\good_WBI\\new_midline_done\\20250225_0g-ov-24d_004'
label_corner = True
new_data = True   # 202502之后的数据是新数据，视频帧率为30-40Hz
half_width = 20   # 脂片宽度一半
x_shift  = 0 
hlf_spd_inv = 0.5
hlf_agl_spd_inv = 0.2
hlf_agl_inv = 1
Pos_CTX_vec = [-1,0]
t_crit = 3.8
min_agl_spd = 75
df_motion = {}
df_vol_time = {}
calcium = True   # 默认有荧光信号的所有文件都包含钙信号

In [ ]:
is_contained, missing_files = is_contained_or_not(p_f, label_corner=label_corner)
if is_contained:
    pass
else:
    print(f'missing files{missing_files}')

##### 2.钙信号数据处理和检查
+ 需要对时将钙信号帧转为时间信息方便查看异常帧/时间段

In [ ]:
# 数据读取(在步骤一已读取可省略)
basename = os.path.basename(os.path.normpath(p_f))
files = [f for f in os.listdir(p_f)]
for f in files:
    if ('laser-data' in f)or ("s-data" in f):
        # 读取激光激活数据
        print('读取激光照射时间数据并转为s_data')
        try:
            s_data = pd.read_csv(p_f + '\laser-data.txt', header=None)  #读取激光激活数据
        except FileNotFoundError:
            s_data = pd.read_csv(p_f + '\s-data.txt', header=None)
        # 注意：这里s_data已经转换成秒
        s_data.loc[:,0] = s_data.loc[:,0]/1000
    elif 'calcium_intensity.npy' in f:
                # 钙信号数据(* 如果有)
                print(f'读取钙信号数据：{p_f+f}')
                Cal_data_path = os.path.join(p_f, f)
                calcium_intensity= np.load(Cal_data_path)

# 根据激光开始和结束时间生成vol_time时间戳
t_frame, _, _ = get_laser_on_and_off(s_data)
df_vol_time = pd.DataFrame(t_frame, columns=['Vol_Time']) 


In [ ]:
def visualize_traces(calcium_intensity,smooth=True, sigma=0.3,delta_y=1.2 ):
    # 平滑和可视化
    n_neurons, _ = calcium_intensity.shape
    t_max = df_vol_time['Vol_Time'].max()
    plt.figure(figsize=(t_max/100, n_neurons/4))

    for i in range(n_neurons):
        trace = calcium_intensity[i, :]
        if smooth:
            trace = gaussian_filter1d(trace, sigma=sigma)
        plt.plot(df_vol_time['Vol_Time'], trace + i * delta_y,lw=0.5, label=f"Neuron {i}")

    # 设置坐标轴
    plt.xlabel("Time (frames)")
    plt.ylabel("Neuron index")
    desired_ticks = np.arange(0, df_vol_time['Vol_Time'].max(), 30) # 生成一个数组 [10, 30, 50, 70, 90]
    plt.xticks(desired_ticks, rotation=30)
    plt.yticks(np.arange(0, n_neurons * delta_y, delta_y), np.arange(n_neurons))
    plt.title(f"Calcium Traces (Gaussian smoothed){sigma}")
    plt.grid(True, linestyle='--', color='grey', linewidth=0.35, alpha=0.5)
    plt.show()
visualize_traces(calcium_intensity,smooth=True, sigma=0.3,delta_y=1.2 )

In [ ]:
def calcium_filter(calcium_traces,std_low = 0.05, std_high = 2, cvs_low=0.1, cvs_high=2):
    means = np.mean(calcium_traces, axis=1)
    stds = np.std(calcium_traces, axis=1)
    cvs = stds / (means + 1e-8)  # 避免除0

    # 条件：std 不太小、不太大，CV 也在合理范围
    valid_mask = (stds > std_low) & (stds < std_high) & (cvs > cvs_low) & (cvs < cvs_high)
    filtered_traces = calcium_traces[valid_mask]
    return filtered_traces

In [ ]:
filtered_cal = calcium_filter(calcium_intensity,std_low = 0.1, std_high = 0.3, cvs_low=0.5, cvs_high=1.5)
visualize_traces(filtered_cal,smooth=True, sigma=0.3,delta_y=1.2)

##### 3. 运动参数计算和视频对时

In [ ]:
# 数据读取
basename = os.path.basename(os.path.normpath(p_f))
files = [f for f in os.listdir(p_f)]
print(f'\n\n=======开始处理{basename}中文件，\n包含:{files}')
for f in files:
    if 'stage_data' in f:
        # 使用载物台位置信息计算速度角速度
        print('读取载物台位置信息文件')
        column_names = ['Time','p_X','p_Y', 'X', 'Y']
        df_stage = pd.read_csv(os.path.join(p_f, f),sep=r'\s+', header=None,
                                names=column_names)
        columns_to_clean = ['Time', 'p_X', 'p_Y', 'X', 'Y']
        for col in columns_to_clean:
            df_stage[col] = df_stage[col].astype(str).str.replace(',', '', regex=False).astype(float)

        df_motion = df_stage[['Time','X','Y']]/1000  # 转换单位为s和mm
        # 转换坐标，统一x轴坐标起点为0，终点为4.5
        # 保证p_f中有四个顶点文件
        if label_corner:
            df_label, df_motion = Realign_coordinate(p_f, df_motion)
        else:
            df_motion = Realign_coordinate_by_edge(p_f, df_motion)
    elif 'c1.txt' in f:
        # 读取视频帧文件
        print('读取视频帧时间戳文件并转为f_data')
        f_data = np.loadtxt(p_f + '\c1.txt', delimiter=',') # 读取相机时间戳
        if new_data: 
            # 202502之后的数据是新数据，视频帧率为30-40Hz
            t_c = f_data[f_data[:,3] == 1, 1]/1000
        else:
            # 旧数据视频帧率150Hz 每隔5帧取一帧（适用于老数据）
            indices = np.concatenate(([0], np.arange(5, len(f_data), 5)))
            t_c = f_data[indices, 1].astype(int)/1000
    elif ('laser-data' in f)or ("s-data" in f):
        # 读取激光激活数据
        print('读取激光照射时间数据并转为s_data')
        try:
            s_data = pd.read_csv(p_f + '\laser-data.txt', header=None)  #读取激光激活数据
        except FileNotFoundError:
            s_data = pd.read_csv(p_f + '\s-data.txt', header=None)
        # 注意：这里s_data已经转换成秒
        s_data.loc[:,0] = s_data.loc[:,0]/1000
    # elif 'calcium_intensity.npy' in f:
    #     # 钙信号数据(* 如果有)
    #     print(f'读取钙信号数据：{f}')
    #     Cal_data_path = os.path.join(p_f, f)
    #     calcium_intensity= np.load(Cal_data_path)
    #     Calcium = True    # 包含钙信号数据
    # elif 'output-0516' in f:
    #     # 中线npz数据(* 如果有)
    #     print(f'读取中线文件：{f}')
    #     midline_path = os.path.join(p_f,f)
    #     npz_data = np.load(midline_path, allow_pickle=True)
    #     data = npz_data['arr_0'].item()
    #     # paths= {key: value['all_paths'] for key, value in data.items()}
    #     data_keys = list(data.keys())   # 存储的符合条件的帧数
    #     midline_extract = True   # 是否包含抽取中线后的文件
    # elif 'iou_results' in f:
    #     print(f'读取中线处理相关文件{f}')
    #     data_iou = np.load(os.path.join(p_f,'iou_results.npz'))

In [ ]:
# 步骤一：根据载物台运动位置信息抽取运动参数
mean_frame_rate = np.median(1/np.diff(df_motion.Time.values))
print(np.argmin(np.diff(df_motion.Time.values)))
df_motion_als = Sliding_CTX_calation(df_motion, Pos_CTX_vec, hlf_spd_inv, hlf_agl_spd_inv, hlf_agl_inv,
                                    window_size=100, frame_rate=mean_frame_rate,X_shift=x_shift)


# 步骤二： 提取视频帧（根据中线处理结果只保留可用帧）对应的运动参数，输出对齐视频可用帧时间后的运动参数文件
df_motion_pt_sorted = df_motion_als.sort_values('Time')  # 确保 df_motion_pt 也是按时间升序
df_t = pd.DataFrame({'Time': t_c})
df_t = df_t.sort_values('Time')  # merge_asof 要求升序
df_motion_pt_sorted = df_motion_pt_sorted.rename(columns={'Time': 'Stage_time'})
# 使用 merge_asof 匹配最近的时间点
df_matched = pd.merge_asof(
    df_t,
    
    df_motion_pt_sorted,
    left_on='Time',
    right_on='Stage_time',
    direction='nearest'  # 可选：'backward', 'forward', or 'nearest'
)

# 步骤三：加入condition
folder_name = os.path.basename(os.path.normpath(p_f))
df_matched = add_condition_info(folder_name, df_matched)
# 重置中线位置
df_matched['Disp_to_mid'] = df_matched['X']-half_width
# 保存合并运动参数和视频帧对应时间戳的df数据
df_matched.to_csv(os.path.join(p_f,basename+ '_mot_vid.csv'), index=False)
'''文件长度与视频帧数相同'''


##### 2. 中线数据处理和输出

In [ ]:
# 数据读取（中线）
basename = os.path.basename(os.path.normpath(p_f))
files = [f for f in os.listdir(p_f)]
print(f'\n\n=======开始处理{basename}中文件')
for f in files:
    if 'output-0516' in f:
        # 中线npz数据(* 如果有)
        print(f'读取中线文件：{f}')
        midline_path = os.path.join(p_f,f)
        npz_data = np.load(midline_path, allow_pickle=True)
        data = npz_data['arr_0'].item()
        # paths= {key: value['all_paths'] for key, value in data.items()}
        data_keys = list(data.keys())   # 存储的符合条件的帧数
        midline_extract = True   # 是否包含抽取中线后的文件
    elif 'iou_results' in f:
        print(f'读取中线处理相关文件{f}')
        data_iou = np.load(os.path.join(p_f,'iou_results.npz'))

In [ ]:
'''与视频帧数相同的完成对时的运动数据df_matched'''
# 这一步可以直接接上一步，或者从文件夹读取'_mot_vid.csv'
df_matched = df_matched

# 提取可靠帧
choose_frame = get_choose_frame(data, data_keys, data_iou)
df_choose = pd.DataFrame(choose_frame, columns=['key'])
df_choose.loc[:,'choose_frame'] = int(1)
df_choose = df_choose.set_index('key')

# 提取所需中线信息
rows = []
for key in data_keys:
    item = data[key]
    row = {
        'key': key,   # 对应视频帧
        'angle_m': item['angle_m'],  # 头部朝向与运动方向夹角绝对值
        'angle_md': item['angle_md'],  # 头部朝向与运动方向夹角
        'all_paths':item['all_paths'],  # 完整的骨架信息
        'closest_idx':item['closest path'], # 最近的骨架id
    }
    rows.append(row)
df_summary = pd.DataFrame(rows).set_index('key')


# 保留中线数据
# 保存筛选帧信息：哪些中线的置信度高于阈值
df_merged = df_matched.join(df_summary, how='left')
df_merged = df_merged.join(df_choose, how='left')   # 根据index合并，df_choose里没有的填none

# 中线处理，计算turn
# 输出包含‘Time’作为唯一重合列
df_turn_curv = get_omega_turn_curvature(df_merged, midline_path, path_ratio_uplim = 1.5, path_uplim=200, dtheta_lim=220,
                            cl_turn_size = 50, op_turn_size = 50, visualize_turn=True,
                            seg_pixel_step = 13, agl_window = 300, frame_rate=38)

# df_merged_2 = df_merged.join(df_turn_curv, how='left')
df_merged_2 = pd.merge(df_merged, df_turn_curv, on='Time', how='left')
df_merged_2.to_csv(os.path.join(p_f,basename+ '_mot_vid_mid.csv'), index=False)
'''文件仍然与视频长度相同，包含所有中线处理信息以及筛选帧信息'''
'''写出df_matched: 运动参数;df_merged_2:包含运动参数和所有中线相关信息'''

##### *中线处理替代（get_omega_turn_curvature）

In [ ]:
# 输入中线地址，和运动参数df
# 输出包含中线相关参数（以及时间列‘Time’）的df
mid_line_path = midline_path
df_merged = df_merged
path_ratio_uplim = 1.5
path_uplim=200
dtheta_lim=220
cl_turn_size = 50
op_turn_size = 50
cl_rev_size = 50
op_rev_size = 50
visualize_turn=True
seg_pixel_step = 13
agl_window = 300
frame_rate=38
'''
mid_line_path:中线文件路径
'''
npz_data = np.load(mid_line_path, allow_pickle=True)
data = npz_data['arr_0'].item()
# data_keys = list(data.keys())
paths= {key: value['all_paths'] for key, value in data.items()}
closest_idx = {key: value['closest path'] for key, value in data.items()}
pos_phr = {key: value['pos_phr'] for key, value in data.items()}
# angle_m = {key: value['angle_m'] for key, value in data.items()}
vector_h = {key: value['vector_h'] for key, value in data.items()} 
circular = {key: value['circular'] for key, value in data.items()}
branching = {key: value['branching'] for key, value in data.items()}

# 前进后退
df_mot_slice = df_merged[['Time','X', 'Y',"x_velocity","y_velocity", 'angle_m','angle_md']]
df_mot_slice = label_smth_forward_reverse(df_mot_slice, close_size = cl_rev_size, open_size = op_rev_size)
# 运动方向向量
df_mot_slice.loc[:,'moving_vec'] = df_mot_slice.apply(lambda x: (x['x_velocity'], x['y_velocity']),axis = 1)
# 头部朝向向量
df_mot_slice.loc[:,'heading_vec'] = df_mot_slice.apply(lambda x: rotation_mat_2(x['angle_md'], x['moving_vec']), axis=1)
# 求投影头部朝向后的运动向量
df_mot_slice.loc[:,'head_moving'] = df_mot_slice.apply(lambda x: project_vector_A_on_B(x['moving_vec'], x['heading_vec']), axis=1)

# 骨架拆分
seq_paths = {}
for key, value in paths.items():
    # print(merge_and_find_non_branching(value, quant=3))
    seq_path_i, _ = merge_and_find_non_branching(
        value,
        quant=3   # 若不同段的交点不完全重合，可设 quant=2 或 3 像素，把近点聚成同一节点
    )
    seq_paths[key] = seq_path_i
# 骨架合并，得到最长骨架
longest_paths = {}
for key, value in seq_paths.items():
    longest_coords = longest_path(value)
    longest_paths[key] = longest_coords

# 对最常骨架重采样计算曲率
resampled, dtheta = resample_and_dthatas(longest_paths,step=20)
sum_dtheta = {
    key: (value['sum_dtheta'] if isinstance(value, dict) and 'sum_dtheta' in value else None)
    for key, value in dtheta.items()
    }
# 曲率计算结果生成df
df_sum_dtheta = pd.Series(sum_dtheta, name='sum_dtheta')
print(f'df_sum_dtheta的长度{len(df_sum_dtheta)}和最大index{df_sum_dtheta.index.max()}')
# 计算路径比例
df = path_ratio(paths)
df['sum_dtheta'] = df_sum_dtheta
# 合并环形和分叉信息
df_cir_branching = pd.DataFrame.from_dict(
    {"circular": circular, "branching": branching},
    orient="index").T
df[["circular", "branching"]] = df_cir_branching[["circular", "branching"]]

# 分类turn
df['path_lim'] = df['path_len'].apply(
    lambda x: 1 if (isinstance(x, (list, tuple, np.ndarray)) and len(x) > 1 and x[1] >= path_uplim) else 0
)
df['turn'] = 0
df.loc[
    (df['circular'] == 1) |
    (df['branching'] == 1) |
    ((df['path_ratio'] <= path_ratio_uplim)&(df['path_lim'] == 1)) |
    (df['sum_dtheta'].abs() >= dtheta_lim)
    ,
    'turn'] = 1
# 平滑turn
closed = ndimage.binary_closing(df['turn'].values, structure=np.ones(cl_turn_size))
# 再进行开操作（先腐蚀后膨胀）：去除小的噪声点
opened = ndimage.binary_opening(closed, structure=np.ones(op_turn_size))
# 将结果转换为整数并添加到 DataFrame
df['turn_pc'] = opened.astype(int)


# 合并前进后退信息
df[['Time','forward', 'moving_vec', 'heading_vec', 'head_moving']] = df_mot_slice[['Time', 'forward', 'moving_vec', 'heading_vec', 'head_moving']]
df_turn_forw = df[['turn_pc', 'forward']]
df[['turn_cor', 'forward_subs_turn']] = get_cor_turn_reversal(df_turn_forw)

if visualize_turn:
    visualize_turn_labeling(df,path_ratio_uplim,dtheta_lim, cl_turn_size, op_turn_size)
    visualize_turn_correction(df, close_size=cl_turn_size, open_size=op_turn_size)

# 计算曲率
rev_sel_path = select_backbone(paths, closest_idx,pos_phr, longest_paths)
curvature = {}
head_vector = {}
post_phr = {}
phr_noses = {}
for key, sel in rev_sel_path.items():
    pos_phr_i = pos_phr[key]
    re_pos_phr_i = [pos_phr_i[1], pos_phr_i[0]]
    df_bb_i, curvature_i, head_vector_i, post_phr_i, phr_noses_i = label_segments_curvature(sel,re_pos_phr_i, step=seg_pixel_step)
    curvature[key] = curvature_i
    head_vector[key] = head_vector_i
    post_phr[key] = post_phr_i
    phr_noses[key] = phr_noses_i
df_cur = pd.Series(curvature, name='curvature')
df_head_vec = pd.Series(head_vector, name='head_vector')
df_sel = pd.Series(rev_sel_path, name='sel_paths')
df['curvature'] = df_cur
df['head_vector'] = df_head_vec
df['sel_paths'] = df_sel
df = cal_head_agl_velocity(df, window_size=agl_window, frame_rate=frame_rate)
print(df.columns)
# 最后得到的中线df的key和运动相同，并且时间是一样的

##### 4. 中线和运动数据裁剪以及与荧光对时

In [ ]:
def cut_df_on_laser_t(df,p_f, basename, t_start, t_stop, ext_name):
    '''输出df_cut保存csv文件并返回这个df'''
    track_start_index = np.searchsorted(df.Time.values, t_start) - 1   # 截取激光照射段的载物台数据时间戳
    track_end_index = np.searchsorted(df.Time.values, t_stop)
    df_cut = df.loc[track_start_index:track_end_index, :]
    df_cut.to_csv(os.path.join(p_f, basename+ext_name), index = False)
    return df_cut

In [ ]:
# 数据读取(在步骤一已读取可省略)
basename = os.path.basename(os.path.normpath(p_f))
files = [f for f in os.listdir(p_f)]
for f in files:
    if ('laser-data' in f)or ("s-data" in f):
        # 读取激光激活数据
        print('读取激光照射时间数据并转为s_data')
        try:
            s_data = pd.read_csv(p_f + '\laser-data.txt', header=None)  #读取激光激活数据
        except FileNotFoundError:
            s_data = pd.read_csv(p_f + '\s-data.txt', header=None)
        # 注意：这里s_data已经转换成秒
        s_data.loc[:,0] = s_data.loc[:,0]/1000

# 步骤五：根据激光开始和结束时间截取荧光拍摄部分的运动数据/时间戳
t_frame, t_start, t_stop = get_laser_on_and_off(s_data)
df_matched_cut = cut_df_on_laser_t(df_matched,p_f, basename, t_start, t_stop, ext_name='_mot_vid_cut.csv')
df_mot_midline_cut = cut_df_on_laser_t(df_merged_2,p_f, basename, t_start, t_stop, ext_name='_mot_midline_cut.csv')


In [154]:
def get_df_mot_matched_vol_t(df_mat_cut,p_f, df_vol, ext_name = '', write=True):
    '''
    荧光对时: 根据df_vol_time中'Time'提取df_mat_cut中的数据列并写出
    '''
    folder_name = os.path.basename(os.path.normpath(p_f))
    df_vol = get_nearest_value(df_mat_cut, df_vol, col_1 = 'Time', col_2 = 'Vol_Time', ind_col = 'Nearest_Index', col1_copy = 'Nearest_Time')
    # 将df_motionz中的运动参数(有中线加上中线)按照索引对应到视频帧文件的时间戳上
    columns_to_add = df_mat_cut.columns
    for col in columns_to_add:
        df_vol[col] = df_mat_cut.iloc[df_vol['Nearest_Index'].values][col].values
        fn_motion_output = folder_name+ext_name
    if write:
        df_vol.to_csv(os.path.join(p_f, fn_motion_output), index = False)
    print(f'写出与神经数据时间戳对应的运动/中线参数文件到:{fn_motion_output}')
    return df_vol
    

In [171]:
# 步骤六：根据荧光数据时间戳提取对应的运动参数，便于后续合并荧光数据
df_vol_time = pd.DataFrame(t_frame, columns=['Vol_Time'])   # 激光照射时间戳要转换单位为s
df_vol_time.to_csv(os.path.join(p_f, folder_name+'_VotTime.csv'), index = False)
# 有中线, 如果需要mask，write = False
df_mot_mid_vol = get_df_mot_matched_vol_t(df_mot_midline_cut,p_f, df_vol_time.copy(),
                                           ext_name = '_MotionMidlineMatchVol.csv', write=True)
# 如果没有中线
# df_mot_vol = get_df_mot_matched_vol_t(df_matched_cut,p_f, df_vol_time, ext_name = '_MotionMatchVol.csv')

写出与神经数据时间戳对应的运动/中线参数文件到:20250225_0g-ov-24d_004_MotionMidlineMatchVol.csv


#### 5. 视频检查
视频检查应该放在中线和运动数据处理完后，裁剪前完成
视频检查的目的是查看turn_pc, turn_cor, reversal标记正确与否
以及查看是否有追错等异常帧，标记这些异常片段
检查时可以参考前calcium_trace处理部分记录的可能的异常片段

In [ ]:
df_merged_2.columns

In [ ]:
def normalize_points(points):
    """确保骨架点是 (N,2) int32 数组"""
    arr = np.array(points, dtype=np.float32)
    arr = np.vstack(arr)                # 去掉长度为1的维度

    arr = arr.astype(np.int32)           # OpenCV 要 int32
    return arr

In [150]:
# 读取视频
cap = cv2.VideoCapture(os.path.join(p_f, "c1_onnx.mp4"))
# cap = cv2.VideoCapture(os.path.join(folder_path, "c1_onnx.mp4"))
df = df_merged_2  
# 或者从文件夹中读取，行数与视频帧数相同

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"视频总帧数{total_frames}是否等于df行数:", total_frames==len(df))
# 计算头部朝向时的切割像素长度
body_l = 200
delay = 50
step = 5
start_frame = 20000
# 结束帧
frame_idx_end = total_frames
cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
error_ls = []

for frame_idx in range(start_frame, total_frames, step):
    # 视频定位当前帧
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)  # 定位到 frame_idx
    ret, frame = cap.read()
    if frame is None:
        continue
    if not ret:
        break
    # 显示帧索引
    cv2.putText(frame, f"Frame: {frame_idx}", (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
    # 画出向量
    color_vector_h = (128, 0, 128)  # 紫色
    color_vector_m = (0, 165, 255)    # 明黄色
    color_vector_mh= (216, 191, 216) # 淡紫色
    
    row = df.iloc[frame_idx]
    # 时间
    time_i = row.Time
    cv2.putText(frame, f"Time:{time_i}s", (10, 50), 
            cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
    # 读取咽喉点坐标
    angle_md = round(row.angle_md,2)
    cv2.putText(frame, f"angle_md:{angle_md}", (250, 30), 
            cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
    
    # 骨架
    sel_path = row.sel_paths
   
    # 校正turn
    if row.turn_cor == 1:
        cv2.putText(frame, f"turn_cor", (250, 50), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
    else:
        cv2.putText(frame, f"turn_cor", (250, 50), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.75, (200, 200, 200), 2)
    # 后退
    if row.forward == 1:
        cv2.putText(frame, f"reversal", (250, 70), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 0, 0), 2)
    else:
        cv2.putText(frame, f"forward", (250, 70), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.75, (200, 200, 200), 2)
    if row.turn_pc == 1:
        color_bb = (0,0,255)
    else:
        color_bb = color_vector_m
    if isinstance(sel_path, np.ndarray):
        points_raw = sel_path
        midline_points = normalize_points(points_raw)  # 返回 (N,2) int32
        j = 0
        for (x, y) in midline_points:
            # 打印第一帧
            if j == 0:
                cv2.circle(frame, (y,x), 10, (255,0,255), thickness=-1)
            cv2.circle(frame, (y,x), 3, color_bb, -1)
            j += 1
        i += 1
    else:
        print('不存在骨架:frame',frame_idx)
        continue
        
    cv2.imshow("Skeleton Check", frame)
    key = cv2.waitKey(delay) & 0xFF   # 播放速度 (30ms/帧)，也可以改大
    if key == ord('q'):   # 按 q 退出
        break
    elif key == ord(' '): # 按空格暂停
        cv2.waitKey(0)

cap.release()
cv2.destroyAllWindows()


视频总帧数23824是否等于df行数: True
不存在骨架:frame 20020
不存在骨架:frame 20025
不存在骨架:frame 20035
不存在骨架:frame 20050
不存在骨架:frame 20520
不存在骨架:frame 20550
不存在骨架:frame 20565
不存在骨架:frame 20580
不存在骨架:frame 20595
不存在骨架:frame 20610
不存在骨架:frame 20625
不存在骨架:frame 20640
不存在骨架:frame 20650
不存在骨架:frame 20680
不存在骨架:frame 20695
不存在骨架:frame 20705
不存在骨架:frame 20720
不存在骨架:frame 20735
不存在骨架:frame 21390
不存在骨架:frame 21420
不存在骨架:frame 21905
不存在骨架:frame 21910
不存在骨架:frame 21915
不存在骨架:frame 21920
不存在骨架:frame 21925
不存在骨架:frame 21930
不存在骨架:frame 21935
不存在骨架:frame 21940
不存在骨架:frame 21945
不存在骨架:frame 21950
不存在骨架:frame 21955
不存在骨架:frame 21960
不存在骨架:frame 21965
不存在骨架:frame 21970
不存在骨架:frame 21975
不存在骨架:frame 21980
不存在骨架:frame 21985
不存在骨架:frame 21990
不存在骨架:frame 21995
不存在骨架:frame 22000
不存在骨架:frame 22005
不存在骨架:frame 22010
不存在骨架:frame 22015
不存在骨架:frame 22020
不存在骨架:frame 22025
不存在骨架:frame 22030
不存在骨架:frame 22035
不存在骨架:frame 22040
不存在骨架:frame 22045
不存在骨架:frame 22050
不存在骨架:frame 22055
不存在骨架:frame 22060
不存在骨架:frame 22065
不存在骨架:frame 22070
不存在

#### 6. 对异常数据mask并打印钙信号检查
根据可能的追踪异常情况，将4中得到的荧光对时文件打mask，提示异常帧

In [ ]:
folder_name = os.path.basename(os.path.normpath(p_f))
df_vol_time.loc[:,'mask'] = 0 
df_vol_time.loc[df_vol_time['Vol_Time']>= 685,'mask'] = 1
df_vol_time.to_csv(os.path.join(p_f, folder_name+'_VotTime.csv'), index = False)

# 有中线的vol_time对齐文件
df_mot_mid_vol.loc[:,'mask'] = 0 
df_mot_mid_vol.loc[df_mot_mid_vol['Vol_Time']>= 685,'mask'] = 1
df_mot_mid_vol.to_csv(os.path.join(p_f, folder_name+'_MotionMidlineMatchVol.csv'), index = False)


### 单文件处理函数（旧）

In [ ]:
p_f = r'Y:\SZX\\2025_wbi_analysis\\good_WBI\\new_midline_done\\20250221_0g-ov-27.5d_6'
def single_wbi_data_analysis(p_f, label_corner = True, new_data = True, half_width = 20, x_shift  = 0,
                              hlf_spd_inv = 0.5, hlf_agl_spd_inv = 0.2, hlf_agl_inv = 1,
                                Pos_CTX_vec = [-1,0]):
    
    # 检查文件
    is_contained, missing_files = is_contained_or_not(p_f, label_corner=label_corner)
    if is_contained:
        pass
    else:
        print(f'missing files{missing_files}')
        return None

    # 数据读取
    basename = os.path.basename(os.path.normpath(p_f))
    files = [f for f in os.listdir(p_f)]
    print(f'\n\n=======开始处理{basename}中文件，\n包含:{files}')
    for f in files:
        if 'stage_data' in f:
            # 使用载物台位置信息计算速度角速度
            print('读取载物台位置信息文件')
            column_names = ['Time','p_X','p_Y', 'X', 'Y']
            df_stage = pd.read_csv(os.path.join(p_f, f),sep=r'\s+', header=None,
                                    names=column_names)
            columns_to_clean = ['Time', 'p_X', 'p_Y', 'X', 'Y']
            for col in columns_to_clean:
                df_stage[col] = df_stage[col].astype(str).str.replace(',', '', regex=False).astype(float)

            df_motion = df_stage[['Time','X','Y']]/1000  # 转换单位为s和mm
            # 转换坐标，统一x轴坐标起点为0，终点为4.5
            # 保证p_f中有四个顶点文件
            if label_corner:
                df_label, df_motion = Realign_coordinate(p_f, df_motion)
            else:
                df_motion = Realign_coordinate_by_edge(p_f, df_motion)
        elif 'c1.txt' in f:
            # 读取视频帧文件
            print('读取视频帧时间戳文件并转为f_data')
            f_data = np.loadtxt(p_f + '\c1.txt', delimiter=',') # 读取相机时间戳
            if new_data: 
                # 202502之后的数据是新数据，视频帧率为30-40Hz
                t_c = f_data[f_data[:,3] == 1, 1]/1000
            else:
                # 旧数据视频帧率150Hz 每隔5帧取一帧（适用于老数据）
                indices = np.concatenate(([0], np.arange(5, len(f_data), 5)))
                t_c = f_data[indices, 1].astype(int)/1000
        elif ('laser-data' in f)or ("s-data" in f): 
            # 读取激光激活数据
            print('读取激光照射时间数据并转为s_data')
            try:
                s_data = pd.read_csv(p_f + '\laser-data.txt', header=None)  #读取激光激活数据
            except FileNotFoundError:
                s_data = pd.read_csv(p_f + '\s-data.txt', header=None)
            # 注意：这里s_data已经转换成秒
            s_data.loc[:,0] = s_data.loc[:,0]/1000
        elif 'output-0516' in f:
            # 中线npz数据(* 如果有)
            print(f'读取中线文件：{f}')
            midline_path = os.path.join(p_f,f)
            npz_data = np.load(midline_path, allow_pickle=True)
            data = npz_data['arr_0'].item()
            # paths= {key: value['all_paths'] for key, value in data.items()}
            data_keys = list(data.keys())   # 存储的符合条件的帧数
            midline_extract = True   # 是否包含抽取中线后的文件
        elif 'iou_results' in f:
            print(f'读取中线处理相关文件{f}')
            data_iou = np.load(os.path.join(p_f,'iou_results.npz'))
        
            
    # 步骤一：根据载物台运动位置信息抽取运动参数
    mean_frame_rate = np.median(1/np.diff(df_motion.Time.values))
    print(np.argmin(np.diff(df_motion.Time.values)))
    df_motion_als = Sliding_CTX_calation(df_motion, Pos_CTX_vec, hlf_spd_inv, hlf_agl_spd_inv, hlf_agl_inv,
                                        window_size=100, frame_rate=mean_frame_rate,X_shift=x_shift)


    # 步骤二： 提取视频帧（根据中线处理结果只保留可用帧）对应的运动参数，输出对齐视频可用帧时间后的运动参数文件
    df_motion_pt_sorted = df_motion_als.sort_values('Time')  # 确保 df_motion_pt 也是按时间升序
    df_t = pd.DataFrame({'Time': t_c})
    df_t = df_t.sort_values('Time')  # merge_asof 要求升序
    df_motion_pt_sorted = df_motion_pt_sorted.rename(columns={'Time': 'Stage_time'})
    # 使用 merge_asof 匹配最近的时间点
    df_matched = pd.merge_asof(
        df_t,
        df_motion_pt_sorted,
        left_on='Time',
        right_on='Stage_time',
        direction='nearest'  # 可选：'backward', 'forward', or 'nearest'
    )

    # 步骤三：加入condition
    folder_name = os.path.basename(os.path.normpath(p_f))
    df_matched = add_condition_info(folder_name, df_matched)
    # 重置中线位置
    df_matched['Disp_to_mid'] = df_matched['X']-half_width
    # 保存合并运动参数和视频帧对应时间戳的df数据
    df_matched.to_csv(os.path.join(p_f,basename+ '_mot_vid.csv'), index=False)
    '''文件长度与视频帧数相同'''

    # 步骤四:合并中线数据
    if midline_extract:

        # 整理choose帧
        choose_frame = get_choose_frame(data, data_keys, data_iou)
        df_choose = pd.DataFrame(choose_frame, columns=['key'])
        df_choose.loc[:,'choose_frame'] = int(1)
        df_choose = df_choose.set_index('key')

        rows = []
        for key in data_keys:
            item = data[key]
            row = {
                'key': key,   # 对应视频帧
                'angle_m': item['angle_m'],  # 头部朝向与运动方向夹角绝对值
                'angle_md': item['angle_md'],  # 头部朝向与运动方向夹角
                'all_paths':item['all_paths'],  # 完整的骨架信息
                'closest_idx':item['closest path'], # 最近的骨架id
            }
            rows.append(row)
        # 构建 DataFrame
        df_summary = pd.DataFrame(rows).set_index('key')
        # 保留所有的处理后的帧信息
        df_merged = df_matched.join(df_summary, how='left')

        # 保存筛选帧信息：哪些中线的置信度高于阈值
        df_merged = df_merged.join(df_choose, how='left')   # 根据index合并，df_choose里没有的填none

        # 中线处理，计算turn
        df_turn_curv = get_omega_turn_curvature(midline_path, path_ratio_uplim = 1.5, path_uplim=200, dtheta_lim=220,
                                cl_turn_size = 50, op_turn_size = 50, visualize_turn=True,
                                seg_pixel_step = 13, agl_window = 300, frame_rate=38)
        df_merged_2 = df_merged.join(df_turn_curv, how='left')

        df_merged_2.to_csv(os.path.join(p_f,basename+ '_mot_vid_mid.csv'), index=False)
        '''文件仍然与视频长度相同，包含所有中线处理信息以及筛选帧信息'''

    # 步骤五：根据激光开始和结束时间截取荧光拍摄部分的运动数据/时间戳
    t_frame, t_start, t_stop = get_laser_on_and_off(s_data)
    track_start_index = np.searchsorted(df_matched.Time.values, t_start) - 1   # 截取激光照射段的载物台数据时间戳
    track_end_index = np.searchsorted(df_matched.Time.values, t_stop)
    df_matched_cut = df_matched.loc[track_start_index:track_end_index, :]
    df_vol_time = pd.DataFrame(t_frame, columns=['Vol_Time'])   # 激光照射时间戳要转换单位为s
    stage_file = os.path.join(p_f, f's{0+1}.txt')
    vol_file = os.path.join(p_f, f't{0+1}.txt')
    np.savetxt(stage_file, df_matched_cut[['Time','X_org','Y_org']], fmt='%.4f')  # 存储载物台数据(时间戳是视频帧的)
    np.savetxt(vol_file, t_frame, fmt='%.4f')  # 存储荧光每一个volume的数据
    df_matched_cut.to_csv(os.path.join(p_f, basename+'_mot_vid_cut.csv'), index = False)
    if midline_extract:
        df_mot_midline_cut = df_merged_2.loc[track_start_index:track_end_index, :]
        df_mot_midline_cut.to_csv(os.path.join(p_f, basename+'_mot_midline_cut.csv'), index = False)

    # 步骤六：根据荧光数据时间戳提取对应的运动参数，便于后续合并荧光数据
    # 如果有中线数据，就用带中线数据的对齐。否则只用运动参数对齐
    if midline_extract:
        df_mat_cut = df_mot_midline_cut.copy()
        print('有中线数据, 与Volume对齐的运动和中线数据')
    else:
        df_mat_cut = df_matched_cut.copy()
        print('没有中线数据, 输出与Volume时间对齐的运动参数')

    df_vol_time = get_nearest_value(df_mat_cut, df_vol_time, col_1 = 'Time', col_2 = 'Vol_Time', ind_col = 'Nearest_Index', col1_copy = 'Nearest_Time')
    # 将df_motionz中的运动参数(有中线加上中线)按照索引对应到视频帧文件的时间戳上
    columns_to_add = df_mat_cut.columns
    for col in columns_to_add:
        df_vol_time[col] = df_mat_cut.iloc[df_vol_time['Nearest_Index'].values][col].values
    if midline_extract:
        fn_motion_output = folder_name+'_MotionMidlineMatchVol.csv'
    else:
        fn_motion_output = folder_name+'_MotionMatchVol.csv'
    df_vol_time.to_csv(os.path.join(p_f, fn_motion_output), index = False)
    print(f'写出与神经数据时间戳(不包含钙数据)对应的运动参数文件到:{folder_name}/{fn_motion_output}')

In [ ]:
# 函数调用
file_path_all = r'Y:\\SZX\\2025_wbi_analysis\\good_WBI\\no_binary\\new'
key_word = ''
nokey_word = '*'
file_paths = [os.path.join(file_path_all,f_p) for f_p in os.listdir(file_path_all) if ('trash' not in f_p)&
              ('done' not in f_p)&
             os.path.isdir(os.path.join(file_path_all,f_p))&
             (key_word in f_p)&(nokey_word not in f_p)]

#### 单文件处理，旧代码

In [ ]:
#### 单文件处理，旧代码
p_f = r'Y:\SZX\\2025_wbi_analysis\\good_WBI\\new_midline_done\\20250221_0g-ov-27.5d_6'
label_corner = True
new_data = True   # 202502之后的数据是新数据，视频帧率为30-40Hz
half_width = 20   # 脂片宽度一半
x_shift  = 0 
hlf_spd_inv = 0.5
hlf_agl_spd_inv = 0.2
hlf_agl_inv = 1
Pos_CTX_vec = [-1,0]
t_crit = 3.8
min_agl_spd = 75
df_motion = {}
df_vol_time = {}
calcium = True   # 默认有荧光信号的所有文件都包含钙信号

is_contained, missing_files = is_contained_or_not(p_f, label_corner=label_corner)
if is_contained:
    pass
else:
    print(f'missing files{missing_files}')


# 数据读取
basename = os.path.basename(os.path.normpath(p_f))
files = [f for f in os.listdir(p_f)]
print(f'\n\n=======开始处理{basename}中文件，\n包含:{files}')
for f in files:
    if 'stage_data' in f:
        # 使用载物台位置信息计算速度角速度
        print('读取载物台位置信息文件')
        column_names = ['Time','p_X','p_Y', 'X', 'Y']
        df_stage = pd.read_csv(os.path.join(p_f, f),sep=r'\s+', header=None,
                                names=column_names)
        columns_to_clean = ['Time', 'p_X', 'p_Y', 'X', 'Y']
        for col in columns_to_clean:
            df_stage[col] = df_stage[col].astype(str).str.replace(',', '', regex=False).astype(float)

        df_motion = df_stage[['Time','X','Y']]/1000  # 转换单位为s和mm
        # 转换坐标，统一x轴坐标起点为0，终点为4.5
        # 保证p_f中有四个顶点文件
        if label_corner:
            df_label, df_motion = Realign_coordinate(p_f, df_motion)
        else:
            df_motion = Realign_coordinate_by_edge(p_f, df_motion)

    elif 'c1.txt' in f:
        # 读取视频帧文件
        print('读取视频帧时间戳文件并转为f_data')
        f_data = np.loadtxt(p_f + '\c1.txt', delimiter=',') # 读取相机时间戳
        if new_data: 
            # 202502之后的数据是新数据，视频帧率为30-40Hz
            t_c = f_data[f_data[:,3] == 1, 1]/1000
        else:
            # 旧数据视频帧率150Hz 每隔5帧取一帧（适用于老数据）
            indices = np.concatenate(([0], np.arange(5, len(f_data), 5)))
            t_c = f_data[indices, 1].astype(int)/1000
    elif ('laser-data' in f)or ("s-data" in f):
        # 读取激光激活数据
        print('读取激光照射时间数据并转为s_data')
        try:
            s_data = pd.read_csv(p_f + '\laser-data.txt', header=None)  #读取激光激活数据
        except FileNotFoundError:
            s_data = pd.read_csv(p_f + '\s-data.txt', header=None)
        # 注意：这里s_data已经转换成秒
        s_data.loc[:,0] = s_data.loc[:,0]/1000
    elif 'calcium_intensity.npy' in f:
        # 钙信号数据(* 如果有)
        print(f'读取钙信号数据：{f}')
        Cal_data_path = os.path.join(p_f, f)
        calcium_intensity= np.load(Cal_data_path)
        Calcium = True    # 包含钙信号数据
    elif 'output-0516' in f:
        # 中线npz数据(* 如果有)
        print(f'读取中线文件：{f}')
        midline_path = os.path.join(p_f,f)
        npz_data = np.load(midline_path, allow_pickle=True)
        data = npz_data['arr_0'].item()
        # paths= {key: value['all_paths'] for key, value in data.items()}
        data_keys = list(data.keys())   # 存储的符合条件的帧数
        midline_extract = True   # 是否包含抽取中线后的文件
    elif 'iou_results' in f:
        print(f'读取中线处理相关文件{f}')
        data_iou = np.load(os.path.join(p_f,'iou_results.npz'))
    
        
# 步骤一：根据载物台运动位置信息抽取运动参数
mean_frame_rate = np.median(1/np.diff(df_motion.Time.values))
print(np.argmin(np.diff(df_motion.Time.values)))
df_motion_als = Sliding_CTX_calation(df_motion, Pos_CTX_vec, hlf_spd_inv, hlf_agl_spd_inv, hlf_agl_inv,
                                    window_size=100, frame_rate=mean_frame_rate,X_shift=x_shift)


# 步骤二： 提取视频帧（根据中线处理结果只保留可用帧）对应的运动参数，输出对齐视频可用帧时间后的运动参数文件
df_motion_pt_sorted = df_motion_als.sort_values('Time')  # 确保 df_motion_pt 也是按时间升序
df_t = pd.DataFrame({'Time': t_c})
df_t = df_t.sort_values('Time')  # merge_asof 要求升序
df_motion_pt_sorted = df_motion_pt_sorted.rename(columns={'Time': 'Stage_time'})
# 使用 merge_asof 匹配最近的时间点
df_matched = pd.merge_asof(
    df_t,
    df_motion_pt_sorted,
    left_on='Time',
    right_on='Stage_time',
    direction='nearest'  # 可选：'backward', 'forward', or 'nearest'
)

# 步骤三：加入condition
folder_name = os.path.basename(os.path.normpath(p_f))
df_matched = add_condition_info(folder_name, df_matched)
# 重置中线位置
df_matched['Disp_to_mid'] = df_matched['X']-half_width
# 保存合并运动参数和视频帧对应时间戳的df数据
df_matched.to_csv(os.path.join(p_f,basename+ '_mot_vid.csv'), index=False)
'''文件长度与视频帧数相同'''

# 步骤四:合并中线数据
if midline_extract:

    # 整理choose帧
    choose_frame = get_choose_frame(data, data_keys, data_iou)
    df_choose = pd.DataFrame(choose_frame, columns=['key'])
    df_choose.loc[:,'choose_frame'] = int(1)
    df_choose = df_choose.set_index('key')

    rows = []
    for key in data_keys:
        item = data[key]
        row = {
            'key': key,   # 对应视频帧
            'angle_m': item['angle_m'],  # 头部朝向与运动方向夹角绝对值
            'angle_md': item['angle_md'],  # 头部朝向与运动方向夹角
            'all_paths':item['all_paths'],  # 完整的骨架信息
            'closest_idx':item['closest path'], # 最近的骨架id
        }
        rows.append(row)
    # 构建 DataFrame
    df_summary = pd.DataFrame(rows).set_index('key')
    # 保留所有的处理后的帧信息
    df_merged = df_matched.join(df_summary, how='left')

    # 保存筛选帧信息：哪些中线的置信度高于阈值
    df_merged = df_merged.join(df_choose, how='left')   # 根据index合并，df_choose里没有的填none

    # 中线处理，计算turn
    df_turn_curv = get_omega_turn_curvature(midline_path, path_ratio_uplim = 1.5, path_uplim=200, dtheta_lim=220,
                              cl_turn_size = 50, op_turn_size = 50, visualize_turn=True,
                              seg_pixel_step = 13, agl_window = 300, frame_rate=38)
    df_merged_2 = df_merged.join(df_turn_curv, how='left')

    df_merged_2.to_csv(os.path.join(p_f,basename+ '_mot_vid_mid.csv'), index=False)
    '''文件仍然与视频长度相同，包含所有中线处理信息以及筛选帧信息'''

# 步骤五：根据激光开始和结束时间截取荧光拍摄部分的运动数据/时间戳
t_frame, t_start, t_stop = get_laser_on_and_off(s_data)
track_start_index = np.searchsorted(df_matched.Time.values, t_start) - 1   # 截取激光照射段的载物台数据时间戳
track_end_index = np.searchsorted(df_matched.Time.values, t_stop)
df_matched_cut = df_matched.loc[track_start_index:track_end_index, :]
df_vol_time = pd.DataFrame(t_frame, columns=['Vol_Time'])   # 激光照射时间戳要转换单位为s
stage_file = os.path.join(p_f, f's{0+1}.txt')
vol_file = os.path.join(p_f, f't{0+1}.txt')
np.savetxt(stage_file, df_matched_cut[['Time','X_org','Y_org']], fmt='%.4f')  # 存储载物台数据(时间戳是视频帧的)
np.savetxt(vol_file, t_frame, fmt='%.4f')  # 存储荧光每一个volume的数据
df_matched_cut.to_csv(os.path.join(p_f, basename+'_mot_vid_cut.csv'), index = False)
if midline_extract:
    df_mot_midline_cut = df_merged_2.loc[track_start_index:track_end_index, :]
    df_mot_midline_cut.to_csv(os.path.join(p_f, basename+'_mot_midline_cut.csv'), index = False)

# 步骤六：根据荧光数据时间戳提取对应的运动参数，便于后续合并荧光数据
# 如果有中线数据，就用带中线数据的对齐。否则只用运动参数对齐
if midline_extract:
    df_mat_cut = df_mot_midline_cut.copy()
    print('有中线数据, 与Volume对齐的运动和中线数据')
else:
    df_mat_cut = df_matched_cut.copy()
    print('没有中线数据, 输出与Volume时间对齐的运动参数')

df_vol_time = get_nearest_value(df_mat_cut, df_vol_time, col_1 = 'Time', col_2 = 'Vol_Time', ind_col = 'Nearest_Index', col1_copy = 'Nearest_Time')
# 将df_motionz中的运动参数(有中线加上中线)按照索引对应到视频帧文件的时间戳上
columns_to_add = df_mat_cut.columns
for col in columns_to_add:
    df_vol_time[col] = df_mat_cut.iloc[df_vol_time['Nearest_Index'].values][col].values
if midline_extract:
    fn_motion_output = folder_name+'_MotionMidlineMatchVol.csv'
else:
    fn_motion_output = folder_name+'_MotionMatchVol.csv'
df_vol_time.to_csv(os.path.join(p_f, fn_motion_output), index = False)
print(f'写出与神经数据时间戳(不包含钙数据)对应的运动参数文件到:{folder_name}/{fn_motion_output}')

# 主函数：批量计算运动参数

In [ ]:
# 输入每一对txt文件所在的的文件夹

file_path_all = r'Y:\\SZX\\2025_wbi_analysis\\good_WBI\\no_binary\\new'

# 包含所有荧光数据的文件夹（没有则不加）
folder_beh_label = ''    # 行为标记文件夹名称
file_path_beh_label = os.path.join(file_path_all, folder_beh_label)

根据行为数据记录原始文件，依次进行：
+ 对齐stage, camera, fluorescent imaging时间轴
+ 根据位置坐标序列抽取运动参数，输出MotionExtract文件（待裁剪/拼接）
+ 根据volume timestamp对齐神经数据时间和位置序列时间，将行为参数列和从calcium_trace.npy的神经活动迹合并到单个文件NeuralAlignMotion中
    (如果暂时没有calcium可以选择不合并)

In [ ]:
# 放入所有包含单个实验文件的文件夹

label_corner = True
new_data = True # 2025.2之后的数据，c1中1为视频存储帧
x_shift  = 0     # mm 中线相对于梯度中线的偏离
key_word = ''
nokey_word = '*'
hlf_spd_inv = 0.5
hlf_agl_spd_inv = 0.2
hlf_agl_inv = 1
Pos_CTX_vec = [-1,0]
t_crit = 3.8
min_agl_spd = 75
cut_window = 2000
half_width = 20  #mm 琼脂片宽度的一半
Calcium = False
midline_extract = True

if label_corner:
    required_files = ['stage_data.txt', 'c1.txt',
                     'upper_left.txt', 'lower_left.txt', 'upper_right.txt',
                     'lower_right.txt']   # 数据处理需要的文件
else:
    required_files = ['stage_data.txt', 'c1.txt',
                     'edge0.txt', 'edge45.txt']   # 数据处理需要的文件
required_files_set = set(required_files)
# p_f_test = r'Z:\data space+\C. elegans chemotaxis\2025_WBI\good_WBI\test'  # 用于储存临时输出的文件夹
file_paths = [os.path.join(file_path_all,f_p) for f_p in os.listdir(file_path_all) if ('trash' not in f_p)&
              ('done' not in f_p)&
             os.path.isdir(os.path.join(file_path_all,f_p))&
             (key_word in f_p)&(nokey_word not in f_p)]

for i, p_f in enumerate(file_paths):
    df_motion = {}
    df_vol_time = {}
    calcium = True   # 默认有荧光信号的所有文件都包含钙信号
    files = [f for f in os.listdir(p_f) if '.txt' in f ]
    # 转化为集合进行比较
    files_set = set(files)
    # 判断文件夹中的文件是否完全包含自定义列表中的元素
    is_contained = required_files_set.issubset(files_set)
    # 找出missing_files
    missing_files = [file for file in required_files if file not in files]
    
    if is_contained:
        basename = os.path.basename(os.path.normpath(p_f))
        print(f'\n\n=======开始处理{basename}中文件，\n包含:{files}')
        
        files = [f for f in os.listdir(p_f)]
        for f in files:
            if 'stage_data' in f:
                # 使用载物台位置信息计算速度角速度
                print('读取载物台位置信息文件')
                column_names = ['Time','p_X','p_Y', 'X', 'Y']
                df_stage = pd.read_csv(os.path.join(p_f, f),sep=r'\s+', header=None,
                                        names=column_names)
                columns_to_clean = ['Time', 'p_X', 'p_Y', 'X', 'Y']
                for col in columns_to_clean:
                    df_stage[col] = df_stage[col].astype(str).str.replace(',', '', regex=False).astype(float)

                df_motion = df_stage[['Time','X','Y']]/1000  # 转换单位为s和mm
                # 转换坐标，统一x轴坐标起点为0，终点为4.5
                # 保证p_f中有四个顶点文件
                if label_corner:
                    df_label, df_motion = Realign_coordinate(p_f, df_motion)
                else:
                    df_motion = Realign_coordinate_by_edge(p_f, df_motion)
                # 打印轨迹
            elif 'c1.txt' in f:
                # 读取视频帧文件
                print('读取视频帧时间戳文件并转为f_data')
                f_data = np.loadtxt(p_f + '\c1.txt', delimiter=',') # 读取相机时间戳
                if new_data: 
                    # 202502之后的数据是新数据，视频帧率为30-40Hz
                    t_c = f_data[f_data[:,3] == 1, 1]/1000
                else:
                    # 旧数据视频帧率150Hz 每隔5帧取一帧（适用于老数据）
                    indices = np.concatenate(([0], np.arange(5, len(f_data), 5)))
                    t_c = f_data[indices, 1].astype(int)/1000
            elif ('laser-data' in f)or ("s-data" in f):
                # 读取激光激活数据
                print('读取激光照射时间数据并转为s_data')
                try:
                    s_data = pd.read_csv(p_f + '\laser-data.txt', header=None)  #读取激光激活数据
                except FileNotFoundError:
                    s_data = pd.read_csv(p_f + '\s-data.txt', header=None)
                # 注意：这里s_data已经转换成秒
                s_data.loc[:,0] = s_data.loc[:,0]/1000
            elif 'calcium_intensity.npy' in f:
                # 钙信号数据(* 如果有)
                print(f'读取钙信号数据：{f}')
                Cal_data_path = os.path.join(p_f, f)
                calcium_intensity= np.load(Cal_data_path)
                Calcium = True    # 包含钙信号数据
            elif 'output-0516' in f:
                # 中线npz数据(* 如果有)
                print(f'读取中线文件：{f}')
                midline_path = os.path.join(p_f,f)
                npz_data = np.load(midline_path, allow_pickle=True)
                data = npz_data['arr_0'].item()
                data_keys = list(data.keys())   # 存储的符合条件的帧数
                midline_extract = True   # 是否包含抽取中线后的文件
            elif 'iou_results' in f:
                print(f'读取中线处理相关文件{f}')
                data_iou = np.load(os.path.join(p_f,'iou_results.npz'))
        if midline_extract:
            # 步骤零点五：根据骨架处理结果筛选符合条件的帧
            # 读取骨架npz中的数据，排除异常帧
    #         path_summary = {key: value['path_summary'] for key, value in data.items()} # 骨架分析summary
            # paths = {key: value['all_paths'] for key, value in data.items()} # 骨架坐标
            # vector_h = {key: value['vector_h'] for key, value in data.items()} # 头部方向
            # vector_m = {key: value['vector_m'] for key, value in data.items()} # 运动方向
            # pos_phr = {key: value['pos_phr'] for key, value in data.items()} # 咽喉坐标
            # data_keys = list(data.keys())   # 存储的处理后符合条件的视频帧数
            # angle_m = {key: value['angle_m'] for key, value in data.items()} # 前进方向与身体方向角度
            # closest_idx = {key: value['closest path'] for key, value in data.items()}
            # circle_dict = {key: value['circular'] for key, value in data.items()}
            # branching_dict = {key: value['branching'] for key, value in data.items()}
            # 识别卷曲动作区域
            circle = np.array(list({key: value['circular'] for key, value in data.items()}.values())) 
            branching = np.array(list({key: value['branching'] for key, value in data.items()}.values())) 
            choose = np.array(list({key: value['choose_frame'] for key, value in data.items()}.values())) 
            circular_frames = np.array(data_keys)[circle == 1]
            branching_frames = np.array(data_keys)[branching == 1]
            merged = list(heapq.merge(circular_frames, branching_frames))
            choose_frame = np.array(data_keys)[choose == 1]
            # 识别面积抖动区间
            # iou = np.load(os.path.join(folder_path,'iou_results.npz'))['iou']
            iou = data_iou[data_iou.files[0]] 
            ano_f = np.vstack([[1, 1], iou]) < 0.7
            pre_f= np.where(ano_f[:,0] == True)[0]
            post_f= np.where(ano_f[:,1] == True)[0]
            ano_merged = list(heapq.merge(pre_f, post_f))
            # 排除IOU异常帧
            choose_frame = np.setdiff1d(choose_frame, ano_merged)
                
        # 步骤一：根据载物台运动位置信息抽取运动参数
        mean_frame_rate = np.median(1/np.diff(df_motion.Time.values))
        print(np.argmin(np.diff(df_motion.Time.values)))
        df_motion_als = Sliding_CTX_calation(df_motion, Pos_CTX_vec, hlf_spd_inv, hlf_agl_spd_inv, hlf_agl_inv,
                                          window_size=100, frame_rate=mean_frame_rate,X_shift=x_shift)
        # df_motion_pt = Label_Pirouettes_by_id(df_motion_als, t_crit=t_crit, min_agl_spd=min_agl_spd)
#         visualization check
        # Scatter_Pirouette(df_motion_pt, folder=p_f+'\\')
        # Scatter_Pos_Grad_Dir(df_motion_pt, Pos_CTX_vec, folder=p_f+'\\')
#         df_motion_pt.to_csv(os.path.join(p_f_test,'df_motion_pt.csv'), index=False)
        
        
        # 步骤二： 提取视频帧（根据中线处理结果只保留可用帧）对应的运动参数，输出对齐视频可用帧时间后的运动参数文件
        # 确保 df_motion_pt 也是按时间升序
        df_motion_pt_sorted = df_motion_als.sort_values('Time')
#         df_t['Time'] = pd.to_datetime(df_t['Time'])
#         df_motion_pt_sorted['Time'] = pd.to_datetime(df_motion_pt_sorted['Time'])
        df_t = pd.DataFrame({'Time': t_c})
        df_t = df_t.sort_values('Time')  # merge_asof 要求升序
#         df_motion_pt_sorted.to_csv(os.path.join(p_f_test,'df_motion_pt_sorted.csv'), index=False)
        # 无论有没有中线
        df_motion_pt_sorted = df_motion_pt_sorted.rename(columns={'Time': 'Stage_time'})

        # 使用 merge_asof 匹配最近的时间点
        df_matched = pd.merge_asof(
            df_t,
            df_motion_pt_sorted,
            left_on='Time',
            right_on='Stage_time',
            direction='nearest'  # 可选：'backward', 'forward', or 'nearest'
        )
        
        # 加入condition
        folder_name = os.path.basename(os.path.normpath(p_f))
        features = folder_name.split('_')
        if len(features) <3:
            print('文件名异常，未检测到Condition信息')
        else:
            # 如果Condition信息存在，在df_vol_time中加入condition信息
            df_matched['Date'] = features[0]
            df_matched['Rep_id'] = features[-1]
            condition_ls = features[-2].split('-')
            print(f'日期:{features[0]}, 编号：{features[-1]}\n Conditions:{condition_ls}')
            for i, c in enumerate(condition_ls):
                if len(condition_ls) > 1:
                    con_col = 'Condition'+str(i)
                    df_matched[con_col] = c
                else:
                    df_matched['Condition'] = features[-2]
            if len(condition_ls) > 1:
                df_matched['Date_Con_Rep'] = features[-2]
            df_matched['Disp_to_mid'] = df_matched['X']-half_width
        
        
        # 保存合并运动参数和视频帧对应时间戳的df数据
        df_matched.to_csv(os.path.join(p_f,basename+ '_mot_vid.csv'), index=False)
        '''文件长度与视频帧数相同'''
        
        if midline_extract:
            # 读取npz中的数据
            """
                t_stage:载物台移动数据，原始帧率200，降采样到40fps，提供位置信息
                t_c:追踪视频帧时间戳，帧率30，提供线虫中线等信息  
                data_keys：基于追踪视频进行处理后的视频帧。
            """
            rows = []
            for key in data_keys:
                item = data[key]
                row = {
                    'key': key,   # 对应视频帧
                    'angle_m': item['angle_m'],  # 头部朝向与运动方向夹角绝对值
                    'angle_md': item['angle_md'],  # 头部朝向与运动方向夹角
                    'all_paths':item['all_paths'],  # 完整的骨架信息
                    'closest_idx':item['closest path'], # 最近的骨架id
                    # 'circular':item["circular"],  # 成环
                    # 'branching':item['branching'],  # 分叉
                    # 'vector_h':item['vector_h'],   # 头部朝向
                    # 'vector_m':item['vector_m'],  # 运动方向
                    # 'pos_phr':item['pos_phr'],    # 咽部位置
                }
                rows.append(row)
                
            # 构建 DataFrame
            df_summary = pd.DataFrame(rows).set_index('key')
            # 保留所有的处理后的帧信息
            df_merged = df_matched.join(df_summary, how='left')
            # 保存筛选帧信息：哪些中线的置信度高于阈值
            # choose_frame是一个列表，包含了被选择的中线的索引
            df_choose = pd.DataFrame(choose_frame, columns=['key'])
            df_choose.loc[:,'choose_frame'] = int(1)
            df_choose = df_choose.set_index('key')
            df_merged_2 = df_merged.join(df_choose, how='left')   # 根据index合并，df_choose里没有的填none
            df_merged_2.to_csv(os.path.join(p_f,basename+ '_mot_vid_mid.csv'), index=False)
            '''文件仍然与视频长度相同，包含所有中线处理信息以及筛选帧信息'''

        # 步骤三：根据激光开始和结束时间截取荧光拍摄部分的运动数据/时间戳     
        # 提取每个激光照射起始和终止时刻
        s_start = s_data[s_data[1] == 2].index
        s_stop = s_data[s_data[1] == 3].index
        if s_stop.empty:
            s_s = [len(s_data[1])]
        else:
            s_s = []
            for m in s_start:
                # 如果有多个照射片段，分别找到结束的时间装入s_s
                data_1 = (s_stop >= m)
                s_s.append(s_stop[data_1 == 1][0])
        
        i=0  # 不考虑其它段，只考虑第一段
        #         t_l = int(round((s_data[0][s_start[i] + 1 : s_s[i] - 1]).diff().sum(skipna=True) / (s_s[i] - s_start[i] - 2)))
        t_l = (s_data[0][s_start[i] + 1 : s_s[i] - 1]).diff().sum(skipna=True) / (s_s[i] - s_start[i] - 2)
        print('时间间隔t_l',t_l)
        t = s_data[0].values
        # 取从激光照射开始前2帧到激光照射结束的数据
        t_frame = np.concatenate([[t[s_start[i]] - t_l *2, t[s_start[i]] - t_l], t[s_start[i]:s_s[i]]])
        t_start = s_data[0][s_start[i]] - t_l *2
        t_stop = s_data[0][s_s[i] - 1] + t_l

        # 截取激光照射段的载物台数据时间戳
        track_start_index = np.searchsorted(df_matched.Time.values, t_start) - 1
        track_end_index = np.searchsorted(df_matched.Time.values, t_stop)

        df_matched_cut = df_matched.loc[track_start_index:track_end_index, :]
        df_vol_time = pd.DataFrame(t_frame, columns=['Vol_Time'])   # 激光照射时间戳要转换单位为s

        stage_file = os.path.join(p_f, f's{i+1}.txt')
        vol_file = os.path.join(p_f, f't{i+1}.txt')

        # 保存文件
        np.savetxt(stage_file, df_matched_cut[['Time','X_org','Y_org']], fmt='%.4f')  # 存储载物台数据(时间戳是视频帧的)
        np.savetxt(vol_file, t_frame, fmt='%.4f')  # 存储荧光每一个volume的数据
        df_matched_cut.to_csv(os.path.join(p_f, basename+'_mot_vid_cut.csv'), index = False)
        if midline_extract:
            df_mot_midline_cut = df_merged_2.loc[track_start_index:track_end_index, :]
            df_mot_midline_cut.to_csv(os.path.join(p_f, basename+'_mot_midline_cut.csv'), index = False)
        
        # 步骤四：根据荧光数据时间戳提取对应的运动参数，便于后续合并荧光数据
        # 如果有中线数据，就用带中线数据的对齐。否则只用运动参数对齐
        if midline_extract:
            df_mat_cut = df_mot_midline_cut.copy()
            print('有中线数据，输出与Volume对齐的运动和中线数据')
        else:
            df_mat_cut = df_matched_cut.copy()
            print('没有中线数据，只输出与Volume时间对齐的运动参数')
        
        nearest_times = []
        nearest_indices = []
        for vol_time in df_vol_time['Vol_Time']:
            # 计算时间差的绝对值
            differences = np.abs(df_mat_cut['Time'] - vol_time)
            # 找到最小差值的索引
            nearest_idx = np.argmin(differences)
            # 保存最近时间值和索引
            nearest_times.append(df_mat_cut.iloc[nearest_idx, df_mat_cut.columns.get_loc('Time')])
            nearest_indices.append(nearest_idx)

        # 将结果存入 DataFrame
        df_vol_time['Nearest_Time'] = nearest_times
        df_vol_time['Nearest_Index'] = nearest_indices

        # 将df_motionz中的运动参数(有中线加上中线)按照索引对应到视频帧文件的时间戳上
        if midline_extract:
#             columns_to_add = ['X', 'Y', 'speed','x_velocity','y_velocity', 'agl_speed', 'CTX_left', 'Event', 'Reorientation', 'angle_m','all_paths','choose_frame'],  # 头部朝向与运动方向夹角
            columns_to_add = df_mat_cut.columns
        else:
            columns_to_add = df_mat_cut.columns
#             columns_to_add = ['X', 'Y', 'speed','x_velocity','y_velocity', 'agl_speed', 'CTX_left', 'Event', 'Reorientation']
#         for col in columns_to_add:
#             df_vol_time[col] = df_vol_time['Nearest_Index'].map(df_mat_cut[col])
        for col in columns_to_add:
            df_vol_time[col] = df_mat_cut.iloc[df_vol_time['Nearest_Index'].values][col].values
        
        # 输出与神经数据时间戳对齐的运动数据
        if midline_extract:
            fn_motion_output = folder_name+'_MotionMidlineMatchVol.csv'
        else:
            fn_motion_output = folder_name+'_MotionMatchVol.csv'
        df_vol_time.to_csv(os.path.join(p_f, fn_motion_output), index = False)
        print(f'写出与神经数据时间戳(不包含钙数据)对应的运动参数文件到:{folder_name}/{fn_motion_output}')
        
        if Calcium:
            print('处理前文件大小:neuron*timestamp',calcium_intensity.shape)
            # 将神经数据与行为数据合并
            calcium_intensity_T = calcium_intensity.T
            # 创建列名
            col_neuron_names = [f"neuron{i+1}" for i in range(calcium_intensity_T.shape[1])]
            df_calcium = pd.DataFrame(calcium_intensity_T, columns=col_neuron_names)
            # 按照行索引合并
            df_neu_motion = pd.concat([df_calcium, df_vol_time], axis=1)
            
            if midline_extract:
                fn_nt_output = folder_name+'_MotionMidlineMatchNeural.csv'
                df_neu_motion.to_csv(os.path.join(p_f, fn_nt_output), index = False)
                print(f'写出与神经匹配的运动信息(包含中线和钙数据)到:{folder_name}/{fn_nt_output}')
            else:
                fn_nt_output = folder_name+'_MotionMatchNeural.csv'
                df_neu_motion.to_csv(os.path.join(p_f, fn_nt_output), index = False)
                print(f'写出与神经匹配的运动信息(只钙数据无中线)到:{folder_name}/{fn_nt_output}')

    else:
        print(f'{os.path.basename(os.path.normpath(p_f))}中缺少数据处理必须的文件如下：\n' + "\n".join(missing_files))
        continue

### 数据裁剪
+ 对于追丢的数据，需要裁剪追踪丢失时间点前的数据作为最终数据

In [ ]:
# 包含同一天数据的大文件夹
file_path_beh = r'Z:\data space+\C. elegans chemotaxis\20250410_WBI\track'
# # 包含所有荧光数据的文件夹
# file_path_fluo = r'D:\data analysis\fluorescent'
# 裁剪文件夹名称和裁剪时段(s)按顺序摆放
file_to_slice = ['20250410_1g-ov-24d_001'
                ]
idx_end = [4250]

write = True

print('检查文件数量和idx_ls数量是否相同：', len(idx_end)==len(file_to_slice))
for i in range(len(idx_end)):
    print(f"{file_to_slice[i]}:{idx_end[i]}\n")
for i,f in enumerate(file_to_slice):
    print(f'=======开始处理文件{f}====================')
    f_p = os.path.join(file_path_beh, f)
    
    f_mot = f+'_MotionExtract.csv'
    df_mot = pd.read_csv(os.path.join(f_p, f_mot))
    file_contain_cal = [file for file in os.listdir(os.path.join(file_path_beh, f)) if '_NeuralAlignMotionCal' in file]
    if len(file_contain_cal):
        f_neu = f+'_NeuralAlignMotionCal.csv'
    else:
        f_neu = f+'_NeuralAlignMotion.csv'
    df_neu = pd.read_csv(os.path.join(f_p, f_neu))
    print(f'原本NeuralAlignMotion文件的时间跨度为:{df_neu.Vol_Time.min()}~{df_neu.Vol_Time.max()}')
    df_neu_slice = df_neu.loc[:idx_end[i], :]
    print(f'裁剪后NeuralAlignMotion文件的时间跨度：{df_neu_slice.Vol_Time.min()}~{df_neu_slice.Vol_Time.max()}')
    nearest_time = df_neu.loc[idx_end[i], 'Nearest_Time']
    print(f'原本MotionExtract文件的时间跨度为:{df_mot.Time.min()}~{df_mot.Time.max()}')
    df_mot_slice = df_mot[df_mot.Time <= nearest_time]
    print(f'裁剪后MotionExtract文件的时间跨度：{df_mot_slice.Time.min()}~{df_mot_slice.Time.max()}')
    
#     # 用于寻找神经数据
#     f_decom_name = f.split("_")
#     f_date = f_decom_name[0]
#     f_idx  = f_decom_name[-1]
    
#     cal_path = os.path.join(file_path_fluo, f_date, f_idx)
#     cal_int= np.load(os.path.join(cal_path, 'calcium_intensity.npy'))
#     print('处理前文件大小:neuron*timestamp',cal_int.shape)
#     cal_int_slice = cal_int[:,:idx_end[i]]
#     print('处理后:neuron*timestamp',cal_int_slice.shape)
    
    if write:
          f_mot_new = f+'_MotionExtract_slice.csv'
          df_mot_slice.to_csv(os.path.join(f_p, f_mot_new), index = False)
          f_neu_new = f+'_NeuralAlignMotion_slice.csv'
          df_neu_slice.to_csv(os.path.join(f_p, f_neu_new), index = False)
print('裁剪处理完成>.<\n======================================')
#           cal_new =" calcium_intensity_slice.npy"
#           np.save(os.path.join(cal_path,cal_new), cal_int_slice)

### 数据拼接
+ 对于行为状态良好，但是意外追丢的线虫可能会重开继续追，将多段数据根据时间关系拼接在一起
+ 先抽运动参数再拼接
+ 思路：根据两个运动数据文件的创建时间差加上后一个文件的原时间戳

In [ ]:
# 大文件夹装有待拼接的文件
file_path_all = r'Z:\data space+\C. elegans chemotaxis\20250409_WBI'
# 根据时序关系放置
file_to_concat = ['20250409_1g-ov-24d_012', '20250409_1g-ov-24d_013']

write = True

ctimes = []
for i, f_n in enumerate(file_to_concat):
    # 获取每个文件夹中state_data的创建时间
    f_p = os.path.join(file_path_all,f_n)
    stage_data_i = os.path.join(f_p, 'stage_data.txt')
    ctimes.append(os.stat(stage_data_i).st_ctime)    # 创建时间（s）
    
# 计算每个数据与第一个数据stage_data创建时间的时间差
# 计算时间差，保留两位小数（0.01秒）
time_diff_ls  = []
for i, f_n in enumerate(file_to_concat):
    print(f_n)
    time_dff_i = round(abs(ctimes[i] - ctimes[0]), 3)
    time_diff_ls.append(time_dff_i)
    f_path = os.path.join(file_path_all, f_n)
    f_slice_ls = [f for f in os.listdir(f_path) if ('_slice.csv' in f)]
    if len(f_slice_ls):
        # 如果有切片文件，使用切片文件
        f_mot = [f for f in os.listdir(f_path) if ('MotionExtract_slice.csv' in f)][0]
        f_neu = [f for f in os.listdir(f_path) if ('NeuralAlignMotion_slice.csv' in f)][0]
    else:
        f_mot = [f for f in os.listdir(f_path) if ('MotionExtract.csv' in f)][0]
        f_neu = [f for f in os.listdir(f_path) if ('NeuralAlignMotion.csv' in f)][0]

    df_mot = pd.read_csv(os.path.join(f_path, f_mot))
    df_neu = pd.read_csv(os.path.join(f_path, f_neu))

    df_mot['Time'] = df_mot['Time'] + time_dff_i
    df_neu['Vol_Time'] = df_neu['Vol_Time']+ time_dff_i
    df_neu['Nearest_Time'] = df_neu['Nearest_Time']+ time_dff_i

    if write:
        print('写出文件')
        # 写出运动参数文件
        mot_name_new = f_mot.replace("MotionExtract", 'MotionExtract_mod')
        df_mot.to_csv(os.path.join(f_path, mot_name_new), index = False)
        # 写出对齐神经数据时间戳的运动参数文件
        neu_name_new = f_neu.replace("NeuralAlignMotion", 'NeuralAlignMotion_mod')
        df_neu.to_csv(os.path.join(f_path, neu_name_new), index = False)

print(f"验证文件创建时间差:{time_diff_ls}")
print(f'=====拼接多个文件======')

folder_output = os.path.join(file_path_all, file_to_concat[0])    # 选择拼接文件输出的文件夹，一般设置为第一个片段

# 运动数据
key_word = 'MotionExtract_mod.csv'
df_to_concat = []
rep_ids = []
for i, f_p in enumerate(file_to_concat):
    f_path = os.path.join(file_path_all,f_p)
    # 收集所有需要concat的数据路径
    f_path_to_concat_i  = [os.path.join(f_path, f) for f in os.listdir(f_path) if key_word in f][0]
    df_i = pd.read_csv(f_path_to_concat_i)
#     if i == 0:
    # 取第一个文件的编号作为rep_id
    rep_id_i = df_i.Rep_id.unique()[0]
    rep_ids.append(rep_id_i)
    df_to_concat.append(df_i)
df_concat = pd.concat(df_to_concat, ignore_index=True)
df_concat['Rep_id'] = rep_ids[0]
print(f'拼接后使用同一个Rep_id:{rep_ids[0]}')
if write:
    idx_exp_first = '-'.join(map(str, rep_ids[1:]))
    concat_name = file_to_concat[0]+'-'+idx_exp_first+'_MotionExtract_concat.csv'
    df_concat.to_csv(os.path.join(folder_output,concat_name), index = False)
    
# 神经配准数据
key_word = 'NeuralAlignMotion_mod.csv'
df_to_concat = []
rep_ids = []
for i, f_p in enumerate(file_to_concat):
    f_path = os.path.join(file_path_all,f_p)
    # 收集所有需要concat的数据路径
    f_path_to_concat_i  = [os.path.join(f_path, f) for f in os.listdir(f_path) if key_word in f][0]
    df_i = pd.read_csv(f_path_to_concat_i)
    df_i['Old_index'] = df_i.index
    rep_id_i = str(df_i.Rep_id.unique()[0])
    rep_ids.append(rep_id_i)
    df_to_concat.append(df_i)
    
# 拼接时保留index
df_concat = pd.concat(df_to_concat, ignore_index = False)

# 取第一个文件的编号作为rep_id
df_concat['Rep_id'] = rep_ids[0]
print(f'拼接后使用同一个Rep_id:{rep_ids[0]}')
if write:
    # 修改部分
    idx_exp_first = '-'.join(map(str, rep_ids[1:]))  # 将 rep_ids 中的元素转换为字符串类型=
    concat_name = file_to_concat[0]+'-'+idx_exp_first+'_NeuralAlignMotion_concat.csv'
    df_concat.to_csv(os.path.join(folder_output,concat_name), index = False)

### 特殊数据处理
+ 对于缺少标定点的文件，可跳过对齐坐标轴的步骤只处理文件本身


In [ ]:

# 特殊文件文件夹地址
p_f = r'Z:\data space+\C. elegans chemotaxis\20250312_WBI\20250312_4.5g-24d-4.5h_006'


position_file_names  = ['s1']
vol_time_file_names = ['t1']

required_files = ['stage_data.txt', 'c1.txt','s-data.txt',
                 ]   # 数据处理需要的文件
required_files_set = set(required_files)
write = True    # 是否写出文件
hlf_spd_inv = 0.5
hlf_agl_spd_inv = 0.2
hlf_agl_inv = 1
Pos_CTX_vec = [-1,0]
t_crit = 3.8
min_agl_spd = 75
cut_window = 2000
half_width = 20  #mm 琼脂片宽度的一半


calcium = True   # 默认有荧光信号的所有文件都包含钙信号
files = [f for f in os.listdir(p_f) if '.txt' in f ]
# 转化为集合进行比较
files_set = set(files)
# 判断文件夹中的文件是否完全包含自定义列表中的元素
is_contained = required_files_set.issubset(files_set)
# 找出missing_files
missing_files = [file for file in required_files if file not in files]

if is_contained:
    print(f'\n\n=======开始处理{os.path.basename(os.path.normpath(p_f))}中文件，\n包含:{files}')
    # 预处理：时间戳对齐
    Align_timestamp(p_f)
    for f in files:
        if (len(position_file_names)==1):
            if position_file_names[0] in f:
                # 运动参数位置文件
                print('读取运动参数文件')
                column_names = ['Time', 'X', 'Y']
                print(os.path.join(p_f, f))
                df_motion = pd.read_csv(os.path.join(p_f, f),sep=r'\s+', header=None,
                                        names=column_names)
                df_motion = df_motion.astype(float)

                # 转换坐标，统一x轴坐标起点为0，终点为4.5
                df_label, df_motion = Realign_coordinate_false(p_f, df_motion)
                # 打印轨迹
            elif vol_time_file_names[0] in f:
                # volume时间戳文件
                print('读取volume时间戳文件')
                column_names = ['Vol_Time']
                print(os.path.join(p_f, f))
                df_vol_time = pd.read_csv(os.path.join(p_f, f),sep=r'\s+', header=None, names=column_names)
    # 查找在motion.Time中最近值和索引，加入df_vol_time中
    nearest_times = []
    nearest_indices = []
    for vol_time in df_vol_time['Vol_Time']:
        # 计算时间差的绝对值
        differences = np.abs(df_motion['Time'] - vol_time)
        # 找到最小差值的索引
        nearest_idx = np.argmin(differences)
        # 保存最近时间值和索引
        nearest_times.append(df_motion.loc[nearest_idx, 'Time'])
        nearest_indices.append(nearest_idx)

    # 将结果存入 DataFrame
    df_vol_time['Nearest_Time'] = nearest_times
    df_vol_time['Nearest_Index'] = nearest_indices


    # 根据运动位置信息抽取运动参数

    mean_frame_rate = np.mean(1/np.diff(df_motion.Time.values))
    df_motion_als = Sliding_CTX_calation(df_motion, Pos_CTX_vec, hlf_spd_inv, hlf_agl_spd_inv, hlf_agl_inv,
                                      window_size=100, frame_rate=mean_frame_rate)
    df_motion_pt = Label_Pirouettes_by_id(df_motion_als, t_crit=t_crit, min_agl_spd=min_agl_spd)
    # visualization check
    Scatter_Pirouette(df_motion_pt, folder=p_f+'\\')
    Scatter_Pos_Grad_Dir(df_motion_pt, Pos_CTX_vec, folder=p_f+'\\')
#     cut_and_draw(df_motion_pt, n = cut_window)

    # 将df_motionz中的运动参数按照索引对应到神经数据时间戳上
    columns_to_add = ['X', 'Y', 'speed', 'agl_speed', 'CTX_left', 'Event', 'Reorientation']
    for col in columns_to_add:
        df_vol_time[col] = df_vol_time['Nearest_Index'].map(df_motion_pt[col])

    # 加入condition
    folder_name = os.path.basename(os.path.normpath(p_f))
    features = folder_name.split('_')
    df_motion_pt['Date'] = features[0]
    df_vol_time['Date'] = features[0]
    df_motion_pt['Rep_id'] = features[-1]
    df_vol_time['Rep_id'] = features[-1]
    condition_ls = features[-2].split('-')
    print(f'日期:{features[0]}, 编号：{features[-1]}\n Conditions:{condition_ls}')
    for i, c in enumerate(condition_ls):
        if len(condition_ls) > 1:
            con_col = 'Condition'+str(i)
            df_motion_pt[con_col] = c
            df_vol_time[con_col] = c
        else:
            df_motion_pt['Condition'] = features[-2]
            df_vol_time['Condition'] = features[-2]
    if len(condition_ls) > 1:
        df_vol_time['Date_Con_Rep'] = features[-2]

    df_motion_pt['Disp_to_mid'] = df_motion_pt['X']-half_width
    df_vol_time['Disp_to_mid'] = df_vol_time['X']-half_width
    if file_path_fluo:
        # 如果有神经数据的地址，就加上
        # 加入神经数据
        # 用于寻找神经数据
        f = os.path.basename(p_f)
        f_decom_name = f.split("_")
        f_date = f_decom_name[0]
        f_idx  = f_decom_name[-1]

        cal_path = os.path.join(file_path_fluo, f_idx)
        files = list(Path(cal_path).rglob("calcium_intensity.npy"))
        if not files:
            calcium = False
            print("未找到任何 calcium_trace.npy 文件。")
        elif len(files)>1:

            print("找到以下文件：")
            for i, file in enumerate(files):
                print(f"{i}: {file}")

            index = int(input(f"请输入要读取的文件索引 (0~{len(files) - 1}): "))

            if 0 <= index < len(files):
                selected_file = files[index]
                calcium_intensity = np.load(selected_file)
                print(f"成功加载: {selected_file}")
            else:
                print("输入索引无效！")
        else:
            calcium_intensity= np.load(files[0])
        if calcium:
            print('处理前文件大小:neuron*timestamp',calcium_intensity.shape)
            # 将神经数据与行为数据合并
            calcium_intensity_T = calcium_intensity.T
            # 创建列名
            col_neuron_names = [f"{i+1}" for i in range(calcium_intensity_T.shape[1])]
            df_calcium = pd.DataFrame(calcium_intensity_T, columns=col_neuron_names)
            # 按照行索引合并
            df_cal_motion = pd.concat([df_calcium, df_vol_time], axis=1)

            # 写出文件
            if write:
                # folder_name为当前数据日期批次
                fn_motion_output = folder_name+'_MotionExtract.csv'
                df_motion_pt.to_csv(os.path.join(p_f, fn_motion_output), index = False)
                print(f'写出抽取的运动参数文件到:{folder_name}/{fn_motion_output}')
                fn_nt_output = folder_name+'_NeuralAlignMotionCal.csv'
                df_cal_motion.to_csv(os.path.join(p_f, fn_nt_output), index = False)
                print(f'写出与神经数据时间戳(包含钙数据)对应的运动参数文件到:{folder_name}/{fn_nt_output}')
        else:
            # 没有找到钙信号
            if write:
                # folder_name为当前数据日期批次
                fn_motion_output = folder_name+'_MotionExtract.csv'
                df_motion_pt.to_csv(os.path.join(p_f, fn_motion_output), index = False)
                print(f'写出抽取的运动参数文件到:{folder_name}/{fn_motion_output}')
                fn_nt_output = folder_name+'_NeuralAlignMotion.csv'
                df_vol_time.to_csv(os.path.join(p_f, fn_nt_output), index = False)
                print(f'写出与神经数据时间戳(不包含钙数据)对应的运动参数文件到:{folder_name}/{fn_nt_output}')
    else:
        # 写出文件
        if write:
            # folder_name为当前数据日期批次
            fn_motion_output = folder_name+'_MotionExtract.csv'
            df_motion_pt.to_csv(os.path.join(p_f, fn_motion_output), index = False)
            print(f'写出抽取的运动参数文件到:{folder_name}/{fn_motion_output}')
            fn_nt_output = folder_name+'_NeuralAlignMotion.csv'
            df_vol_time.to_csv(os.path.join(p_f, fn_nt_output), index = False)
            print(f'写出与神经数据时间戳(不包含钙数据)对应的运动参数文件到:{folder_name}/{fn_nt_output}')
else:
    print(f'{os.path.basename(os.path.normpath(p_f))}中缺少数据处理必须的文件如下：\n' + "\n".join(missing_files))

可视化手动标注行为

In [ ]:
# 提取所有行为文件csv，进行时间戳对齐
f_b_l_ls = [f for f in os.listdir(file_path_beh_label) if '_beh' in f]
for f in f_b_l_ls:
    df_b = pd.read_csv(os.path.join(file_path_beh_label, f))
    f_name = os.path.basename(f)
    f_num = f_name.split('_')[0]
    print(f'处理文件编号：{f_num}')
    
    # 在主文件夹中找到对应编号子文件夹，提取tracking文件创建时间
    p_track_f = [os.path.join(file_path_all, f) for f in os.listdir(file_path_all) if
                 (f_num in f)&(os.path.isdir(os.path.join(file_path_all,f)))]
    c_time = os.path.getctime(p_track_f[0])
    file_creation_datetime = pd.to_datetime(datetime.fromtimestamp(c_time))
    df_b['Time'] = pd.to_datetime(df_b['Time'])
    df_b['Time_diff'] = df_b['Time'] - file_creation_datetime
    df_b['Time_diff_sec'] = df_b['Time_diff'].dt.total_seconds()
    print(f'文件{f_name}的首个记录时间为{df_b.loc[0,"Time"]},\n减去stage_data创建时间{file_creation_datetime}为{df_b.loc[0,"Time_diff_sec"]}')
    # 写入文件编号为新列
    df_b['Rep_id'] = f_num
    df_b['Rep_id'] = df_b['Rep_id'].astype(str)
    new_name = f_num+'_align.csv'
    df_b.to_csv(os.path.join(file_path_beh_label, new_name))

In [ ]:
# 合并所有对齐后的行为文件csv
df_ls = []
f_b_align = [f for f in os.listdir(file_path_beh_label) if '_align' in f]
for f in f_b_align:
    df_i = pd.read_csv(os.path.join(file_path_beh_label, f),dtype={'Rep_id': str})
    df_ls.append(df_i)
df = pd.concat(df_ls, ignore_index=True)
num_id = df.Rep_id.unique()
# 定义行为模式映射
behavior_map = {'None':0, 'Forward': 1, 'Reverse': 2, 'Turn': 3, 'Coil': 4, 'Pause':5}
# colors = ['blue', 'red', 'green', 'orange', 'grey']  # 颜色对应行为
# colors = ['#D1D6B6', '#F2E6CE', '#70635F','#C77D7D', '#D4A07B']
# colors = ['#008BA1', '#BF0079', '#75245C','#005CA3', '#4D157D']
# colors = ['#F2E77E', '#5D72A8', '#65CF61','#336131', '#1D3980']
colors = ['#889A9C','#0742D9', '#fdca4e', '#EB1717','#73DE00', '#F5EBD7']
cmap = ListedColormap(colors)
behavior_col = ['Forward', 'Reverse', 'Turn', 'Coil', 'Pause']
color_dict = {i:colors[i] for i in range(len(colors))}

# 生成大图
width = 10
height= 0.5
fig, ax = plt.subplots(len(num_id), 1, figsize=(width, height*len(num_id)))
fig.suptitle("Behavior over Time", fontsize = width*2, y = 1.25)
for i, (idx, df_i) in enumerate(df.groupby('Rep_id')):
    
    # 找到每个时间点的行为模式（只含1的列）
    df_i['Behavior'] = df_i[behavior_col].apply(lambda row: row.idxmax() if row.sum() >0 else 'None', axis=1)  # 获取行为模式列名
    df_i['Behavior_idx'] = df_i['Behavior'].map(behavior_map)  # 转换为数值索引
    df_i['Beh_color'] = df_i['Behavior_idx'].map(color_dict)   # 映射颜色方便之后作轨迹图
    
    df_i.to_csv(os.path.join(file_path_beh_label, str(idx)+'beh_label.csv'))
    
    # 画图
    vectors = df_i['Behavior_idx'].values[np.newaxis, :]
    # 生成颜色列表
    color_vec = [color_dict.get(key, None) for key in df_i['Behavior_idx'].values]
    bound = np.arange(-0.5, 6.5, 1)
    norm = mcolors.BoundaryNorm(bounds, cmap.N)
    sc = ax[i].imshow(vectors, 
                   aspect='auto', cmap=cmap, norm = norm, extent=[df_i['Time_diff_sec'].min(), df_i['Time_diff_sec'].max(), 0, 1])

    # 设置横轴
    ax[i].set_xlim([0, 30*60])
    ax[i].tick_params(axis = 'x', labelsize = width*1.5)
    ax[i].set_yticks([])  # 隐藏 y 轴
    if i < len(num_id)-1:
        ax[i].set_xticks([])
    # xlabel标记编号
    ax[i].set_ylabel(idx, fontsize = width*1.75, labelpad = width*2, rotation=0)
#     ax[i].grid(True, axis = 'x', color='grey', alpha = 0.35, ls = 'dashed')
    if i==0:
        # 添加图例
        # Create legend manually for color mapping
        legend_labels = behavior_map.keys()
        legend_handles = [mpatches.Patch(color=color, label=label) for color, label in zip(colors, legend_labels)]

        # Add the legend with desired properties
        ax[i].legend(handles=legend_handles, loc='upper center', bbox_to_anchor=(0.5, 2.3), 
                  ncol=len(legend_labels), frameon=False, markerscale=0.5, fontsize=width*1.25)

        # Remove the axes' outer border
    ax[i].spines['top'].set_visible(False)
    ax[i].spines['right'].set_visible(False)
    ax[i].spines['bottom'].set_visible(False)
    ax[i].spines['left'].set_visible(False)
    if i == len(num_id)-1:
        ax[i].set_xlabel("Time (s)", fontsize = width*2)
        ax[i].spines['bottom'].set_visible(True)
plt.subplots_adjust(hspace = 0.3)
plt.show()

根据注释行为作轨迹图


In [ ]:
# # behavior_map = {'None':0, 'Forward': 1, 'Reverse': 2, 'Turn': 3, 'Coil': 4, 'Pause':5}
# # colors = ['#889A9C','#0742D9', '#fdca4e', '#EB1717','#73DE00', '#F5EBD7']
# # color_dict = {i:colors[i] for i in range(len(colors))}

# f_b_anot = [f for f in os.listdir(file_path_beh_label) if 'beh_label' in f]
# for f in f_b_anot:
#     df_i = pd.read_csv(os.path.join(file_path_beh_label, f),dtype={'Rep_id': str})
#     f_num = f.split('beh')[0]
#     p_track_folder = [os.path.join(file_path_all, f) for f in os.listdir(file_path_all) if
#                  (f_num in f)&(os.path.isdir(os.path.join(file_path_all,f)))][0]
#     track_f = [os.path.join(p_track_folder,f) for f in os.listdir(p_track_folder) if 'MotionExtract' in f][0]
#     df_motion = pd.read_csv(track_f)
    
#     for t in df_motion['Time']:
#         diff_time = np.abs(df_i['Time_diff_sec']-t)
#         nearest_idx = np.argmin(diff_time)
#         df_motion['Nearest_idx'] = nearest_idx
#         df_motion['Beh_color'] = df_i.loc[nearest_idx,'Beh_color']
#     fig,ax = plt.subplots()
#     ax.scatter(df_motion['X'], df_motion['Y'], c = df_motion['Beh_color'], s = 0.8)
#     ax.set_aspect(1)
#     ax.set_title(f)
#     plt.show()
        
f_b_anot = [f for f in os.listdir(file_path_beh_label) if 'beh_label' in f]

for f in f_b_anot:
    # 读取行为标签文件
    df_i = pd.read_csv(os.path.join(file_path_beh_label, f), dtype={'Rep_id': str})
    f_num = f.split('beh')[0]

    # 获取对应的轨迹文件夹
    p_track_folder = next(os.path.join(file_path_all, f) for f in os.listdir(file_path_all) 
                          if (f_num in f) and os.path.isdir(os.path.join(file_path_all, f)))
    
    # 读取运动轨迹文件
    track_f = next(os.path.join(p_track_folder, f) for f in os.listdir(p_track_folder) if 'MotionExtract' in f)
    print(track_f)
    df_motion = pd.read_csv(track_f)

    # **使用 `merge_asof()` 进行最近时间点匹配，提高效率**
    df_i = df_i.sort_values('Time_diff_sec')  # 先按时间排序
    df_motion = df_motion.sort_values('Time')  # 轨迹数据按时间排序

    # **最近匹配最近的行为**
    df_motion = pd.merge_asof(df_motion, df_i[['Time_diff_sec', 'Beh_color']], 
                              left_on='Time', right_on='Time_diff_sec', direction='nearest')

    # 绘制轨迹图
    fig, ax = plt.subplots()
    ax.scatter(df_motion['X'], df_motion['Y'], c=df_motion['Beh_color'], s=0.8)
    ax.legend(handles=legend_handles, loc='upper center', bbox_to_anchor=(1.2, 0.9), 
                  ncol=1, frameon=False, markerscale=0.5, fontsize=width*1.25)

    ax.set_aspect(1)
    ax.set_title(f)
    plt.show()

# （*）多文件可视化行为

## 函数定义

In [ ]:
# 输入所有一级文件夹列表，合并所有csv数据
def combine_csv_to_csv(folder_ls, key_1='_mot_vid.csv', key_2='', key_3='', nokey_1='*', nokey_2='*', columns=[], t_inv_draw=15, d_inv_draw=5):
    '''
    合并指定文件夹中所有的子文件夹中的指定csv文件
    输入folder_ls包含所有感兴趣的文件夹路径(一般每个路径是单天实验)
    '''
    p_f_ls = []
    for folder in folder_ls:
        # 对于所有1级文件夹地址，收起其中所有文件夹地址
        p_f_ls += [os.path.join(folder, p_f) for p_f in os.listdir(folder) if 
                   (os.path.isdir(os.path.join(folder, p_f))&('trash' not in p_f)&(key_2 in p_f)&(key_3 in p_f)
                    &(nokey_1 not in p_f)&(nokey_2 not in p_f))]
        print(p_f_ls)
    data_frames = []
    for p_f in p_f_ls:
        files =  [
            f for f in os.listdir(p_f)
            if f.endswith('.csv') and key_1 in f ]
        if len(files) > 1:
            # 说明可能是拼接文件，选择有concat的，否则跳过
            file_sel_ls = [f for f in files if 'concat' in f]
            if len(file_sel_ls)==0:
                print(os.path.basename(p_f),'中没有符合条件的文件')
                continue
            else:
                file_sel = file_sel_ls[0]
        elif len(files) == 0:
            print(os.path.basename(p_f)+'中无符合条件的文件')
            continue
        else:
            file_sel = files[0]
        print(file_sel,'\n')
        df = pd.read_csv(os.path.join(p_f, file_sel))
        data_frames.append(df)
    df_als = pd.concat(data_frames, ignore_index=True)
    if 'Date_Con_Rep' in df_als.columns:
        df_als = df_als.drop(['Date_Con_Rep'], axis=1)
    # 合并Date, condition, rep_id以便于分组单个数据
    df_als['Date_Con_Rep'] = df_als['Date'].astype(str)+'_'
    for col in df_als.columns:
        if 'Condition' in col:
            if '0' in col:
                df_als['Date_Con_Rep'] += df_als[col].astype(str)
            else:
                df_als['Date_Con_Rep']    += '-'+df_als[col].astype(str)

    df_als['Date_Con_Rep'] += ('_'+df_als['Rep_id'].astype(str))
        
    return df_als

In [ ]:
def Sort_by_GroupCol(df, group_col, group_col_ls):
    '''
    根据group_col_ls排序group_col
    '''
    df[group_col] = pd.Categorical(df[group_col], categories=group_col_ls, ordered=True)
    df.sort_values(by = [group_col])
    return df

## 数据导入

In [ ]:
folder_ls = [r'Y:\SZX\BehaviorData\20250823_WBI']

key_word_2 = '1gNa'
df_als = combine_csv_to_csv(folder_ls, key_2=key_word_2)

# 设置一个存放所有输出图片的文件夹
folder_fig_name = 'BehAnalysis'
p_f_fig = os.path.join(folder_ls[0], folder_fig_name)
if not os.path.isdir(p_f_fig):
    os.mkdir(p_f_fig)
save = 1
# 排序
# 排序
# sort_col = 'Condition0'
# df[sort_col] = pd.Categorical(df[sort_col], ordered=True)
# df_als = df_als.sort_values(by = [sort_col,'Condition1', 'Vol_Time'])

## Visualization of Trajectory distribution

### 轨迹分布大图(所有数据画在一个图上)
+ 将所有包含单次数据的文件夹放在一个总文件夹中，file_path_all
+ 依次读出所有单次数据，不合并为一个df作图

In [ ]:
df = df_als.copy()
fs = 20
group_col = 'Date_Con_Rep'
kw = key_word_2
sort = False
sort_col = 'Condition3'
# 可视化所有轨迹
figure_0,ax = plt.subplots(figsize = (10,10))
# 定义要使用的 sequential colormaps
colormaps = ['Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds', 'BuPu',
             'YlOrBr', 'PuRd', 'YlOrRd', 'OrRd', 'RdPu',
             'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn']
if sort:
    # 排序输出，美观
    df[sort_col] = pd.Categorical(df[sort_col], ordered=True)
    df = df.sort_values(by = [sort_col,'Condition1', 'Vol_Time'])

for i,(g,df_g) in enumerate(df.groupby(group_col, sort = False)):
    
    # 设置每个数据的colormap
    orig_cmap = mpl.colormaps[colormaps[i%len(colormaps)]]
    # 重新缩放 colormap，使最浅部分变深
    new_cmap = mcolors.LinearSegmentedColormap.from_list(
        "truncated_cmap", orig_cmap(np.linspace(0.3, 1, 256))  # 从 30% 处开始
    )

    ax.scatter(df_g['X'], df_g['Y'], c = df_g['Time'], cmap = new_cmap, 
               s = 1)
    first_valid_idx = df_g['X'].first_valid_index()
    
    ax.scatter(df_g.loc[first_valid_idx, 'X'], df_g.loc[first_valid_idx, 'Y'], 
              s=30, c = 'r', marker='*')
    legend_color = new_cmap(0.5)
    ax.scatter([], [], color=legend_color, label=g, s=5, marker='o')  # 仅用于图例
ax.axvline(x=32, color = 'grey', ls = 'dashed', alpha = 0.8, lw = 2)
ax.axvline(x=8, color = 'red', ls = 'dashed', alpha = 0.8, lw = 2, label='border')
# 定义图例的颜色
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), markerscale=2, fontsize=20)
ax.set_title('Trajectories', fontsize = fs*1.5, pad= fs)
ax.tick_params(axis = 'both', labelsize = fs)
ax.set_xlabel('mm', fontsize = fs)
ax.set_ylabel('mm', fontsize = fs)
ax.set_xlim([-1,41])
ax.set_aspect('equal')
if save:
    figure_0.savefig(os.path.join(p_f_fig, f'Trajectories of all data{kw}.png'), 
                    bbox_inches='tight')
else:
    plt.show()

分条件画图

In [ ]:
df_als.Rep_id

In [ ]:
df_als['Condition3'].unique()

In [ ]:
df = df_als[(df_als['Condition3']=='8h')&(df_als['Rep_id']!=15)]
fs = 20
group_col = 'Date_Con_Rep'
kw = key_word_2
sort = False
sort_col = 'Condition3'
# 可视化所有轨迹
figure_0,ax = plt.subplots(figsize = (10,10))
# 定义要使用的 sequential colormaps
colormaps = ['Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds', 'BuPu',
             'YlOrBr', 'PuRd', 'YlOrRd', 'OrRd', 'RdPu',
             'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn']
if sort:
    # 排序输出，美观
    df[sort_col] = pd.Categorical(df[sort_col], ordered=True)
    df = df.sort_values(by = [sort_col,'Condition1', 'Vol_Time'])

for i,(g,df_g) in enumerate(df.groupby(group_col, sort = False)):
    
    # 设置每个数据的colormap
    orig_cmap = mpl.colormaps[colormaps[i%len(colormaps)]]
    # 重新缩放 colormap，使最浅部分变深
    new_cmap = mcolors.LinearSegmentedColormap.from_list(
        "truncated_cmap", orig_cmap(np.linspace(0.3, 1, 256))  # 从 30% 处开始
    )

    ax.scatter(df_g['X'], df_g['Y'], c = df_g['Time'], cmap = new_cmap, 
               s = 1)
    first_valid_idx = df_g['X'].first_valid_index()
    
    ax.scatter(df_g.loc[first_valid_idx, 'X'], df_g.loc[first_valid_idx, 'Y'], 
              s=30, c = 'r', marker='*')
    legend_color = new_cmap(0.5)
    ax.scatter([], [], color=legend_color, label=g, s=5, marker='o')  # 仅用于图例
ax.axvline(x=32, color = 'grey', ls = 'dashed', alpha = 0.8, lw = 2)
ax.axvline(x=8, color = 'red', ls = 'dashed', alpha = 0.8, lw = 2, label='border')
# 定义图例的颜色
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), markerscale=2, fontsize=20)
ax.set_title('Trajectories', fontsize = fs*1.5, pad= fs)
ax.tick_params(axis = 'both', labelsize = fs)
ax.set_xlabel('mm', fontsize = fs)
ax.set_ylabel('mm', fontsize = fs)
ax.set_xlim([-1,41])
ax.set_aspect('equal')
if save:
    figure_0.savefig(os.path.join(p_f_fig, f'Trajectories of all data{kw}.png'), 
                    bbox_inches='tight')
else:
    plt.show()

#### 分子图画轨迹
+ 将所有包含单次数据的文件夹放在一个总文件夹中，file_path_all
+ 依次读出所有单次数据，不合并为一个df作图

In [ ]:
df = df_als.copy()
fs = 20
kw = key_word_2
group_col = 'Date_Con_Rep'
sort_col = 'Condition0'
interval = 300   # s
sort = False
# 定义要使用的 sequential colormaps, 获取颜色列表
colormaps = ['Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds', 'BuPu',
             'YlOrBr', 'PuRd', 'YlOrRd', 'OrRd', 'RdPu',
             'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn']

if sort:
    # 排序
    df[sort_col] = pd.Categorical(df[sort_col], ordered=True)
    df = df.sort_values(by = [sort_col,'Condition1', 'Time'])

# 计算每个子图的高度比例
height_ratios = []

for g, df_g in df.groupby(group_col, sort=False):
    y_min = df_g['Y'].min()
    y_max = df_g['Y'].max()
    y_range = y_max - y_min

    # 增加 10% 的缓冲区
    padding = 0.1 * y_range
    y_min -= padding
    y_max += padding
    y_range_with_padding = y_max - y_min

    height_ratios.append(y_range_with_padding)

# 创建大图
group_num = df[group_col].unique()
figure_0, ax = plt.subplots(
    len(group_num), 1, 
    figsize=(10, 3 * len(group_num)), 
    sharex=True,  # 共享 x 轴
    gridspec_kw={'height_ratios': height_ratios}  # 子图高度自适应
)

# 找到所有子图中 Vol_Time 的最大值，作为 colorbar 的统一上限
vmax = df.Time.max()

# 绘制每个子图
for i,(g,df_g) in enumerate(df.groupby(group_col, sort = False)):
            
    # 设置colormap 
    # 设置截断colormap
    orig_cmap = mpl.colormaps[colormaps[i%len(colormaps)]]
    # 重新缩放 colormap，使最浅部分变深
    new_cmap = mcolors.LinearSegmentedColormap.from_list(
        "truncated_cmap", orig_cmap(np.linspace(0.3, 1, 256))  # 从 30% 处开始
    )
    
    # 绘制散点图
    scatter = ax[i].scatter(df_g['X'], df_g['Y'], c=df_g['Time'], 
                            cmap=new_cmap, s=1, vmin=0, vmax=vmax)  # 统一 colorbar 上限
    ax[i].axvline(x=20, ls='dashed', c='r')  # 添加红色虚线
    
    # 标记起始位置
    first_valid_idx = df_g['X'].first_valid_index()
    
    ax[i].scatter(df_g.loc[first_valid_idx, 'X'], df_g.loc[first_valid_idx, 'Y'], c = 'r', marker = '*', s = 25)
    
    # 间隔指定时间打印时间节点文字
    df_g_dn = df_g.dropna(subset=['X', 'Y', 'Time']).reset_index(drop = True)  # 删除包含 NaN 的行
    
    # 最后打印的时间序列元素是原始时间
    time_series = np.arange(df_g_dn.Time.min(), df_g_dn.Time.max(), interval)
    # 转为字符串，便于打印
    time_list = list(np.int32(time_series))
    time_list_str = list(map(str, time_list))
    # 取完整时间序列，扩展成矩阵
    time_vec = df_g_dn.Time
    mat_time = np.tile(time_vec, (len(time_series), 1))
    # 使用广播机制，将矩阵每一行减去时间序列对应元素
    mat_time_subtracted = mat_time-time_series[:,np.newaxis]
    # 返回每一行最小值的索引
#     min_diff_idx = np.argmin(np.abs(mat_time_subtracted), axis=1)
    min_diff_idx = np.abs(mat_time_subtracted).argmin(axis=1)
#     ax[i].text(df_motion.loc[min_diff_idx, 'X'],df_motion.loc[min_diff_idx, 'Y'], time_list_str, c = 'r', fontsize = 5)
    # zip将两个列表打包
    for idx, time_str in zip(min_diff_idx, time_list_str):
        ax[i].text(df_g_dn.loc[idx, 'X'], 
                   df_g_dn.loc[idx, 'Y'], 
                   time_str+'s', c='r', fontsize=10)

    # 设置标题
    ax[i].set_title(g, fontsize=20)
    ax[i].set_xlim([-1,41])
    # 设置 y 轴范围（增加缓冲区）
    y_min = df_g['Y'].min()
    y_max = df_g['Y'].max()
    padding = 0.1 * (y_max - y_min)  # 10% 的缓冲区
    ax[i].set_ylim(y_min - padding, y_max + padding)
    ax[i].tick_params(axis = 'both', labelsize = 15)
    # 设置子图宽高比
    ax[i].set_aspect('equal')
    ax[i].set_ylabel('mm', fontsize = 15)
    if i == len(group_num)-1:
        
        ax[i].set_xlabel('mm', fontsize = 15)
    # 为每个子图添加 colorbar
    divider = make_axes_locatable(ax[i])
    cax = divider.append_axes("right", size="5%", pad=0.1)  # colorbar 宽度为 5%，间距为 0.1
    cbar = plt.colorbar(scatter, cax=cax)
    cbar.set_label('Time(s)', fontsize = 15)

# 调整子图间距
plt.subplots_adjust(hspace=0.2)
if save:
    figure_0.savefig(os.path.join(p_f_fig, f'Trajectory of each data{kw}.png'), 
                    bbox_inches='tight')
else:
    plt.show()

#### 分子图画轨迹分布
+ 根据时间段出图，每一行为单个重复组的数据，每一列表示时间窗
+ 将所有包含单次数据的文件夹放在一个总文件夹中，file_path_all
+ 依次读出所有单次数据，不合并为一个df作图

In [ ]:
# 定义函数生成指定colormap指定位置的颜色的RGB和16进制代码列表
def trans_colormap_to_RGB(colormaps, ratio):
    '''
    根据一组colormap序列得到指定位置的单一颜色，如ratio=0.5时取中位颜色
    '''
    # 用于存储中位颜色的 RGB 和 16进制代码
    RGB_ls = []
    hex_ls = []
    # 遍历每个 colormap，计算中位颜色
    for cmap_name in colormaps:
        cmap = mpl.colormaps[cmap_name]
        # 获取 colormap 中的颜色值（在 0 到 1 之间）
        colors = cmap(np.linspace(0, 1, 256))  # 256 表示 colormap 中的颜色分辨率
        # 计算中位数索引
        idx = int(len(colors)*ratio)
        rgb = colors[idx, :3]  # 获取 RGB (去除 alpha)

        # 将 RGB 转换为 16 进制代码
        hex_color = '#{:02x}{:02x}{:02x}'.format(int(rgb[0] * 255), int(rgb[1] * 255), int(rgb[2] * 255))
        
        RGB_ls.append(rgb)
        hex_ls.append(hex_color)
        
    return RGB_ls, hex_ls

In [ ]:
# 画组间比较位置分布

df = df_als.copy()
fs = 20
kw = key_word_2
group_col = 'Date_Con_Rep'
sort = False
sort_col = 'Condition1'
interval_min = 5  # min
interval_s = interval_min*60 # s
time_series = []
bin_size = 2.5 # mm
x_min, x_max = 0, 40  # 设定 X 的范围

if sort:
    # 排序
    df[sort_col] = pd.Categorical(df[sort_col], ordered=True)
    df = df.sort_values(by = [sort_col, 'Vol_Time'])

# 定义要使用的 sequential colormaps
colormaps = ['Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds', 'BuPu',
             'YlOrBr', 'PuRd', 'YlOrRd', 'OrRd', 'RdPu',
             'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn']
rgb_ls, hex_ls = trans_colormap_to_RGB(colormaps, 0.55)
num_colormap = len(colormaps)
fs = 25

# 数据分组数量
group_num = df[group_col].unique()

# 找出最大时间
max_time = df['Vol_Time'].max()
print(f'所有数据中的最大时间{max_time}')

# 根据最大时间分时间段间隔
if not len(time_series):
    time_series = np.arange(0, max_time, interval_s)
# 分割X区间

bins = list(np.arange(x_min, x_max + bin_size, bin_size))  # 创建区间边界
bins_array = np.array(bins)
labels = list((bins_array[:-1]+bins_array[1:])/2)
# 使用 pd.cut() 将 X 分割成区间
df['X_bin'] = pd.cut(df['X'], bins=bins,labels=labels, right=False)  # 右开区间 

# 创建大图
figure_0, ax = plt.subplots(
    len(group_num), len(time_series)-1, 
    figsize=(5*(len(time_series)-1), 2.5 * len(group_num)), 
    sharex=True,  # 共享 x 轴
    sharey=True  
)


# 绘制每个子图
for i,(g,df_g) in enumerate(df.groupby(group_col)):
    
    for j in range(len(time_series)-1):
        
        df_g_t = df_g[(df_g['Vol_Time']>=time_series[j])&(df_g['Vol_Time']<time_series[j+1])]
    
        # 绘制柱状图
        sum_timepoint = df_g_t['X_bin'].notna().count()
        df_gb_xbin = df_g_t.groupby('X_bin').agg({'Vol_Time':'count'}).reset_index()
        df_gb_xbin = df_gb_xbin.rename(columns={'Vol_Time':'count'})
        df_gb_xbin['ratio_count'] = df_gb_xbin['count']/sum_timepoint
        ax[i][j].bar(df_gb_xbin['X_bin'],df_gb_xbin['ratio_count'], width=2,color = rgb_ls[i%num_colormap])
#         ax[i].axvline(x=20, ls='dashed', c='r')  # 添加红色虚线
        ax[i][j].tick_params(axis = 'both', labelsize = fs)
        if i == 0:
            # 设置标题
            begin_min = int(time_series[j]/60)
            end_min = int(time_series[j+1]/60)
            ax[i][j].set_title(f'{str(begin_min)}~{str(end_min)}min', fontsize=fs*1.25)
        if j == 0:
            y_label = '\n'.join(g.split('_'))
            ax[i][j].set_ylabel(y_label, fontsize = fs, rotation=60, labelpad = fs*1.5)
        # 设置x轴刻度范围
        # 设置 X 轴刻度间隔
        ax[i][j].set_xticks(np.arange(x_min, x_max+1, 10))  # 设置刻度间隔为 5mm
        ax[i][j].grid(True, color='grey', alpha = 0.5, ls = 'dashed')
        
        if i == len(group_num)-1:
            ax[i][j].set_xlabel('mm', fontsize = fs, labelpad = fs*0.25)

# 调整子图间距
plt.subplots_adjust(hspace=0.2, wspace = 0.2)
if save:
    figure_0.savefig(os.path.join(p_f_fig, f'Pos Distribution of each worm{kw}.png'), 
                    bbox_inches='tight')
else:
    plt.show()

## CTX

In [ ]:
# 速度角速度随时间的变化


df = df_als.copy()
kw = key_word_2
group_col = 'Condition2'
# group_col = 'Date_Con_Rep'
value_col = ['speed', 'agl_speed']
group_col_ls=[]
color_dict={}
color_map = 'plasma'
# t_inv_overtime = [0,30]
t_max = 30    # min
time_inv = 5  # min
n_col = 2
sharey=True
fs=25
y_lim=[]
frame_rate = 20
lw= 2
folder=None
'''
比较value_col的均值随时间变化的组内差异（按照group_col分组，组内可能按照group_id不同比较）
group_cols:用于分组的列的列表（可排序后再输入），相当于原本按照condition分组的conds
group_col_ls用于排序
value_col：指定可视化的参数列，比如CTX_left
n_col: 自适应子图数量，n_col指定列数
sharey=True
'''

# 根据所有group_col的数量生成一张大图
if not len(group_col_ls):
    # 如果指定了组列表，直接使用
    group_col_ls = df[group_col].unique()

# 去除turn
df_run = df[df.Event==0]
# 颜色
if not color_dict:
    color_dict  = map_color_dict(group_col_ls, color_map)
    
# 设置时间区间
df_run,_ = set_time_intervals(df_run, t_max*60, time_inv*60)   
df_time = df_run.copy()
df_time['Period_label'] = df_time['Period_label'].astype(float)
# df_time_p = df_time[(df_time['Period_label']>=t_inv_overtime[0]) & (df_time['Period_label']<=t_inv_overtime[1])]

# 根据指定的group_col_ls排序
if len(group_col_ls):
    df_time = Sort_by_GroupCol(df_time, group_col,group_col_ls)
# 可视化
n_row = len(value_col)   # 每一行表示不同的可视化量
n_col = len(group_col_ls)  # 每一列表示不同的分组

fig, ax = plt.subplots(n_row, n_col, figsize=(5.5 * n_col, 5 * n_row), sharex=True, sharey=False)

# 计算每一行(y变量)的全局y轴范围
y_limits = {}
for n, v_col in enumerate(value_col):
    y_min, y_max = float('inf'), float('-inf')
    for g, df_f in df_time.groupby(group_col):
        df_mean_ctx_by_g = df_f.groupby(['Date_Con_Rep', 'Period_label']).agg({v_col: 'mean'}).reset_index()
        y_min = min(y_min, df_mean_ctx_by_g[v_col].min())
        y_max = max(y_max, df_mean_ctx_by_g[v_col].max())
    y_limits[v_col] = (y_min-(y_max-y_min)*0.1, y_max+(y_max-y_min)*0.1)

# 绘制图像
for n, v_col in enumerate(value_col):
    for i, (g, df_f) in enumerate(df_time.groupby(group_col)):

        df_mean_ctx_by_g = df_f.groupby(['Date_Con_Rep', 'Period_label']).agg({v_col: 'mean'}).reset_index()

        sns.lineplot(df_mean_ctx_by_g, x='Period_label', y=v_col, ax=ax[n][i],
                     color=color_dict[g], markers=True, style='Date_Con_Rep',
                     linewidth=lw)

        ax[n][i].set_title(g, fontsize=fs * 1.25, pad=fs * 0.5)
        ax[n][i].set_xlabel('Time(s)', fontsize=fs, labelpad=fs * 0.5)

        # 统一每一行的 y 轴范围
        ax[n][i].set_ylim(y_limits[v_col])

        if n == 1:
            ax[n][i].legend(loc='upper center', bbox_to_anchor=(0.5, -0.3),
                            fontsize=fs * 0.85, ncol=1, frameon=False)
        else:
            ax[n][i].get_legend().remove()  # 移除 legend

        ax[n][i].grid(True, linestyle=':', color='grey', alpha=0.6)
        ax[n][i].tick_params(labelsize=fs * 0.85)

        # 画 y=0
        ax[n][i].axhline(y=0, color='r', linestyle='dashed', alpha=0.5)

        # 共享 y 轴
        if i > 0:
            ax[n][i].sharey(ax[n][0])
            ax[n][i].set_ylabel('')
        else:
            ax[n][i].set_ylabel(v_col, fontsize=fs, labelpad=fs * 0.5)

if save:
    # 保存图片
    values = '_'.join(value_col)
    title_name = f"{values}_OverTimeWihtinG_{kw}.png"
    file_path = os.path.join(p_f_fig, title_name)
    fig.savefig(file_path, bbox_inches='tight')
else:
    plt.show()
    
   

In [ ]:
# CTX随时间变化

df = df_als.copy()
kw = key_word_2
group_col = 'Condition2'
# group_col = 'Date_Con_Rep'
value_col = 'CTX_left'
group_col_ls=[]
color_dict={}
color_map = 'plasma'
# t_inv_overtime = [0,30]
t_max = 30    # min
time_inv = 5  # min
n_col = 2
sharey=True
fs=25
y_lim=[]
frame_rate = 20
lw= 2
folder=None
'''
比较value_col的均值随时间变化的组内差异（按照group_col分组，组内可能按照group_id不同比较）
group_cols:用于分组的列的列表（可排序后再输入），相当于原本按照condition分组的conds
group_col_ls用于排序
value_col：指定可视化的参数列，比如CTX_left
n_col: 自适应子图数量，n_col指定列数
sharey=True
'''

# 根据所有group_col的数量生成一张大图
if not len(group_col_ls):
    # 如果指定了组列表，直接使用
    group_col_ls = df[group_col].unique()
    
# 子图数量与分组数量相同
num_fig = len(group_col_ls)

# 去除turn
df_run = df[df.Event==0]
# 颜色
if not color_dict:
    color_dict  = map_color_dict(group_col_ls, color_map)
    
# 设置时间区间
df_run,_ = set_time_intervals(df_run, t_max*60, time_inv*60)   
df_time = df_run.copy()
df_time['Period_label'] = df_time['Period_label'].astype(float)
# df_time_p = df_time[(df_time['Period_label']>=t_inv_overtime[0]) & (df_time['Period_label']<=t_inv_overtime[1])]

# 根据指定的group_col_ls排序
if len(group_col_ls):
    df_time = Sort_by_GroupCol(df_time, group_col,group_col_ls)

# 可视化
n_row = 1
n_col = num_fig
fig,axes = plt.subplots(n_row, n_col, figsize = (5.5*n_col,5*n_row), sharex = True, sharey = sharey)
ax = axes.ravel()

for i, (g, df_f) in enumerate(df_time.groupby(group_col)):
 
    df_mean_ctx_by_g = df_f.groupby(['Date_Con_Rep', 'Period_label']).agg({value_col:'mean'}).reset_index()

    sns.lineplot(df_mean_ctx_by_g,x='Period_label',
                y=value_col, ax = ax[i],
                color=color_dict[g], markers=True, style='Date_Con_Rep',
                 linewidth=lw
                )
    ax[i].set_title(g, fontsize=fs*1.25, pad = fs)
    ax[i].set_xlabel('Time(s)', fontsize=fs, labelpad = fs*0.5)
    ax[i].legend(loc='upper center', bbox_to_anchor=(0.5, -0.3),fontsize=fs*0.85, ncol=1, frameon=False)
    ax[i].grid(True,linestyle=':',color='grey',alpha=0.6)
    ax[i].tick_params(labelsize=fs*0.85)
    # 画y=0
    ax[i].axhline(y=0, color='r', linestyle='dashed', alpha = 0.5)
    ax[i].set_ylabel(value_col, fontsize=fs, labelpad = fs*0.5)
    if y_lim:
        ax[i].set_ylim(y_lim[0], y_lim[1])

if save:
#  保存图片
    title_name = f'{kw}_CTX-leftOverTimeWihtinG.png'
    file_path = os.path.join(p_f_fig, title_name)
    plt.savefig(file_path,bbox_inches='tight')
else:
    plt.show()

    
   

In [ ]:
half_width = 20  #mm
df_als['Disp_to_mid'] = df_als['X']-half_width
x_min, x_max = -20, 20
dist_inv = 2.5   # mm
dist_vec = np.arange(x_min,x_max,dist_inv)
labels = [(dist_vec[i] + dist_vec[i + 1]) / 2 for i in range(len(dist_vec) - 1)]
df_als['Disp_label'] = pd.cut(df_als['Disp_to_mid'], bins=dist_vec, labels=labels, right=False, include_lowest=True)

In [ ]:
# 距中线的平均距离随时间变化
df = df_als.copy()
kw = key_word_2
group_col = 'Condition2'
# group_col = 'Date_Con_Rep'
value_col = 'Disp_to_mid'
group_col_ls=[]
color_dict={}
color_map = 'plasma'
# t_inv_overtime = [0,30]
t_max = 30    # min
time_inv = 5  # min
n_col = 2
sharey=True
fs=25
y_lim=[]
frame_rate = 20
lw= 2
folder=None
'''
比较value_col的均值随时间变化的组内差异（按照group_col分组，组内可能按照group_id不同比较）
group_cols:用于分组的列的列表（可排序后再输入），相当于原本按照condition分组的conds
group_col_ls用于排序
value_col：指定可视化的参数列，比如CTX_left
n_col: 自适应子图数量，n_col指定列数
sharey=True
'''

# 根据所有group_col的数量生成一张大图
if not len(group_col_ls):
    # 如果指定了组列表，直接使用
    group_col_ls = df[group_col].unique()
    
# 子图数量与分组数量相同
num_fig = len(group_col_ls)

# 去除turn
df_run = df[df.Event==0]
# 颜色
if not color_dict:
    color_dict  = map_color_dict(group_col_ls, color_map)
    
# 设置时间区间
df_run,_ = set_time_intervals(df_run, t_max*60, time_inv*60)   
df_time = df_run.copy()
df_time['Period_label'] = df_time['Period_label'].astype(float)
# df_time_p = df_time[(df_time['Period_label']>=t_inv_overtime[0]) & (df_time['Period_label']<=t_inv_overtime[1])]

# 根据指定的group_col_ls排序
if len(group_col_ls):
    df_time = Sort_by_GroupCol(df_time, group_col,group_col_ls)

# 可视化
n_row = 1
n_col = num_fig
fig,axes = plt.subplots(n_row, n_col, figsize = (5.5*n_col,5*n_row), sharex = True, sharey = sharey)
ax = axes.ravel()

for i, (g, df_f) in enumerate(df_time.groupby(group_col)):
 
    df_mean_ctx_by_g = df_f.groupby(['Date_Con_Rep', 'Period_label']).agg({value_col:'mean'}).reset_index()

    sns.lineplot(df_mean_ctx_by_g,x='Period_label',
                y=value_col, ax = ax[i],
                color=color_dict[g], markers=True, style='Date_Con_Rep',
                 linewidth=lw
                )
    ax[i].set_title(g, fontsize=fs*1.25, pad = fs)
    ax[i].set_xlabel('Time(s)', fontsize=fs, labelpad = fs*0.5)
    ax[i].legend(loc='upper center', bbox_to_anchor=(0.5, -0.3),fontsize=fs*0.85, ncol=1, frameon=False)
    ax[i].grid(True,linestyle=':',color='grey',alpha=0.6)
    ax[i].tick_params(labelsize=fs*0.85)
    # 画y=0
    ax[i].axhline(y=0, color='r', linestyle='dashed', alpha = 0.5)
    ax[i].set_ylabel(value_col, fontsize=fs, labelpad = fs*0.5)
    if y_lim:
        ax[i].set_ylim(y_lim[0], y_lim[1])

if save:
#  保存图片
    title_name = f'{kw}_CTX-leftOverTimeWihtinG.png'
    file_path = os.path.join(p_f_fig, title_name)
    plt.savefig(file_path,bbox_inches='tight')
else:
    plt.show()
    
   

# 神经数据清洗
1. 通过可视化神经trace裁标记明显异常的区间
+ Z轴严重偏离
+ 追丢
+ 线虫瘫痪，运动状态异常

2. 标记信号异常的神经元
+ 无信号神经元
+ 假信号(信号波动过于剧烈)的神经元


## 可视化Calcium traces/Behavior选择异常时间区间

In [ ]:
def draw_calcium_curve(
    calcium_intensity, 
    smooth_kernel=None, 
    save=True, 
    scale=1, 
    percentile_threshold=None  # 用 99 分位筛选阈值
):
    colors = ['orangered', 'blue', 'limegreen', 'purple', 'gold', 'cyan', 'magenta', 'coral', 'skyblue', 'orange']
    import cv2

    plot_idx = 0  # 紧凑排列
    yticks_labels = []  # 原 neuron ID

    # 先统计要画的 neuron 索引
    keep_indices = []
    for i in range(calcium_intensity.shape[0]):
        line_data = calcium_intensity[i] * scale
        if percentile_threshold is not None:
            perc99 = np.percentile(line_data, 99)
            if perc99 <= percentile_threshold:
                continue
        keep_indices.append(i)

    # 根据保留数量动态计算画布大小
    num_to_plot = len(keep_indices)
    fig_h = int(10 * round(num_to_plot / 10)) if num_to_plot > 0 else 5
    fig_w = int(0.7 * round(calcium_intensity.shape[1] / 100))
    plt.figure(figsize=(fig_w, fig_h))

    # 再画图
    for i in keep_indices:
        line_data = calcium_intensity[i] * scale
        color = colors[plot_idx % len(colors)]

        if smooth_kernel:
            smooth_line_data = cv2.blur(calcium_intensity[i],  (1,smooth_kernel)) * scale
        else:
            smooth_line_data = cv2.blur(calcium_intensity[i], (1, 7)) * scale

        plt.plot(smooth_line_data + plot_idx, color=color, linestyle='-', label='A' if plot_idx == 0 else "", linewidth=1)
        plt.scatter(
            x=np.arange(line_data.shape[0]), 
            y=line_data + plot_idx, 
            color=color, s=1, 
            label='A' if plot_idx == 0 else ""
        )

        # if i in bound:
        #     plt.axhline(y=plot_idx - 0.2, color='r', linestyle='dashdot', linewidth=4)

        yticks_labels.append(i)
        plot_idx += 1
    
    # 坐标轴设置
    plt.xlabel("Time point", fontsize=20)
    plt.ylabel("Intensity", fontsize=20)
    plt.ylim(bottom=-1, top=plot_idx + 1)
    plt.yticks(np.arange(0, plot_idx, 1), labels=yticks_labels, fontsize=20)
    plt.xticks(np.arange(0, calcium_intensity.shape[1] + 1, 200), fontsize=20, rotation=45)
    plt.tight_layout()
    print(plot_idx)
    # 保存文件
    if save:
        filename = "cluster_calcium_curve"
        if smooth_kernel:
            filename += f"(smooth_{smooth_kernel})"
        if percentile_threshold is not None:
            filename += f"_p99_{percentile_threshold}"
        plt.savefig(f"{signal_save_path}/{filename}.png")
        
    return keep_indices

# 标记前进后退和Turning行为

## 函数定义
+ 找事件起始点
+ 开闭操作
+ 保留正常的事件epoch
+ 可视化事件和behavioral traces

In [ ]:
def get_start_points(forward_reversal_series):
    # 创建一个 DataFrame，只包含原始列，方便处理
    df = pd.DataFrame({'forward_reversal_series': forward_reversal_series})
#     df['rev_start'] = 0

    # 差分找变化点
    df['diff_qr'] = df['forward_reversal_series'].diff()
#     plt.plot(diff_qr)
#     print(len(df.loc[diff_qr == 1, 'rev_start']), len(df.loc[diff_qr == 1, 'rev_start'])/len(df))
    # 标记起始和结束
    df.loc[:,'rev_start'] = np.nan
    df.loc[df['diff_qr'] == 1, 'rev_start'] = 1
    df.loc[df['diff_qr'] == -2, 'rev_start'] = 2    # 从nan转到reverse
    df.loc[df['diff_qr'] == -1, 'rev_start'] = 3
    df.loc[df['diff_qr'] == -3, 'rev_start'] = 4    # 从nan转到forward
    # 特殊情况处理：开头或结尾就是 1
#     if df['quick_reversal'].iloc[0] == 1:
#         df.loc[df.index[0], 'quick_reversal_point'] = 1
#     if df['quick_reversal'].iloc[-1] == 1:
#         df.loc[df.index[-1], 'quick_reversal_point'] = 2

    return df['rev_start'].values

In [ ]:
def Opening_Closing(vector,structure_size_open=10, structure_size_close=5):
    '''利用开操作删除和合并后退
    vector: 一维向量
    structure_size的第一和第二个值分别控制开和闭的大小'''
    # 可视化重定向的位置，找到索引
#     vector = df_PCA_motion.forward.values

    # 将一维向量扩展为二维（例如 1 行或 1 列）
    heatmap_data = vector[np.newaxis, :]  # 变为 1 行 N 列
    print(heatmap_data)
    # Define two colors
    colors = ['blue', 'yellow']
    # Create a ListedColormap
    two_color_cmap = ListedColormap(colors)

    # Define the size of the structuring element
#     structure_size_open = 10
    # Perform opening
    '''这里opening的效果是合并！'''
    opened_vector = grey_opening(heatmap_data, size=structure_size_open)

    # Perform closing
    '''这里的close的效果是擦除！'''
    vector = grey_closing(opened_vector, size=structure_size_close)

#     plt.figure(figsize=(20, 1))  # 设置热图大小
#     plt.imshow(vector, cmap=two_color_cmap, aspect='auto')  # 选择 colormap
#     plt.colorbar(label='Value')  # 添加颜色条
#     plt.title('Removed small turnings')
#     plt.xticks()  # 设置x轴刻度
#     plt.yticks([])  # 隐藏y轴刻度
#     # plt.title("Heatmap of 1D Vector")
#     plt.show()
    return vector

In [ ]:
def label_event_segments(df, start_indices, event_col = "forward_opcl",
                          accept_ab_end = True,
                          output_col='forward_sel'):
    """
    根据开始和结束标记对DataFrame进行段落标注。
    注意这个代码需要匹配对于forward-reverse transition的四种情况的标记语法

    参数：
    - start_flags：开始的标记值列表
    - accept_ab_end: 接收异常结尾
    - output_col：新列的名字
    - debug：是否打印调试信息
    """
    # 初始化输出列
    df[output_col] = 0

    # 获取 end 索引及标签

    end_nor_indices = df[df['rev_start_opcl']==3].index
    end_abnor_indices = df[df['rev_start_opcl']==4].index
    
    rev_end_idx = []
    for s_idx in start_indices:
        nearest_nor_end = end_nor_indices[bisect.bisect_left(end_nor_indices, s_idx)]
        nearest_abnor_end = end_abnor_indices[bisect.bisect_left(end_abnor_indices, s_idx)]
        print('起始index：',s_idx)
#         if not end_candidates:
#             if debug:
#                 print(f'⚠ No end found for start at {s_idx}')
#             continue
        if nearest_abnor_end <= nearest_nor_end:
            print(f'abnormal{nearest_abnor_end} < normal:{nearest_nor_end}')
            # 异常结束
            if accept_ab_end:
                df.loc[s_idx+1:nearest_abnor_end,output_col] = 1
                rev_end_idx.append(nearest_abnor_end)
                print('保存abnormal起始结束：',s_idx, nearest_abnor_end)
            else:
                pass
        else:
            df.loc[s_idx+1:nearest_nor_end,output_col] = 1
            rev_end_idx.append(nearest_nor_end)
            print('保存normal起始结束:', s_idx, nearest_nor_end)
    return df, rev_end_idx

In [ ]:
def plot_custom_timeseries(
    df,
    time_col,
    line_cols=None,
    line_color_by_col=None,
    scatter_col=None,
    colorbar1_col=None,
    colorbar2_col=None,
    nanbar1_col=None,
    vertical_line_col=None,
    triangle_times=None,
    plot_line=None,
    plot_scatter=None,
    plot_colorbar1=None,
    plot_colorbar2=None,
    plot_colorbar3=None,
    plot_vertical_lines=None,
    plot_triangles=None,
    figsize=(20, 4),
    title=None,
    time_range=None
):
    def col_should_plot(col, flag):
        if flag is None:
            return col is not None and col in df.columns
        return flag

    if time_range:
        start, end = time_range
        df = df[(df[time_col] >= start) & (df[time_col] <= end)].reset_index(drop=True)

    draw_bar1 = col_should_plot(colorbar1_col, plot_colorbar1)
    draw_bar2 = col_should_plot(colorbar2_col, plot_colorbar2)
    draw_bar3 = col_should_plot(nanbar1_col, plot_colorbar3)
    num_bars = int(draw_bar1) + int(draw_bar2) + int(draw_bar3)

#     fig, ax_main = plt.subplots(figsize=figsize, constrained_layout=True)
    fig,ax = plt.subplots(figsize=figsize)
    if title:
        ax.set_title(title)
#     if num_bars > 0:
    fig.set_constrained_layout(False)
    gs = GridSpec(len(line_cols)+1, 1, height_ratios=[1.0]*len(line_cols)+[0.3 * num_bars], wspace=0.3, hspace=0.25, figure=fig)
    # 可能画多个折线图
    ax_bar = fig.add_subplot(gs[-1], sharex=ax)
    
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.set_yticks([])                  # 去除刻度
    ax.set_yticklabels([])            # 去除刻度标签
    ax.tick_params(axis='y', left=False)  # 去除刻度线
    # Plot line
    i = 0
    for line_col in line_cols:
        ax_main = fig.add_subplot(gs[i])
        if isinstance(line_col, str):
            line_col_list = [line_col]
        else:
            line_col_list = line_col

        for col in line_col_list:
            if line_color_by_col and line_color_by_col in df.columns:
                
                # 分段上色
                segment_time, segment_value, current_color = [], [], None
                handles_drawn = {'red': False, 'blue': False}

                for t, y, c in zip(df[time_col], df[col], df[line_color_by_col]):
                    if current_color is None:
                        current_color = c
                    if c != current_color:
                        if segment_time:
                            color = 'red' if current_color == 1 else 'blue'
                            label = 'reversal' if color == 'red' and not handles_drawn['red'] else None
                            ax_main.plot(segment_time, segment_value, color=color)
                            handles_drawn[color] = True
                        segment_time, segment_value = [t], [y]
                        current_color = c
                    else:
                        segment_time.append(t)
                        segment_value.append(y)
                if segment_time:
                    color = 'red' if current_color == 1 else 'blue'
                    label = 'reversal' if color == 'red' and not handles_drawn['red'] else None
                    ax_main.plot(segment_time, segment_value, color=color)
            else:
                ax_main.plot(df[time_col], df[col])
            if "smoothed_" in col:
                col = col.replace("smoothed_","")
            ax_main.set_ylabel('\n'.join(col.split('_')), rotation=0,labelpad = 15, va='center')
            if time_range:
                ax_main.set_xlim(time_range)
            else:
#                 print(df[time_col].min(), df[time_col].max())
                ax_main.set_xlim(df[time_col].min(), df[time_col].max())
            ax_main.set_xticks([])
            
            # Triangle markers
            if len(triangle_times) and (plot_triangles is None or plot_triangles):
                y_min, y_max = ax_main.get_ylim()
                triangle_y = y_max + 0.05 * (y_max - y_min)
                for t in triangle_times:
                    ax_main.plot(df.loc[t,time_col], triangle_y, marker='v', color='purple', markersize=8)
                    ax_main.axvline(x=df.loc[t,time_col], color='black', linestyle='--', linewidth=1)
                ax_main.set_ylim(y_min, triangle_y + 0.1 * (y_max - y_min))
        i += 1

    # Plot scatter
    if col_should_plot(scatter_col, plot_scatter):
        ax_main.scatter(df[time_col], df[scatter_col], color='r', s=2, label=scatter_col)

    # Vertical lines
    if col_should_plot(vertical_line_col, plot_vertical_lines):
        for vt in df[df[vertical_line_col] == 1][time_col]:
            ax_main.axvline(x=vt, color='green', linestyle='--', linewidth=1)
    # Colorbar
    handles, labels = ax_main.get_legend_handles_labels()
    if ax_bar:
        bar_data = []
        if draw_bar3:
            nan_mask = df[nanbar1_col].isna().values
            bar_data.append((np.where(nan_mask, 1, 0)[np.newaxis, :], mcolors.ListedColormap(['black', 'red'])))
        if draw_bar2:
            vals2 = df[colorbar2_col].fillna(0).astype(int).values[np.newaxis, :]
            bar_data.append((vals2, mcolors.ListedColormap(['black', 'yellow'])))
        if draw_bar1:
            vals1 = df[colorbar1_col].fillna(0).astype(int).values[np.newaxis, :]
            bar_data.append((vals1, mcolors.ListedColormap(['black', 'yellow', 'red'])))

        bar_height = 1.0 / num_bars
        for i, (vals, cmap) in enumerate(bar_data):
            extent = [df[time_col].iloc[0], df[time_col].iloc[-1], i * bar_height, (i + 1) * bar_height]
            ax_bar.imshow(vals, aspect='auto', cmap=cmap, extent=extent)

        bar_labels = []
        if draw_bar3: bar_labels.append('untrust')
        if draw_bar2: bar_labels.append('reversal(after)')
        if draw_bar1: bar_labels.append('reversal(before)')

        ax_bar.set_yticks([(i + 0.5) / num_bars for i in range(num_bars)])
        ax_bar.set_yticklabels(bar_labels)
        ax_bar.set_ylim(0, 1)
        ax_bar.tick_params(left=False)
        ax_bar.set_xlabel('Time')

        # Legend patch
        patches = []
        if draw_bar1: 
            patches.append(Patch(color='yellow', label='reversal'))
        if draw_bar2: patches.append(Patch(color='red', label='untrusted frames'))
        handles += patches
        labels += [p.get_label() for p in patches]

    if handles:
#         ax_main.legend(handles=handles, labels=labels, loc='upper right')
        ax_main.legend(loc='upper left',handles=handles, labels=labels, bbox_to_anchor=(1.01, 0.85), borderaxespad=0.)

#     ax_main.set_ylabel('Value')
    
#     plt.tight_layout()
    return fig, ax_main


## 数据导入与行为事件计算

### Import data and smooth

In [ ]:
p_f = r'Z:\data space+\C. elegans chemotaxis\2025_WBI\good_WBI\no_binary\20250116_4.5g-ov_05'

# 行为数据导入
'''激光段的行为数据，帧率为30Hz左右'''
# if not midline info, use mot_vid_cut
mot_mid_cut = [f for f in os.listdir(p_f) if 'mot_midline_cut' in f elif 'mot_vid_cut' in f][0]
df_motion_cut = pd.read_csv(os.path.join(p_f, mot_mid_cut))
# 行为数据进行平滑和分割前进后退
# 对数据进行平滑
# 平滑速度
window_size = 15
# 计算移动平均值
df_motion_cut['smoothed_speed'] = df_motion_cut['speed'].rolling(window=window_size, min_periods=1).mean()
# 平滑角速度
window_size = 15
# 计算移动平均值
df_motion_cut['smoothed_agl_speed'] = df_motion_cut['agl_speed'].rolling(window=window_size, min_periods=1).mean()
# 平滑ctx
window_size = 15
# 计算移动平均值
df_motion_cut['smoothed_CTX'] = df_motion_cut['CTX_left'].rolling(window=window_size, min_periods=1).mean()

### Calculate curvature using backbone info

In [ ]:
# # 计算头部曲率head curvature

# df_motion_cut['all_paths'][0]

### visualize angle_m and separate forward and reverse by threshold
+ separate forward and reverse
+ calculate heading velocity

In [ ]:
# 根据角速度分隔前进后退状态
agl_threshold = 100
df_motion_cut.loc[:,'forward'] = 0
df_motion_cut.loc[df_motion_cut['angle_m']>=agl_threshold,'forward'] = 1
df_motion_cut.loc[df_motion_cut['angle_m'].isna(),'forward'] = np.nan
# df_PCA_motion.loc[df_PCA_motion['choose_frame']!=1,'forward'] = 3
label_rev = get_start_points(df_motion_cut['forward'].values)
df_motion_cut['rev_start'] = label_rev
# 计算前向速度
df_motion_cut['cos_m'] = np.cos(np.radians(df_motion_cut['angle_m']))
df_motion_cut['head_velocity'] = df_motion_cut['cos_m']*df_motion_cut['smoothed_speed']
df_motion_cut['head_speed'] = df_motion_cut['head_velocity'].abs()


In [ ]:
# 对行为数据分割的前进后退进行开闭操作
# 开操作：去除噪声，structure_size = 0.65s * 30 = 19.5 ~ 20
# 闭操作：合并两个 reversal: structure_size = 40
structure = np.ones(50)
opened_first = binary_opening(df_motion_cut['forward'].astype(bool), structure=structure).astype(int)

structure = np.ones(40)
closed = binary_closing(opened_first, structure=structure).astype(int)

# opened = binary_opening(closed, structure=structure).astype(int)
df_motion_cut['forward_opcl'] = closed
df_motion_cut.loc[df_motion_cut['angle_m'].isna(),'forward_opcl'] = 3  # 还是将没有通过筛选的处理为3，灰色
# 开闭后对节点进行检查
label_rev = get_start_points(df_motion_cut['forward_opcl'].values)
df_motion_cut['rev_start_opcl'] = label_rev
rev_start_idx = df_motion_cut[df_motion_cut['rev_start_opcl']==1].index
rev_end_idx = df_motion_cut[df_motion_cut['rev_start_opcl']==3].index

event_idx_ls = rev_start_idx
draw_window = 100  # 在节点前后画图的时间窗

for i, idx in enumerate(event_idx_ls):
    start_idx = idx-draw_window
    end_idx = idx+draw_window
    df_i = df_motion_cut.loc[start_idx:end_idx, :]
    plt.figure()
    plt.grid(ls='dashed', c = 'grey', alpha = 0.5)
    plt.axis('equal')
    plt.title('event_idx:'+str(idx))
    plt.scatter(df_i.loc[idx,'X'], df_i.loc[idx,'Y'], s = 100, marker = '*', color='green')
    colors = df_i['forward_opcl'].map({0: 'blue', 1: 'red', 3:'grey'})
    plt.scatter(df_i['X'], df_i['Y'],c = colors ,s=5)
    plt.show()

In [ ]:
rev_start_idx

In [ ]:

rev_end_idx

In [ ]:
# 0116-05 start point删除: 8788,21257,22302,23661,27812
rev_start_idx = [ 846,  2876,  3863,  4627,  7388,  8386,  9327, 11892, 12063,
       13455, 16554, 19578, 20896, 23113, 26328, 27482]

In [ ]:
df_motion_cut_sel, rev_end_idx = label_event_segments(df_motion_cut, rev_start_idx, event_col = "forward_opcl",
                          accept_ab_end = True,
                          output_col='forward_sel')

In [ ]:
# 可视化行为数据与前进后退分割(30Hz)
rev_end_idx = df_motion_cut[df_motion_cut['rev_start_opcl']==3].index
rev_start_idx = df_motion_cut[df_motion_cut['rev_start_opcl']==1].index

df_motion_cut_reset = df_motion_cut.reset_index().rename(columns={'index': 'Index'})

fig, ax = plot_custom_timeseries(
    df=df_motion_cut_reset,
    time_col='Index',
    line_cols=["smoothed_CTX", "smoothed_agl_speed", "smoothed_speed",'head_velocity'],
    colorbar1_col='forward_opcl',
    colorbar2_col='forward_sel',
    nanbar1_col='angle_m',
#     time_range=(750,900),
    triangle_times=rev_start_idx,       # 指定几个时间点画三角形
    title='Motion(30Hz) Labeled with Reversal End'
)

# 备用

特殊文件单独处理

In [ ]:
# # 单独处理：没有标记四个顶点坐标
# # 输入每一对txt文件所在的的文件夹
# # file_paths = [r'Z:\data space+\C. elegans chemotaxis\20250219_WBI\20250219-1']
# file_paths = [r'Z:\data space+\C. elegans chemotaxis\20250219_WBI\trash\20250219_4.5g-ov-nan_3']
# # 放入所有包含单个实验文件的文件夹

# position_file_names  = ['s1']
# vol_time_file_names = ['t1']

# # position_file_names = ['s1', 's1', 's2', 's1', 's1']
# # vol_time_file_names = ['t1','t1','t2','t1','t1'] 

# required_files = ['stage_data.txt', 'c1.txt','s-data.txt']   # 数据处理需要的文件
# required_files_set = set(required_files)
# write = True    # 是否写出文件
# hlf_spd_inv = 0.5
# hlf_agl_spd_inv = 0.2
# hlf_agl_inv = 1
# Pos_CTX_vec = [-1,0]
# t_crit = 3.8
# min_agl_spd = 75
# cut_window = 2000
# half_width = 20  #mm 琼脂片宽度的一半


# for i, p_f in enumerate(file_paths):
    
#     files = [f for f in os.listdir(p_f) if '.txt' in f ]
#     # 转化为集合进行比较
#     files_set = set(files)
#     # 判断文件夹中的文件是否完全包含自定义列表中的元素
#     is_contained = required_files_set.issubset(files_set)
#     # 找出missing_files
#     missing_files = [file for file in required_files if file not in files]
    
#     if is_contained:
#         print(f'\n\n=======开始处理{os.path.basename(os.path.normpath(p_f))}中文件，\n包含:{files}')
#         # 预处理：时间戳对齐
#         Align_timestamp(p_f)
#         for f in files:
#             if (len(position_file_names)==1):
#                 if position_file_names[0] in f:
#                     # 运动参数位置文件
#                     column_names = ['Time', 'X', 'Y']
#                     df_motion = pd.read_csv(os.path.join(p_f, f),sep=r'\s+', header=None,
#                                             names=column_names)
#                     df_motion = df_motion.astype(float)

# #                     # 转换坐标，统一x轴坐标起点为0，终点为4.5
# #                     df_label, df_motion = Realign_coordinate(p_f, df_motion)
# #                     # 打印轨迹


#                 elif vol_time_file_names[0] in f:
#                     # volume时间戳文件
#                     column_names = ['Vol_Time']
#                     df_vol_time = pd.read_csv(os.path.join(p_f, f),sep=r'\s+', header=None, names=column_names)

#             elif (len(position_file_names)>1):
#                 if position_file_names[i] in f:
#                     # 运动参数位置文件
#                     column_names = ['Time', 'X', 'Y']
#                     df_motion = pd.read_csv(os.path.join(p_f, f),sep=r'\s+', header=None, names=column_names)
#                     df_motion = df_motion.astype(float)

#                     # 转换坐标，统一x轴坐标起点为0，终点为4.5
#                     df_label, df_motion = Realign_coordinate(p_f, df_motion)
#                     # 打印轨迹


#                 elif vol_time_file_names[i] in f:
#                     # volume时间戳文件
#                     column_names = ['Vol_Time']
#                     df_vol_time = pd.read_csv(os.path.join(p_f, f),sep=r'\s+', header=None, names=column_names)


#         # 查找在motion.Time中最近值和索引，加入df_vol_time中
#         nearest_times = []
#         nearest_indices = []
#         for vol_time in df_vol_time['Vol_Time']:
#             # 计算时间差的绝对值
#             differences = np.abs(df_motion['Time'] - vol_time)
#             # 找到最小差值的索引
#             nearest_idx = np.argmin(differences)
#             # 保存最近时间值和索引
#             nearest_times.append(df_motion.loc[nearest_idx, 'Time'])
#             nearest_indices.append(nearest_idx)

#         # 将结果存入 DataFrame
#         df_vol_time['Nearest_Time'] = nearest_times
#         df_vol_time['Nearest_Index'] = nearest_indices


#         # 根据运动位置信息抽取运动参数

#         mean_frame_rate = np.mean(1/np.diff(df_motion.Time.values))
#         df_motion_als = Sliding_CTX_calation(df_motion, Pos_CTX_vec, hlf_spd_inv, hlf_agl_spd_inv, hlf_agl_inv,
#                                           window_size=100, frame_rate=mean_frame_rate)
#         df_motion_pt = Label_Pirouettes_by_id(df_motion_als, t_crit=t_crit, min_agl_spd=min_agl_spd)
#         # visualization check
#         Scatter_Pirouette(df_motion_pt, folder=p_f+'\\', x_lim = [0,40])
#         Scatter_Pos_Grad_Dir(df_motion_pt, Pos_CTX_vec, folder=p_f+'\\', x_lim = [0,40])
#     #     cut_and_draw(df_motion_pt, n = cut_window)

#         # 将df_motionz中的运动参数按照索引对应到神经数据时间戳上
#         columns_to_add = ['X', 'Y', 'speed', 'agl_speed', 'CTX_left', 'Event', 'Reorientation']
#         for col in columns_to_add:
#             df_vol_time[col] = df_vol_time['Nearest_Index'].map(df_motion_pt[col])

#         # 加入condition
#         folder_name = os.path.basename(os.path.normpath(p_f))
#         features = folder_name.split('_')
#         df_motion_pt['Date'] = features[0]
#         df_vol_time['Date'] = features[0]
#         df_motion_pt['Rep_id'] = features[-1]
#         df_vol_time['Rep_id'] = features[-1]
#         condition_ls = features[-2].split('-')
#         print(f'日期:{features[0]}, 编号：{features[-1]}\n Conditions:{condition_ls}')
#         for i, c in enumerate(condition_ls):
#             if len(condition_ls) > 1:
#                 con_col = 'Condition'+str(i)
#                 df_motion_pt[con_col] = c
#                 df_vol_time[con_col] = c
#             else:
#                 df_motion_pt['Condition'] = features[-2]
#                 df_vol_time['Condition'] = features[-2]
#         if len(condition_ls) > 1:
#             df_vol_time['Date_Con_Rep'] = features[-2]

#         df_motion_pt['Disp_to_mid'] = df_motion_pt['X']-half_width
#         df_vol_time['Disp_to_mid'] = df_vol_time['X']-half_width
#         # 写出文件
#         if write:
#             # folder_name为当前数据日期批次
#             fn_motion_output = folder_name+'_MotionExtract.csv'
#             df_motion_pt.to_csv(os.path.join(p_f, fn_motion_output), index = False)
#             print(f'写出抽取的运动参数文件到:{folder_name}/{fn_motion_output}')
#             fn_nt_output = folder_name+'_NeuralAlignMotion.csv'
#             df_vol_time.to_csv(os.path.join(p_f, fn_nt_output), index = False)
#             print(f'写出与神经数据时间戳对应的运动参数文件到:{folder_name}/{fn_nt_output}')
#     else:
#         print(f'{os.path.basename(os.path.normpath(p_f))}中缺少数据处理必须的文件如下：\n' + "\n".join(missing_files))
#         continue

以不合并所有数据的方式作图，优点是节省内存，方便，缺点是不利于排序和组间比较

In [ ]:
def get_data_subfolders(file_path_all, nokey_1 = 'trash', nokey_2 ='*'):
    '''
    file_path_all：输入包含所有一级文件夹的地址的list
    （每个一级文件夹中包含单个日期下的所有数据文件的子文件夹）
    '''
    file_paths = []
    for fir_p_f in file_path_all:
        file_paths += [os.path.join(fir_p_f,f_p) for f_p in os.listdir(fir_p_f) if
                      (os.path.isdir(os.path.join(fir_p_f, f_p))) & (nokey_1 not in f_p) & (nokey_2 not in f_p)]
    return file_paths

In [ ]:
file_path_all = [r'Z:\data space+\C. elegans chemotaxis\20250221_WBI', r'Z:\data space+\C. elegans chemotaxis\20250219_WBI']
file_paths = get_data_subfolders(file_path_all)
print('所有用于作图的数据:\n'+"\n".join(file_paths))

## 轨迹分布

### 轨迹分布大图(所有数据画在一个图上)
+ 将所有包含单次数据的文件夹放在一个总文件夹中，file_path_all
+ 依次读出所有单次数据，不合并为一个df作图

In [ ]:

fs = 20
# 可视化所有轨迹
figure_0,ax = plt.subplots(figsize = (10,10))
# 定义要使用的 sequential colormaps
colormaps = ['Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
             'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
             'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn']

for i, p_f in enumerate(file_paths):
    
    
    
    # 设置截断colormap
    orig_cmap = mpl.colormaps[colormaps[i%len(colormaps)]]
    # 重新缩放 colormap，使最浅部分变深
    new_cmap = mcolors.LinearSegmentedColormap.from_list(
        "truncated_cmap", orig_cmap(np.linspace(0.3, 1, 256))  # 从 30% 处开始
    )
    
    
    
    for f in os.listdir(p_f):
        if 'NeuralAlignMotion' in f:
            df_motion = pd.read_csv(os.path.join(p_f, f))
    base_name = os.path.basename(p_f)
#     name,_= os.path.splitext(base_name)
    ax.scatter(df_motion['X'], df_motion['Y'], c = df_motion['Time'], cmap = new_cmap, 
               s = 1)
    first_valid_idx = df_motion['X'].first_valid_index()
    
    ax.scatter(df_motion.loc[first_valid_idx, 'X'], df_motion.loc[first_valid_idx, 'Y'], 
              s=30, c = 'r', marker='*')
    ax.scatter([], [], color=legend_color, label=base_name, s=5, marker='o')  # 仅用于图例
    ax.axvline(x=20, color = 'grey', ls = 'dashed', alpha = 0.2, lw = 2)
    # 定义图例的颜色
    legend_color = new_cmap(0.7)
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), markerscale=2, fontsize=20)
    ax.set_title('Trajectories', fontsize = fs*1.5)
    ax.tick_params(axis = 'both', labelsize = fs)
    ax.set_xlabel('mm', fontsize = fs)
    ax.set_ylabel('mm', fontsize = fs)
    ax.set_xlim([-1,41])
    ax.set_aspect('equal')
plt.show()

#### 分子图画轨迹
+ 将所有包含单次数据的文件夹放在一个总文件夹中，file_path_all
+ 依次读出所有单次数据，不合并为一个df作图

In [ ]:
# 

# file_path_all = r'Z:\data space+\C. elegans chemotaxis\20250221_WBI'
# file_paths = [os.path.join(file_path_all,f_p) for f_p in os.listdir(file_path_all)]

# 定义要使用的 sequential colormaps
colormaps = ['Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
             'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
             'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn']

# 计算每个子图的高度比例
height_ratios = []


for p_f in file_paths:
    for f in os.listdir(p_f):
        if 'NeuralAlignMotion' in f:
            df_motion = pd.read_csv(os.path.join(p_f, f))
            y_min = df_motion['Y'].min()
            y_max = df_motion['Y'].max()
            y_range = y_max - y_min
            
            # 增加 10% 的缓冲区
            padding = 0.1 * y_range
            y_min -= padding
            y_max += padding
            y_range_with_padding = y_max - y_min
            
            height_ratios.append(y_range_with_padding)

# 创建大图
figure_0, ax = plt.subplots(
    len(file_paths), 1, 
    figsize=(10, 3 * len(file_paths)), 
    sharex=True,  # 共享 x 轴
    gridspec_kw={'height_ratios': height_ratios}  # 子图高度自适应
)

# 找到所有子图中 Vol_Time 的最大值，作为 colorbar 的统一上限
vmax = max(
    pd.read_csv(os.path.join(p_f, f))['Vol_Time'].max()
    for p_f in file_paths
    for f in os.listdir(p_f)
    if 'NeuralAlignMotion' in f
)

# 绘制每个子图
for i, p_f in enumerate(file_paths):
    for f in os.listdir(p_f):
        if 'NeuralAlignMotion' in f:
            df_motion = pd.read_csv(os.path.join(p_f, f))
            
    # 设置colormap 
    # 设置截断colormap
    orig_cmap = mpl.colormaps[colormaps[i]]
    # 重新缩放 colormap，使最浅部分变深
    new_cmap = mcolors.LinearSegmentedColormap.from_list(
        "truncated_cmap", orig_cmap(np.linspace(0.3, 1, 256))  # 从 30% 处开始
    )
    
    # 标题使用文件名特征
    base_name = os.path.basename(p_f)
    
    # 绘制散点图
    scatter = ax[i].scatter(df_motion['X'], df_motion['Y'], c=df_motion['Vol_Time'], 
                            cmap=new_cmap, s=1, vmin=0, vmax=vmax)  # 统一 colorbar 上限
    ax[i].axvline(x=20, ls='dashed', c='r')  # 添加红色虚线
    
    # 标记起始位置
    first_valid_idx = df_motion['X'].first_valid_index()
    
    ax[i].scatter(df_motion.loc[first_valid_idx, 'X'], df_motion.loc[first_valid_idx, 'Y'], c = 'r', marker = '*', s = 25)
    
    # 间隔指定时间打印时间节点文字
    interval = 300   # s
    df_motion_dn = df_motion.dropna(subset=['X', 'Y', 'Vol_Time']).reset_index(drop = True)  # 删除包含 NaN 的行
    
    # 最后打印的时间序列元素是原始时间
    time_series = np.arange(df_motion_dn.Vol_Time.min(), df_motion_dn.Vol_Time.max(), interval)

    # 转为字符串，便于打印
    time_list = list(np.int32(time_series))
    time_list_str = list(map(str, time_list))

    # 取完整时间序列，扩展成矩阵
    time_vec = df_motion_dn.Vol_Time
    mat_time = np.tile(time_vec, (len(time_series), 1))

    # 使用广播机制，将矩阵每一行减去时间序列对应元素
    mat_time_subtracted = mat_time-time_series[:,np.newaxis]
    # 返回每一行最小值的索引
#     min_diff_idx = np.argmin(np.abs(mat_time_subtracted), axis=1)
    min_diff_idx = np.abs(mat_time_subtracted).argmin(axis=1)
#     ax[i].text(df_motion.loc[min_diff_idx, 'X'],df_motion.loc[min_diff_idx, 'Y'], time_list_str, c = 'r', fontsize = 5)
    # zip将两个列表打包
    for idx, time_str in zip(min_diff_idx, time_list_str):
        ax[i].text(df_motion_dn.loc[idx, 'X'], 
                   df_motion_dn.loc[idx, 'Y'], 
                   time_str+'s', c='r', fontsize=10)

    # 设置标题
    ax[i].set_title(base_name, fontsize=20)
    
    # 设置 y 轴范围（增加缓冲区）
    y_min = df_motion['Y'].min()
    y_max = df_motion['Y'].max()
    padding = 0.1 * (y_max - y_min)  # 10% 的缓冲区
    ax[i].set_ylim(y_min - padding, y_max + padding)
    ax[i].tick_params(axis = 'both', labelsize = 15)
    # 设置子图宽高比
    ax[i].set_aspect('equal')
    ax[i].set_ylabel('mm', fontsize = 15)
    if i == len(file_paths)-1:
        
        ax[i].set_xlabel('mm', fontsize = 15)
    # 为每个子图添加 colorbar
    divider = make_axes_locatable(ax[i])
    cax = divider.append_axes("right", size="5%", pad=0.1)  # colorbar 宽度为 5%，间距为 0.1
    cbar = plt.colorbar(scatter, cax=cax)
    cbar.set_label('Time(s)', fontsize = 15)

# 调整子图间距
plt.subplots_adjust(hspace=0.2)

plt.show()

#### 分子图画轨迹分布
+ 根据时间段出图，每一行为单个重复组的数据，每一列表示时间窗
+ 将所有包含单次数据的文件夹放在一个总文件夹中，file_path_all
+ 依次读出所有单次数据，不合并为一个df作图

In [ ]:
interval_min = 5  # min
interval_s = interval_min*60 # s
time_series = []
bin_size = 2.5 # mm
x_min, x_max = 0, 40  # 设定 X 的范围


# 定义要使用的 sequential colormaps
colormaps = ['Grey', 'Purple', 'Blue', 'Green', 'Orange', 'Red']
num_colormap = len(colormaps)
fs = 25

# 找出最大时间
max_times = []
for i, p_f in enumerate(file_paths):
    for f in os.listdir(p_f):
        if 'NeuralAlignMotion' in f:
            df_motion = pd.read_csv(os.path.join(p_f, f))
            max_times.append(df_motion.Vol_Time.max())
max_time = max(max_times)
print(f'所有数据中的最大时间{max_time}')

# 根据最大时间分时间段间隔
if not len(time_series):
    time_series = np.arange(0, max_time, interval_s)
    

# 创建大图
figure_0, ax = plt.subplots(
    len(file_paths), len(time_series)-1, 
    figsize=(5*(len(time_series)-1), 2.5 * len(file_paths)), 
    sharex=True,  # 共享 x 轴
    sharey=True  
)


# 绘制每个子图
for i, p_f in enumerate(file_paths):
    for f in os.listdir(p_f):
        if 'NeuralAlignMotion' in f:
            df_motion = pd.read_csv(os.path.join(p_f, f))
    
    # 提取文件名
    base_name = os.path.basename(p_f)
#     name, _ = os.path.splitext(base_name)
    
    # 分割X区间
    
    bins = list(np.arange(x_min, x_max + bin_size, bin_size))  # 创建区间边界
    bins_array = np.array(bins)
    labels = list((bins_array[:-1]+bins_array[1:])/2)
    # 使用 pd.cut() 将 X 分割成区间
    df_motion['X_bin'] = pd.cut(df_motion['X'], bins=bins,labels=labels, right=False)  # 右开区间
    
    for j in range(len(time_series)-1):
        
        df_mot_time = df_motion[(df_motion['Vol_Time']>=time_series[j])&(df_motion['Vol_Time']<time_series[j+1])]
    
        # 绘制柱状图
        sum_timepoint = df_mot_time['X_bin'].notna().count()
        df_gb_xbin = df_mot_time.groupby('X_bin').agg({'Vol_Time':'count'}).reset_index()
        df_gb_xbin = df_gb_xbin.rename(columns={'Vol_Time':'count'})
        df_gb_xbin['ratio_count'] = df_gb_xbin['count']/sum_timepoint
        ax[i][j].bar(df_gb_xbin['X_bin'],df_gb_xbin['ratio_count'], width=2,color = colormaps[i%num_colormap])
#         ax[i].axvline(x=20, ls='dashed', c='r')  # 添加红色虚线
        ax[i][j].tick_params(axis = 'both', labelsize = fs)
        if i == 0:
            # 设置标题
            begin_min = int(time_series[j]/60)
            end_min = int(time_series[j+1]/60)
            ax[i][j].set_title(f'{str(begin_min)}~{str(end_min)}min', fontsize=fs*1.25)
        if j == 0:
            y_labels = '\n'.join(base_name.split('_'))
            ax[i][j].set_ylabel(y_labels, fontsize = fs, rotation=60, labelpad = fs*1.5)
        # 设置x轴刻度范围
        # 设置 X 轴刻度间隔
        ax[i][j].set_xticks(np.arange(x_min, x_max+1, 10))  # 设置刻度间隔为 5mm
        ax[i][j].grid(True, color='grey', alpha = 0.5, ls = 'dashed')
        
        if i == len(file_paths)-1:
            ax[i][j].set_xlabel('mm', fontsize = fs, labelpad = fs*0.25)

# 调整子图间距
plt.subplots_adjust(hspace=0.2, wspace = 0.2)
plt.show()